In [2]:
import os
import pandas as pd

data_file = "input/raw_data.xlsx"
template_file = "input/wpqr_template.xlsx"

# Validate file existence
if not os.path.exists(data_file) or not os.path.exists(template_file):
    raise FileNotFoundError("❌ Required input files not found in the 'input/' folder.")

# Load Excel files
df_data = pd.read_excel(data_file)
df_template = pd.read_excel(template_file)

print("✅ Files loaded successfully.")



Saving Welding Data Recording.xlsx to Welding Data Recording.xlsx
Saving WPQR-FORMAT.xlsx to WPQR-FORMAT.xlsx


FileNotFoundError: ❌ Could not find the required input files.

In [2]:
# ✅ WPQR Data Extraction & Summary Tool (Colab-Compatible)
# ----------------------------------------------------------------------
# 📌 PURPOSE:
# This Python script is designed to automate the extraction of Welding Procedure Qualification Record (WPQR)
# data from a structured Excel file that contains multiple sheets of technical welding information.
#
# 👨‍🏭 The goal is to reduce manual effort and errors in pulling out key welding data.
#
# 🔎 The Excel workbook is expected to contain the following sheets:
# - 'Design Details': Welding design-related variables such as joint type and position
# - 'Weld Details': Parameters used in weld preparation like bevel angle, backing, etc.
# - 'Weld Data Record': Actual values recorded during the welding process such as current, voltage, heat input
# - 'WPS Range': Automatically calculated or defined ranges allowed for WPS based on WPQR
# - 'PWHT Details': Post Weld Heat Treatment parameters like soaking temperature and time
#
# 🧠 This tool helps:
# - Eliminate manual copying of welding data
# - Ensure consistency and repeatability
# - Prepare summaries for WPS and WPQR forms
#
# 🔰 Beginner Friendly: Each step is clearly documented so that even team members with no coding background can follow and understand.

# ----------------------------------------------------------------------
# 📦 Import required Python libraries
# ----------------------------------------------------------------------

import pandas as pd                          # 🔧 pandas is used to read, manage, and manipulate Excel data
from google.colab import files              # 📁 Allows us to upload files directly in Google Colab
from IPython.display import display, Markdown  # 🖥️ Used for clean, formatted display of outputs


# --------------------------------------------------------
# 🔹 STEP 1: Upload WPQR Excel File
# --------------------------------------------------------
# 📌 PURPOSE:
# This section allows the user to upload a WPQR Excel workbook from their local computer.
# The workbook should contain key welding sheets like:
#   - 'Design Details'
#   - 'Weld Details'
#   - 'Weld Data Record'
#   - 'WPS Range'
#   - 'PWHT Details'
#
# ✅ WHY THIS IS DONE:
# Google Colab runs in the cloud and doesn't have access to your local files.
# So, we use the Colab `files.upload()` function to manually upload the WPQR Excel file.
# Once uploaded, we extract the filename automatically so the user does not have to type it manually.

# ⬇️ Open the file upload dialog in Colab

# ✅ Use the WPQR data file uploaded earlier
# This replaces the old "files.upload()" logic
excel_path = data_file  # data_file was defined at the top during common file upload

# ✅ Show confirmation to the user
print(f"✅ Excel file loaded: {excel_path}")

# 📑 List all available sheets in the uploaded WPQR Excel file
print("\n📄 Available sheets in this Excel file:")
print(pd.ExcelFile(excel_path).sheet_names)



# --------------------------------------------------------
# 🔹 STEP 2: Load Required Sheets from the Excel
# --------------------------------------------------------
# 📌 PURPOSE:
# In this step, we load the individual worksheets from the uploaded WPQR Excel file.
# Each sheet contains specific types of welding data required for documentation and analysis.

# 🧾 SHEETS WE ARE LOADING:
# - 'Design Details': Contains joint type, welding position, base material info, etc.
# - 'Weld Details': Includes setup info like bevel angle, root gap, gas used, etc.
# - 'Weld Data Record': The actual measured welding data during the qualification (e.g., current, voltage, travel speed)
# - 'WPS Range': Pre-calculated or allowable qualified range based on WPQR tests

# ✅ WHY THIS IS DONE:
# Each of these sheets contains different parts of the WPQR. By loading them into separate pandas DataFrames,
# we can later extract, process, and summarize the values programmatically.

# 📥 Load the entire Excel workbook for selective access to individual sheets
xls = pd.ExcelFile(excel_path)  # Creates an ExcelFile object for efficient access

# 🧩 Load 'Design Details' sheet into a DataFrame
# This sheet usually includes general info such as joint design, welding position, and materials
df_design = pd.read_excel(xls, sheet_name='Design Details')

# 🧩 Load 'Weld Details' sheet
# This contains setup parameters like bevel type, backing, gas composition, etc.
df_weld = pd.read_excel(xls, sheet_name='Weld Details')

# 🧩 Load 'Weld Data Record' sheet
# This is the most important sheet that holds actual values recorded during welding trials
df_data_record = pd.read_excel(xls, sheet_name='Weld Data record')  # Note: Make sure exact sheet name matches case

# 🧩 Load 'WPS Range' sheet
# This sheet outlines the minimum and maximum values that define the valid WPS range based on qualification
df_range = pd.read_excel(xls, sheet_name='WPS Range')

# ✅ At this point:
# - We now have all the major WPQR sheets loaded into memory
# - Each sheet is stored in a separate DataFrame for clarity and modularity
# - These DataFrames can now be processed individually for extraction or summary

# 🔎 OPTIONAL: You can preview the top rows of each sheet like this (uncomment to use)
# display(df_design.head())
# display(df_weld.head())
# display(df_data_record.head())
# display(df_range.head())



# --------------------------------------------------------
# 🔹 STEP 3: Preview Raw Data for Validation
# --------------------------------------------------------
# 📌 PURPOSE:
# This step displays the contents of each loaded sheet from the Excel file.
# It helps you visually validate:
#   - The correct file was uploaded
#   - The expected sheets are present
#   - The sheet formatting and headers are correct
#
# 🧠 WHY THIS IS IMPORTANT:
# Before performing any data extraction or calculations, it's a good practice to preview the data.
# This allows the user (especially non-coders) to catch:
#   - Typo in sheet names
#   - Unexpected column headers
#   - Misaligned data
#
# ✅ Once confirmed, we can safely proceed with data extraction and summary logic.

# 🖥️ Loop over each sheet name and corresponding DataFrame
for sheet_name, df in zip(
    ["Design Details", "Weld Details", "Weld Data Record", "WPS Range"],  # Sheet names (for display only)
    [df_design, df_weld, df_data_record, df_range]                        # Corresponding DataFrames
):
    # 📄 Show the sheet name as a formatted Markdown heading in Colab
    display(Markdown(f"### ✅ Preview: {sheet_name}"))

    # 👁️ Display the actual table data (top rows by default)
    display(df)



# --------------------------------------------------------
# 🔹 STEP 4: Extract Only 'Applicable' Design Variables
# --------------------------------------------------------
# 📌 PURPOSE:
# The "Design Details" sheet often contains many variables,
# but only some of them are marked as "Applicable" for the specific welding procedure.
#
# ✅ GOAL:
# Extract only those rows where the "Applicable" column is explicitly marked as "Applicable"
# so we can focus on meaningful data for WPQR documentation.

# 🧠 WHY THIS IS NEEDED:
# This allows us to ignore unused or irrelevant design settings,
# which makes our summary cleaner and more relevant to the qualified procedure.

# --------------------------------------------------------
# 📥 STEP 4.1: Select the relevant data range from 'Design Details'
# --------------------------------------------------------

# 🔍 The actual design data starts from row 6 in Excel (which is index 5 in Python, since it's 0-based)
# We're only interested in columns E, F, and G which map to index positions 4, 5, and 6 respectively.
df_clean = df_design.iloc[5:, [4, 5, 6]]  # rows: from row 6 downwards, cols: E, F, G

# 🏷️ Assign readable column names to the DataFrame
df_clean.columns = ["Description", "Applicable", "Variable Details"]

# --------------------------------------------------------
# ✅ STEP 4.2: Filter rows where 'Applicable' column is marked as 'Applicable'
# --------------------------------------------------------

# 📌 Convert the "Applicable" column to lowercase text to handle any formatting inconsistencies (e.g., "Applicable", " applicable ", etc.)
# 📌 Then filter rows where it exactly matches "applicable"
df_applicable = df_clean[
    df_clean["Applicable"].astype(str).str.strip().str.lower() == "applicable"
].reset_index(drop=True)

# --------------------------------------------------------
# 🖥️ STEP 4.3: Display the filtered result
# --------------------------------------------------------

# ✅ This shows only the relevant, applicable design variables
display(Markdown("### ✅ Applicable Design Variables"))
display(df_applicable)


# --------------------------------------------------------
# 🔹 STEP 5: Extract Weld Setup Details (Vertical & Horizontal)
# --------------------------------------------------------
# 📌 PURPOSE:
# This step extracts **weld setup parameters** from the "Weld Details" sheet,
# which includes gas type, backing type, bevel angle, root gap, and more.
#
# 🔄 The sheet stores information in two formats:
#   - 🧍 Vertical Format → Rows 3–32 (columns B–C and D–E)
#   - 📐 Horizontal Format → Row 36 (values), Row 37 (labels)
#
# ✅ GOAL:
# Combine all the meaningful descriptions and values into a single list, then convert it to a table.

# --------------------------------------------------------
# 📥 STEP 5.1: Read the raw sheet with NO header row
# --------------------------------------------------------
# 🔧 We disable the header so we can refer to exact Excel row numbers easily.
# Excel starts counting at 1, Python at 0 — so row 3 in Excel is index 2 in Python.
df_weld_raw = xls.parse("Weld Details", header=None)  # Keep all rows intact, no header conversion

# Create a list to collect all extracted key-value pairs
data = []

# --------------------------------------------------------
# 🔍 STEP 5.2: Extract vertically formatted weld setup data
# --------------------------------------------------------

# Select Rows 3 to 32 (index 2 to 32), Columns B–E (index 1 to 4)
# These rows contain two pairs of columns: (B–C) and (D–E), like a two-column layout in Excel
part1_rows = df_weld_raw.iloc[2:33, 1:5]

# 🧹 Loop through each row and check both sides (left and right column pair)
for _, row in part1_rows.iterrows():
    # Left-side data: Columns B and C (index 1 and 2)
    if pd.notna(row[1]) or pd.notna(row[2]):
        data.append({
            "Description": str(row[1]).strip(),
            "Details": str(row[2]).strip()
        })

    # Right-side data: Columns D and E (index 3 and 4)
    if pd.notna(row[3]) or pd.notna(row[4]):
        data.append({
            "Description": str(row[3]).strip(),
            "Details": str(row[4]).strip()
        })

# --------------------------------------------------------
# 🔄 STEP 5.3: Extract horizontally formatted weld setup data
# --------------------------------------------------------

# 🧭 Horizontal format typically appears in Row 36 and 37 (index 35 and 36)
# Row 36 → contains values
# Row 37 → contains matching labels

try:
    # Get row 36 (index 35) and row 37 (index 36), columns B to E (index 1 to 4)
    details_row = df_weld_raw.iloc[35, 1:5].tolist()  # values
    desc_row = df_weld_raw.iloc[36, 1:5].tolist()     # labels

    # Zip through both rows and add valid pairs to our list
    for desc, detail in zip(desc_row, details_row):
        if pd.notna(desc) or pd.notna(detail):
            data.append({
                "Description": str(desc).strip(),
                "Details": str(detail).strip()
            })

except IndexError:
    # ⚠️ Handle case where horizontal rows might be missing (less than 37 rows)
    print("❌ Horizontal rows missing or unreadable in 'Weld Details' sheet.")

# --------------------------------------------------------
# 🧾 STEP 5.4: Convert collected data into a clean DataFrame
# --------------------------------------------------------

# ✅ Convert the list of dicts into a DataFrame for display and further processing
df_weld_details = pd.DataFrame(data)

# 📊 Show the extracted weld details table
display(Markdown("### ✅ Extracted Weld Details (All Rows)"))
display(df_weld_details)


# --------------------------------------------------------
# 🔹 STEP 6: Extract Run-Wise Weld Data (Recorded Values)
# --------------------------------------------------------
# 📌 PURPOSE:
# This step extracts the actual recorded data from each welding run.
# These are the most critical values for WPQR as they reflect what was truly done during qualification.

# ✅ WHY THIS IS IMPORTANT:
# Welding procedure qualification must record the actual parameters used during each run.
# This includes:
#   - Process type (GMAW, FCAW, etc.)
#   - Electrode/filler wire diameter
#   - Current (A), Voltage (V)
#   - Wire feed speed (WFS), Travel speed
#   - Heat input (KJ/mm)
#   - Metal transfer mode, Polarity
#
# These will be used later to calculate WPS qualification ranges.

# --------------------------------------------------------
# 🧾 STEP 6.1: Load 'Weld Data record' sheet with no headers
# --------------------------------------------------------
# ⛔ We disable the header row so we can reference Excel rows by number
df_full = xls.parse("Weld Data record", header=None)

# --------------------------------------------------------
# 🔍 STEP 6.2: Isolate data section starting from row 10 (Excel row 10 = index 9)
# --------------------------------------------------------
# The top 9 rows are likely headers, merged cells, or notes.
# We skip them and keep everything from row 10 onward.
df_data = df_full.iloc[9:, :].reset_index(drop=True)

# Rename all columns temporarily as 'col_0', 'col_1', ... since headers are missing
df_data.columns = [f"col_{i}" for i in range(len(df_data.columns))]

# --------------------------------------------------------
# ✅ STEP 6.3: Filter rows that contain actual weld run data
# --------------------------------------------------------
# We assume that valid weld run rows have a value in column 4 ("Run No" or ID)
# So we keep only those rows where col_4 is not blank
df_data = df_data[df_data["col_4"].notna()].reset_index(drop=True)

# --------------------------------------------------------
# 📊 STEP 6.4: Extract relevant columns and build the welding run summary table
# --------------------------------------------------------

# 🔁 We create a new DataFrame that pulls specific columns by index:
# These are based on the structure of the 'Weld Data record' sheet as observed.
welding_details = pd.DataFrame({
    "Run": df_data["col_4"],                                 # Run number
    "Welding Process": df_data["col_5"],                     # GMAW, FCAW, etc.
    "Filler material dia.(mm)": df_data["col_6"],            # Filler wire size
    "Current (A)": df_data["col_10"],                        # Amperage
    "Voltage (V)": df_data["col_13"],                        # Voltage
    "Type of current/ polarity": df_data["col_8"],           # AC/DC and polarity
    "Wire feed speed (m/min)": df_data["col_17"].astype(float).round(2),  # Wire feed speed
    "Travel speed (mm/Sec)": df_data["col_18"].astype(float).round(2),    # Travel speed
    "Heat input (KJ/mm)": df_data["col_20"].astype(float).round(2),       # Calculated heat input
    "Mode of metal Transfer": df_data["col_7"]               # Short arc, globular, spray, etc.
})

# --------------------------------------------------------
# 📢 STEP 6.5: Display the final extracted welding parameters
# --------------------------------------------------------

# 🖥️ This table shows one row per welding run and its associated parameters
display(Markdown("### ✅ Final Extracted Welding Details Table"))
display(welding_details)


# --------------------------------------------------------
# 🔹 STEP 7: Extract Summary Values (Temp, Heat, Gas Flow)
# --------------------------------------------------------
# 📌 PURPOSE:
# This step extracts high-level **summary values** from the "Weld Data Record" sheet.
# These include:
#   - 🔥 Maximum Interpass Temperature
#   - ⚡ Average Heat Input (middle value from 3 averages)
#   - 💨 Maximum Gas Flow Rates for Shielding and Backing gases
#
# ✅ WHY THIS IS IMPORTANT:
# These values are essential to document:
#   - Welding procedure compliance (interpass temp must not be exceeded)
#   - Heat input for metallurgical impact
#   - Gas flow for process shielding control

# --------------------------------------------------------
# 🌡️ STEP 7.1: Extract Maximum Interpass Temperature
# --------------------------------------------------------

# Column X (Excel) = Index 23 in Python (0-based)
# We're dropping any empty values and taking the highest recorded interpass temp
interpass_temp = df_full.iloc[9:, 23].dropna().max()

# --------------------------------------------------------
# ⚡ STEP 7.2: Extract Average Heat Input
# --------------------------------------------------------

# 🔎 Search for the row that contains "Average Heat Input" (case-insensitive)
# This might appear as a label in any column of the sheet, so we scan entire rows
row_avg_heat = df_full[
    df_full.astype(str).apply(lambda row: row.str.contains("Average Heat Input", case=False).any(), axis=1)
]

# ✅ If such a row is found, extract the 3 values to the right (columns 20–22),
# and take the middle one (usually the average of averages)
if not row_avg_heat.empty:
    # Convert the 3 values to float and round the middle one (index 1)
    heat_values = row_avg_heat.iloc[0, 20:23].astype(float).values
    avg_heat_input = round(heat_values[1], 2)
else:
    avg_heat_input = "Not available"  # Fallback if the row is missing

# --------------------------------------------------------
# 💨 STEP 7.3: Extract Maximum Shielding Gas Flow Rate
# --------------------------------------------------------

# Column Y (Excel) = Index 24
shielding_flow = df_full.iloc[9:, 24].dropna()  # Remove NaNs

# If values exist, take the max; otherwise, return "Not applicable"
shielding_max = shielding_flow.max() if not shielding_flow.empty else "Not applicable"

# --------------------------------------------------------
# 💨 STEP 7.4: Extract Maximum Backing Gas Flow Rate
# --------------------------------------------------------

# Column Z (Excel) = Index 25
backing_flow = df_full.iloc[9:, 25].dropna()

# Same logic as above: max or fallback text
backing_max = backing_flow.max() if not backing_flow.empty else "Not applicable"

# --------------------------------------------------------
# 🖥️ STEP 7.5: Display the Extracted Summary Values
# --------------------------------------------------------

# ✅ These values will be shown in Colab using Markdown formatting
display(Markdown("### ✅ Extracted Summary Values"))
display(Markdown(f"- **Interpass Temperature (°C):** {interpass_temp}"))
display(Markdown(f"- **Average Heat Input (middle value, KJ/mm):** {avg_heat_input}"))
display(Markdown(f"- **Gas Flow Rate (Shielding):** {shielding_max}"))
display(Markdown(f"- **Gas Flow Rate (Backing):** {backing_max}"))


# --------------------------------------------------------
# 🔹 STEP 8: Extract WPS Qualification Ranges
# --------------------------------------------------------
# 📌 PURPOSE:
# This step extracts the allowable **Welding Procedure Specification (WPS)** ranges
# from the 'WPS Range' worksheet. These ranges represent the **qualified limits** for:
# - Current (A), Voltage (V)
# - Wire Feed Speed (WFS), Travel Speed
# - Heat Input
#
# ✅ WHY THIS IS IMPORTANT:
# These ranges define the boundaries within which production welding must stay.
# They are derived from the actual weld runs in the WPQR.

# --------------------------------------------------------
# 🔧 STEP 8.1: Utility Function to Format Ranges
# --------------------------------------------------------

# 📦 This helper function takes min and max values and formats them into a readable range string
# For example: 100 and 120 becomes "100.00 - 120.00"
def format_range(min_val, max_val, digits=2):
    return f"{round(min_val, digits):.{digits}f} - {round(max_val, digits):.{digits}f}"

# --------------------------------------------------------
# 📥 STEP 8.2: Read 'WPS Range' Sheet (No Header)
# --------------------------------------------------------

# Load the WPS Range sheet using no header so we can accurately reference cell positions
df_wps = xls.parse("WPS Range", header=None)

# Skip the top 7 rows (headers or notes); keep data from row 8 onward (index 7)
df_data = df_wps.iloc[7:, :].reset_index(drop=True)

# Filter out any rows that don't have a value in column 4 (this is usually the "Run" or ID column)
df_data = df_data[df_data[4].notna()].reset_index(drop=True)

# --------------------------------------------------------
# 📊 STEP 8.3: Build the WPS Range Summary Table
# --------------------------------------------------------

# ✅ We now extract and organize key parameters into a summary table.
# Some columns are raw values; others are formatted as "min - max" ranges

wps_range_table = pd.DataFrame({
    "Run": df_data[4],                                # Welding run or segment number
    "Welding Process": df_data[5],                    # GMAW, FCAW, GTAW, etc.
    "Size of filler material(mm)": df_data[6],        # Electrode/wire diameter
    "Type of current/ polarity": df_data[7],          # AC/DC or DCEP/DCEN
    "Current (A)": df_data[8].round().astype(int).astype(str) + " - " + df_data[9].round().astype(int).astype(str),
    "Voltage (V)": df_data[10].round().astype(int).astype(str) + " - " + df_data[11].round().astype(int).astype(str),
    "Wire feed speed (m/min)": [
        format_range(a, b) for a, b in zip(df_data[12], df_data[13])
    ],
    "Travel speed (mm/Sec)": [
        format_range(a, b) for a, b in zip(df_data[14], df_data[15])
    ],
    "Heat Input (KJ/mm)": [
        format_range(a, b) for a, b in zip(df_data[16], df_data[17])
    ]
})

# --------------------------------------------------------
# 🖥️ STEP 8.4: Display the Final WPS Range Table
# --------------------------------------------------------

# 📋 This table gives a clear view of the qualified welding parameter ranges
display(Markdown("### ✅ Final Extracted WPS Range Table"))
display(wps_range_table)


# --------------------------------------------------------
# 🔹 STEP 9: Extract PWHT Details
# --------------------------------------------------------
# 📌 PURPOSE:
# PWHT = Post Weld Heat Treatment. This process is critical for relieving residual stress
# and improving mechanical properties after welding.
#
# ✅ GOAL:
# Extract a fixed set of Description–Detail pairs from specific rows and columns in the
# 'PWHT Detail' sheet of the WPQR Excel file.

# 🧠 WHY THIS IS NEEDED:
# These values summarize how the weld was heat-treated, including:
#   - Holding temperature
#   - Holding time
#   - Heating and cooling rates
#   - Soaking periods
#   - Furnace type
#
# They are essential for documenting conformance to codes and materials' requirements.

# --------------------------------------------------------
# 📥 STEP 9.1: Load the 'PWHT Detail' Sheet
# --------------------------------------------------------

try:
    # Read the full PWHT sheet without headers so we can access cells by exact row/column numbers
    df_pwht = pd.read_excel(xls, sheet_name='PWHT Detail', header=None)

    # --------------------------------------------------------
    # 📌 STEP 9.2: Define Which Rows to Extract
    # --------------------------------------------------------

    # We want to extract data from specific rows:
    # Excel Rows 7, 21, and 25–30 → Index 6, 20, 24 to 29 (0-based)
    rows_to_extract = [6, 20] + list(range(24, 30))

    # Create a list to store extracted values
    pwht_data = []

    # --------------------------------------------------------
    # 🔄 STEP 9.3: Loop Through Target Rows and Extract Columns C & D
    # --------------------------------------------------------

    # Column C (index 2) → Description
    # Column D (index 3) → Details

    for i in rows_to_extract:
        # ✅ Ensure the row index is valid within the sheet range
        if i < len(df_pwht):
            desc = df_pwht.iat[i, 2]   # Get the Description
            detail = df_pwht.iat[i, 3] # Get the corresponding Details

            # Skip rows that are fully blank
            if pd.notna(desc) or pd.notna(detail):
                pwht_data.append({
                    "Description": str(desc).strip(),
                    "Details": str(detail).strip()
                })

    # --------------------------------------------------------
    # 📊 STEP 9.4: Convert the List into a DataFrame and Display
    # --------------------------------------------------------

    # Convert the list of key-value rows into a structured table
    df_pwht_details = pd.DataFrame(pwht_data)

    # Show the extracted table in Colab
    display(Markdown("### ✅ Extracted PWHT Details"))
    display(df_pwht_details)

# --------------------------------------------------------
# 🛠️ STEP 9.5: Error Handling (in case sheet is missing or unreadable)
# --------------------------------------------------------

except Exception as e:
    print("❌ Failed to extract PWHT Details:", e)


# --------------------------------------------------------
# 🔹 STEP 10: Create Final Unified Summary Table
# --------------------------------------------------------
# 📌 PURPOSE:
# After extracting individual sections (Design Variables, Weld Details, PWHT, etc.),
# we now combine them into **one consolidated summary table**.
#
# ✅ WHY THIS IS IMPORTANT:
# This unified table can be:
#   - Reviewed by engineers or inspectors
#   - Used for reporting or documentation
#   - Exported to Excel or PDF
#
# 📦 What it includes:
#   - Design Variables → only those marked as "Applicable"
#   - Weld Setup Details → vertical + horizontal values
#   - Weld Summary Values → interpass temp, heat input, gas flow
#   - PWHT Details → extracted from specific rows

# --------------------------------------------------------
# 🧱 STEP 10.1: Format Applicable Design Variables
# --------------------------------------------------------

# Copy the filtered applicable design variables table
design_summary = df_applicable.copy()

# Add a column to tag the sheet origin
design_summary.insert(0, "Sheet Name", "Design Variables")

# Add a serial number column
design_summary.insert(1, "Sl No.", range(1, len(design_summary) + 1))

# Rename columns for clarity (although column names are already appropriate)
design_summary.rename(columns={"Description": "Description", "Variable Details": "Details"}, inplace=True)

# Keep only the standardized columns in order
design_summary = design_summary[["Sheet Name", "Sl No.", "Description", "Details"]]

# --------------------------------------------------------
# 🧱 STEP 10.2: Format Weld Details
# --------------------------------------------------------

weld_summary = df_weld_details.copy()

# Tag this section as "Weld Details"
weld_summary.insert(0, "Sheet Name", "Weld Details")

# Add serial numbers
weld_summary.insert(1, "Sl No.", range(1, len(weld_summary) + 1))

# Reorder columns
weld_summary = weld_summary[["Sheet Name", "Sl No.", "Description", "Details"]]

# --------------------------------------------------------
# 🧱 STEP 10.3: Format Weld Summary Values (Interpass Temp, Heat Input, Gas Flow)
# --------------------------------------------------------

# List of (Description, Value) for key summary metrics
summary_rows = [
    ("Interpass Temperature", interpass_temp),
    ("Average Heat Input", avg_heat_input),
    ("Gas Flow Rate (Shielding)", shielding_max),
    ("Gas Flow Rate (Backing)", backing_max)
]

# Convert to DataFrame
weld_summary_values = pd.DataFrame(summary_rows, columns=["Description", "Details"])

# Label it clearly
weld_summary_values.insert(0, "Sheet Name", "Weld Details - Summary Values")
weld_summary_values.insert(1, "Sl No.", range(1, len(weld_summary_values) + 1))
weld_summary_values = weld_summary_values[["Sheet Name", "Sl No.", "Description", "Details"]]

# --------------------------------------------------------
# 🧱 STEP 10.4: Format PWHT Details
# --------------------------------------------------------

pwht_summary = df_pwht_details.copy()

# Tag this section
pwht_summary.insert(0, "Sheet Name", "PWHT Details")

# Add serial numbers
pwht_summary.insert(1, "Sl No.", range(1, len(pwht_summary) + 1))

# Reorder for standard structure
pwht_summary = pwht_summary[["Sheet Name", "Sl No.", "Description", "Details"]]

# --------------------------------------------------------
# 🔗 STEP 10.5: Combine All Sections into One Unified Table
# --------------------------------------------------------

# Use pandas concat to merge all summary sections vertically
summary_table = pd.concat([
    design_summary,
    weld_summary,
    weld_summary_values,
    pwht_summary
], ignore_index=True)

# --------------------------------------------------------
# 🖥️ STEP 10.6: Display the Final Summary Table
# --------------------------------------------------------

# --------------------------------------------------------
# 🔁 STEP 11: Re-display Final Welding Tables for Reference
# --------------------------------------------------------

# 📌 PURPOSE:
# Show all the important welding data again in a clean and scrollable format.
# This is helpful when reviewing the output, preparing reports, or exporting results.

# --------------------------------------------------------
# 🔍 STEP 11.1: Re-display Final Populated Summary Table
# --------------------------------------------------------

# 🧠 Where it comes from:
# This table (`summary_table`) was built earlier by combining:
# - Design variables (from 'Design Details')
# - Weld setup info (from 'Weld Details')
# - PWHT information (from 'PWHT Detail')
# - Summary values (heat input, gas flow, etc.)

# 🔧 Normally, pandas only shows a few rows of big tables.
# Here, we tell it to display *all rows* temporarily.
pd.set_option('display.max_rows', None)

# 🖥️ Show the full summary table with all extracted details
display(Markdown("### 📌 Final Populated Summary Table (All Values Included)"))
display(summary_table)

# 🔄 Restore the default row display setting
pd.reset_option('display.max_rows')

# --------------------------------------------------------
# 🔁 STEP 11.2: Show Run-Wise Recorded Welding Data
# --------------------------------------------------------

# 🧠 This table (`welding_details`) contains data for each weld run:
# - Process type, current, voltage, travel speed, heat input, etc.
# - This was extracted earlier from the 'Weld Data Record' sheet (Step 6)

# 👁️ Display the table so users can verify actual weld input values
display(Markdown("### 📊 Full Welding Details Table (Tabular Format) — For Reference"))
display(welding_details)

# --------------------------------------------------------
# 🔁 STEP 11.3: Show WPS Qualified Ranges (Min–Max Limits)
# --------------------------------------------------------

# 🧠 This table (`wps_range_table`) shows the acceptable min–max ranges
# for each welding parameter (current, voltage, speed, etc.).
# It comes from the 'WPS Range' sheet (extracted in Step 8).

# 📋 Display this so the user can see what values are qualified for use in production
display(Markdown("### 📊 Welding Details FOR WPS Range Table — For Reference"))
display(wps_range_table)


# --------------------------------------------------------
# 🔹 STEP 12: Extract and Display Essential Variables Table
# --------------------------------------------------------

# 📌 PURPOSE:
# Create a readable table of essential welding variables (e.g., welding process, heat input)
# These values come from the summary table created earlier, and can be used in WPS/WPQR forms.

# ✅ This version:
# - Displays the table in a proper row-column format (not as plain text)
# - Optionally allows download as Excel file (if needed)

# --------------------------------------------------------
# 🧰 STEP 12.1: Import tabulate (only for print view) and display tools
# --------------------------------------------------------

from IPython.display import display
import pandas as pd

# Install tabulate only if needed (used for Markdown printing — optional)
try:
    from tabulate import tabulate
except ImportError:
    !pip install tabulate
    from tabulate import tabulate

# --------------------------------------------------------
# 🧭 STEP 12.2: Define Map of Display Label → Keyword to Match
# --------------------------------------------------------

essential_map = {
    "Impact": "impact requirement",
    "Hardness": "hardness requirement",
    "WPQR Designation:": "wpqr no",
    "Product Form:": "product form",
    "Welding Process(es):": "welding process",
    "Deposited metal Thickness (mm)": "deposited metal thickness",
    "Type of joint and weld:": "type of joint",  # Special logic below
    "Weld Details": "weld details",
    "Parent material group(s) and sub group(s):": "parent material group",
    "Parent material thickness (mm):": "parent material thickness",
    "Throat thickness (mm):": "throat thickness",
    "Single Layer / Multi run:": "single layer",
    "Outside pipe diameter (mm):": "outside pipe diameter",
    "Filler material designation:": "filler material designation",
    "Filler material make:": "filler material make",
    "Filler material size(mm):": "filler material size",
    "Designation of shielding gas/flux:": "shielding gas",
    "Designation of backing gas:": "backing gas",
    "Type of welding current and polarity:": "current and polarity",
    "Mode of metal transfer:": "mode of metal transfer",
    "Heat input (kJ/mm):": "average heat input",  # Special logic below
    "Welding positions:": "welding positions",
    "Preheat temperature Min:": "preheat temperature",
    "Interpass temperature Max:": "interpass temperature",
    "Post heating:": "post heating",
    "Post Weld Heat Treatement (PWHT)": "post weld heat treatement",
    "Other information:": "other information",
    "Tempering Temperature (If Applicable)": "tempering temperature",
    "Soaking Temp. (⁰C)": "soaking temp",
    "Loading Temp. (⁰C)": "loading temp",
    "Rate of Heating & Cooling (⁰C)": "rate of heating",
    "Soaking Time (Hrs)": "soaking time",
    "Unloading Temp. (⁰C)": "unloading temp",
    "Method": "method"
}

# --------------------------------------------------------
# 🧼 STEP 12.3: Clean 'Description' Column for Matching
# --------------------------------------------------------

summary_table["Description_clean"] = (
    summary_table["Description"]
    .astype(str)
    .str.lower()
    .str.replace(r"[:\n]", "", regex=True)
    .str.strip()
)

# --------------------------------------------------------
# 🔁 STEP 12.4: Handle Special Case — "Type of Joint + Type of Weld"
# --------------------------------------------------------

joint_row = summary_table[summary_table["Description_clean"] == "type of joint"]
weld_row = summary_table[summary_table["Description_clean"] == "type of weld"]

joint_val = joint_row.iloc[0]["Details"] if not joint_row.empty else ""
weld_val = weld_row.iloc[0]["Details"] if not weld_row.empty else ""

joint_and_weld_val = ", ".join(filter(None, [joint_val, weld_val])) or "Not Available"

# --------------------------------------------------------
# 🔁 STEP 12.5: Handle Special Case — "Average Heat Input"
# --------------------------------------------------------

heat_input_row = summary_table[
    summary_table["Description_clean"].str.contains(r"\baverage heat input\b", na=False)
]
avg_heat_input = heat_input_row.iloc[0]["Details"] if not heat_input_row.empty else "Not Available"

# --------------------------------------------------------
# 📋 STEP 12.6: Loop and Match All Essential Variables
# --------------------------------------------------------

results = []

for label, keyword in essential_map.items():
    if label == "Type of joint and weld:":
        actual_value = joint_and_weld_val
    elif label == "Heat input (kJ/mm):":
        actual_value = avg_heat_input
    else:
        pattern = rf"\b{keyword}\b"
        match = summary_table[
            summary_table["Description_clean"].str.contains(pattern, na=False)
        ]
        actual_value = match.iloc[0]["Details"] if not match.empty else "Not Available"

    results.append({
        "Essential Variable": label,
        "Actual Value": actual_value,
        "Range": ""  # Reserved for future extensions
    })

# --------------------------------------------------------
# 🧾 STEP 12.7: Create DataFrame and Display as Table
# --------------------------------------------------------

essential_variables_df = pd.DataFrame(results)

# ✅ Display the table (as scrollable, readable DataFrame)
display(Markdown("### 📌 Final Essential Variables Table (Tabular Format)"))
display(essential_variables_df)

# --------------------------------------------------------
# 💾 STEP 12.8: (Optional) Download as Excel File
# --------------------------------------------------------

import sys
from IPython.display import display, Markdown

# Save the table to Excel
excel_filename = "Essential_Variables_Table.xlsx"
essential_variables_df.to_excel(excel_filename, index=False)

# Try importing and downloading only if in Colab
if "google.colab" in sys.modules:
    try:
        from google.colab import files
        files.download(excel_filename)
    except Exception as e:
        print(f"⚠️ Colab download failed: {e}")
else:
    print(f"✅ File saved: {excel_filename} (download manually if needed)")





✅ Excel file loaded: Welding Data Recording.xlsx

📄 Available sheets in this Excel file:
['Design Details', 'Weld Details', 'Weld Data record', 'WPS Range', 'PWHT Detail']


### ✅ Preview: Design Details

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,Design and Customer Requirement,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,WPQR No:,PQR/01,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Sl No.,Customer Spec / Product Code,Clause,Description,Applicable /\nNot Applicable,Variable Details,Remark
5,NaN,1,NaN,NaN,Product Standard,Not Applicable,EN 15087,NaN
6,NaN,2,NaN,NaN,Customer,Not Applicable,XXXX,NaN
7,NaN,3,NaN,NaN,Customer Specification,Not Applicable,XXXX,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,4,NaN,NaN,Impact Requirement,Applicable,Yes,NaN


### ✅ Preview: Weld Details

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,Welding Data Recording Sheet,NaN,NaN,NaN
1,NaN,Testing Standard,ISO 15614,Level,2
2,NaN,NaN,NaN,NaN,NaN
3,NaN,WPQR No:,PQR/01,Product Form:,Plate
4,NaN,Welding Process(es):,MAG (135),Base Material Grade:,S355J2W+N to S355J2W+N
5,NaN,PWPS No:,PWPS/01,Parent material group(s) and sub group(s):,1.4 to 1.4
6,NaN,Filler Material Classification (ASME),SFA-5.18 – ER70S-6,Parent material thickness (mm):,1 - 12 mm \n2 - 12 mm
7,NaN,Filler material designation (ISO) :,ISO 14341-A-G 42 3 M21 3Si1,Test Coupon Dimensions:,150 X 300
8,NaN,Filler material make:,Bohler,Base Metal Heat No:,B18035
9,NaN,Filler material Brand Name:,SG2,Base Metal Batch No:,…………


...

,Run,Welding Process,Size of filler material(mm),Type of current/ polarity,Current (A),Voltage (V),Wire feed speed (m/min),Travel speed (mm/Sec),Heat Input (KJ/mm)
0,1-1,MAG (135),1.2,DCEP,78 - 131,16 - 26,2.70 - 4.50,3.21 - 5.36,0.32 - 0.53
1,2-1,MAG (135),1.2,DCEP,123 - 206,18 - 31,4.42 - 7.38,3.00 - 5.00,0.62 - 1.03
2,3-1,MAG (135),1.2,DCEP,176 - 293,20 - 33,7.28 - 12.12,3.46 - 5.77,0.82 - 1.37
3,3-2,MAG (135),1.2,DCEP,150 - 250,19 - 32,5.25 - 8.75,3.36 - 5.60,0.70 - 1.16


### 📌 Final Essential Variables Table (Tabular Format)

,Essential Variable,Actual Value,Range
0,Impact,Yes,
1,Hardness,Yes,
2,WPQR Designation:,PQR/01,
3,Product Form:,Plate,
4,Welding Process(es):,MAG (135),
5,Deposited metal Thickness (mm),12 mm,
6,Type of joint and weld:,"Butt -Joint, Butt Weld – Full Penetration",
7,Weld Details,"Both Side – with gauging,",
8,Parent material group(s) and sub group(s):,1.4 to 1.4,
9,Parent material thickness (mm):,1 - 12 mm \n2 - 12 mm,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
interpretation_rows = []
essential_output_rows = []


In [4]:
# ============================================================
# 🔧 Essential Variables for WPQR — ISO 15614-1 Reference
# ============================================================

# Essential Variable: 1. Product Form
# - Refers to the shape/form of the base material used in the test.
# - Common forms: Plate, Pipe, Forging, Casting.

# Essential Variable: 2. Outside Pipe Diameter (mm)
# - Refers to the external diameter of the pipe used in welding.
# - Critical for determining range qualification for pipe welding.

# ------------------------------------------------------------------------
# 📘 ISO 15614-1 — Clause 8.3.3: Diameter of Pipes and Branch Connections (Level 2)
# ------------------------------------------------------------------------

# ✅ For LEVEL 2:
# A welding procedure test conducted on a pipe with diameter D qualifies for welding other pipe/branch connections
# with diameters in the following range, as per Table 9:
#     ➤ Qualified range = All diameters ≥ 0.5 × D

# 🧾 Additional Notes:
# - A procedure qualified on a **pipe** also qualifies for **plate**.
# - A procedure qualified on a **plate** also qualifies for **pipe** only when:
#     • Pipe outside diameter (OD) > 500 mm
#     • OR pipe OD > 150 mm AND welded in any of the following positions:
#         - PC (horizontal)
#         - PF (vertical-up, rotated)
#         - PA (flat, rotated)

# ------------------------------------------------------------------------
# 📊 Table 9 — Qualification Range for Pipe & Branch Connection Diameters
# ------------------------------------------------------------------------
# | Diameter of Test Piece (D) | Qualified Range                   |
# |----------------------------|------------------------------------|
# | D                          | All diameters ≥ 0.5 × D            |
# ------------------------------------------------------------------------

# 📝 NOTES:
# NOTE 1: For hollow sections other than circular (e.g., elliptical), D is the **smaller side dimension**.
# NOTE 2: D refers to:
#     • The **outside diameter** of the pipe for a butt weld
#     • OR the **outside diameter** of the branch pipe in a branch connection (see Figure 4 → D2)

# ------------------------------------------------------------------------
# 📘 ISO 15614-1 — Clause 8.3.4: Angle of Branch Connection (Level 2)
# ------------------------------------------------------------------------

# ✅ For LEVEL 2:
# A branch connection's angle (α) is an essential variable during qualification.

# 🎯 Qualification rules based on α (angle between branch and main pipe):
# - If the test is performed at α between 60° and 90°, it qualifies all angles in that same range:
#       ➤ Qualified Range = 60° ≤ α < 90°
# - If α < 60°, a **separate test is required**.
#       ➤ That test qualifies all angles from that α up to 90°

# 🧠 Example:
# - Test at α = 70° → Qualifies angles between 60° and <90°
# - Test at α = 50° → Qualifies angles from 50° to <90°, but does NOT qualify 60°–90° unless retested


# ------------------------------------------------------------
# 📘 INTERPRETATION: What Was Considered for Qualification Logic
# ------------------------------------------------------------

# 1️⃣ The welding procedure qualification is based on product form (plate or pipe).
#     ➤ If the test was done on a **pipe**, it automatically qualifies both pipe and plate.
#     ➤ If the test was done on a **plate**, we apply extra rules using diameter and position.

# 2️⃣ When the test piece is a **plate**, it qualifies for pipe **only if**:
#     ✅ Outside diameter (OD) of the pipe is > 500 mm
#         → This is a general rule that allows qualification without considering position.
#     ✅ OR OD > 150 mm AND the weld was done in **specific rotated positions**:
#         - PC (Horizontal pipe position)
#         - PF (Vertical-up position with rotation)
#         - PA (Flat position with rotation)
#         → These positions are considered more complex and thus broaden the qualified range.

# 3️⃣ If the diameter is not extractable (missing or invalid), the default assumption is:
#     → "Plate only" is qualified (we assume conservative result to stay safe)

# 4️⃣ All conditions and values used were pulled from the **essential variables table**, which contains:
#     - Product form (e.g., Plate or Pipe)
#     - Outside pipe diameter (e.g., "300 mm")
#     - Welding position (e.g., "PC" or "PA")

# 📌 This logic follows ISO 15614-1:2021 Clause 8.3.3 (Level 2 requirements)
#     and ensures that your WPQR output reflects the correct qualified range
#     based on how the procedure test was performed.

# ✅ Final Output:
#     - "Plate and Pipe" → means both are qualified for WPS/WPQR
#     - "Plate only" → only plate is qualified; pipe must be retested
#     - "Unknown — Manual Review Required" → if input is ambiguous


import pandas as pd
import re
from tabulate import tabulate

# ------------------------------------------------------------
# 🔧 These are global lists that collect output from logic checks
# - interpretation_rows: holds human-readable clause notes
# - essential_output_rows: holds final interpreted values
# Make sure these are declared earlier in your code before using
# ------------------------------------------------------------
# interpretation_rows = []
# essential_output_rows = []

# ------------------------------------------------------------
# 🔍 Function to extract a value from a DataFrame by keyword match
# - Searches 'Essential Variable' column in 'essential_variables_df'
# - Returns corresponding 'Actual Value' if match is found
# ------------------------------------------------------------
def extract_val(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return row.iloc[0]["Actual Value"].strip() if not row.empty else "Not Applicable"

# ------------------------------------------------------------
# 🔍 Extract key inputs needed for Clause 8.3.3 logic
# These values are extracted from the essential variables table
# ------------------------------------------------------------
product_form_val = extract_val(essential_variables_df, "product form")
od_val = extract_val(essential_variables_df, "outside pipe diameter")
welding_pos_val = extract_val(essential_variables_df, "welding position")

# ------------------------------------------------------------
# 🔍 Helper function: Extract numeric value from a string (e.g., "500 mm" → 500.0)
# Used to isolate the outside pipe diameter as a number
# ------------------------------------------------------------
def extract_numeric(val):
    nums = re.findall(r"\d+\.?\d*", val)
    return float(nums[-1]) if nums else None

# Extract numeric OD value and normalize welding position string
od_numeric = extract_numeric(od_val)
weld_pos_upper = welding_pos_val.upper()

# ------------------------------------------------------------
# ✅ ISO 15614-1 Clause 8.3.3 Product Form Qualification Logic
# ------------------------------------------------------------
# This block decides what product forms are qualified based on:
# - Product form tested (plate or pipe)
# - Pipe diameter
# - Welding position
# Final result is stored in 'product_form_qualified'

if "pipe" in product_form_val.lower():
    # If test was on a pipe, both pipe and plate are qualified
    product_form_qualified = "Plate and Pipe"
elif "plate" in product_form_val.lower():
    # If test was on a plate, apply special conditions
    if od_numeric:
        if od_numeric > 500:
            product_form_qualified = "Plate and Pipe"
        elif od_numeric > 150 and any(pos in weld_pos_upper for pos in ["PC", "PF", "PA"]):
            product_form_qualified = "Plate and Pipe"
        else:
            product_form_qualified = "Plate only"
    else:
        product_form_qualified = "Plate only"
else:
    product_form_qualified = "Unknown — Manual Review Required"

# ------------------------------------------------------------
# 🧾 Build Display Tables for User Understanding
# ------------------------------------------------------------

# Table listing the essential variables being evaluated
essential_variable_table = [
    ["1. Product Form", ""],
    ["2. Outside Pipe Diameter", ""]
]

# Table showing the actual input values pulled from essential_variables_df
input_table = [
    ["1. Product Form", product_form_val],
    ["2. Outside Pipe Diameter", od_val],
    ["3. Welding Position", welding_pos_val]
]

# ISO clause interpretation displayed in plain language
interpretation = [
    ["Based on ISO 15614-1 Clause 8.3.3, plate qualifies pipe only if OD > 500 mm"],
    ["or OD > 150 mm with position PC/PF/PA."]
]

# Final output result showing what is qualified
output_table = [
    ["Product Form Qualified", product_form_qualified],
    ["Outside Pipe Diameter (mm)", f"{od_numeric if od_numeric is not None else 'N/A'}"]
]

# ------------------------------------------------------------
# 📌 Append results to shared global tables for Excel or later use
# ------------------------------------------------------------
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# ------------------------------------------------------------
# 📊 Display all parts visually using tabulate (in Colab/console)
# ------------------------------------------------------------
print()
print(tabulate(essential_variable_table, headers=["Essential Variable", ""], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", ""], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", ""], tablefmt="grid"))



+--------------------------+----+
| Essential Variable       |    |
+==========================+====+
| 1. Product Form          |    |
+--------------------------+----+
| 2. Outside Pipe Diameter |    |
+--------------------------+----+

+--------------------------+----------------+
| Input Considered         |                |
+==========================+================+
| 1. Product Form          | Plate          |
+--------------------------+----------------+
| 2. Outside Pipe Diameter | Not Applicable |
+--------------------------+----------------+
| 3. Welding Position      | PA (1G)        |
+--------------------------+----------------+

+-----------------------------------------------------------------------------+
| Clause Interpretation                                                       |
+=============================================================================+
| Based on ISO 15614-1 Clause 8.3.3, plate qualifies pipe only if OD > 500 mm |
+------------------------

In [5]:
# ------------------------------------------------------------------------
# Essential Variable: 1. Welding Process(es)
# ------------------------------------------------------------------------

# 📘 ISO 15614-1 — Clause 8.4.1: Welding Processes (Level 2)
# ------------------------------------------------------------------------

# ✅ For LEVEL 2:
# - Each type of welding process used in production must be qualified independently.
# - This includes qualification for different degrees of mechanization such as:
#     • Manual
#     • Partly mechanized
#     • Fully mechanized
#     • Automatic

# 🔄 Multi-process Welding:
# - If multiple welding processes are used in production (e.g., GTAW + SMAW), then:
#     ➤ Each process must either:
#         • Be qualified separately via individual tests
#         • OR be qualified together via a multi-process procedure test

# 🧪 Test Piece Requirements:
# - The welding procedure qualification is valid only for the exact sequence of welding processes used in the test.
# - The test specimens must include deposited weld metal from **each** process used.
# - A back run can be welded using any of the processes used in the test piece.

# ⚠️ Single-Process Use from Multi-Process Qualification:
# - If only one process from a multi-process qualification is used in production,
#   ➤ That individual process must still be **tested and qualified** as a standalone process.


# ------------------------------------------------------------------------------
# 📘 INTERPRETATION — Welding Process Qualification (Clause 8.4.1, ISO 15614-1)
# ------------------------------------------------------------------------------

# ✅ 1. If the test was performed using only one welding process (e.g., "MAG"):
#    → The procedure is qualified **only** for that single process.
#    → It cannot be used for another process unless separately qualified.

# ✅ 2. If the test was performed using **multiple welding processes**:
#    → The qualification is valid **only** for the exact sequence in which
#       the processes were used during the test (e.g., "GTAW → SMAW").
#    → This means both the processes and their order are essential variables.

# ✅ 3. If a manufacturer intends to use just one of the processes (e.g., only SMAW)
#    from a multi-process WPQR (e.g., GTAW + SMAW):
#    → That **single process must be qualified separately** according to ISO 15614-1.

# ⚠️ NOTE:
# - Simply using the same processes in a different order (e.g., SMAW → GTAW) is NOT acceptable.
# - Back-run operations are permitted using any one of the qualified processes from the WPQR.
# - All test specimens must include deposited weld metal from **each** process used.

# 📌 CONCLUSION:
# - This code ensures that the WPS qualification complies strictly with the sequence and type
#   of welding processes used in the original qualification test.
# - The interpretation logic captures both single-process and multi-process scenarios.

# ------------------------------------------------------------------------------


import pandas as pd
import re
from tabulate import tabulate

# ------------------------------------------------------------------------
# 📘 STEP: Extract and Evaluate Welding Process Qualification
# Based on ISO 15614-1 Clause 8.4.1 (Level 2)
# ------------------------------------------------------------------------

# 🔍 Extract the row where the Essential Variable is "Welding Process(es):"
welding_process_row = essential_variables_df[
    essential_variables_df["Essential Variable"].str.strip().str.lower() == "welding process(es):"
]

# ✅ Get the corresponding value (e.g., "GTAW + SMAW")
welding_process_value = (
    welding_process_row.iloc[0]["Actual Value"] if not welding_process_row.empty else "Not Available"
)

# 🧹 Split into individual processes using common delimiters: +, &, /, or ,
process_list = re.split(r"[+&/,]", welding_process_value)
process_list = [p.strip() for p in process_list if p.strip()]  # Remove whitespace and empty entries

# 📘 Apply logic based on number of processes:
# - One process = only that process is qualified
# - Multiple processes = only that exact sequence is qualified
if len(process_list) == 1:
    qualified_for = f"Only for welding with: {process_list[0]}"
elif len(process_list) > 1:
    sequence = " → ".join(process_list)
    qualified_for = f"Only valid for multi-process sequence: {sequence}"
else:
    qualified_for = "Unknown (manual review required)"

# ------------------------------------------------------------------------
# 🧾 Build Output Tables for Reference and Export
# ------------------------------------------------------------------------

# 🎯 Table header (for reference only)
essential_variable_table = [
    ["1. Welding Process(es)", ""]
]

# 🎯 Input values considered from extracted data
input_table = [
    ["1. Welding Process(es)", welding_process_value]
]

# 📘 Interpretation summary based on standard requirements
interpretation = [
    ["Based on ISO 15614-1 Clause 8.4.1:"],
    ["- WPS is only valid for the process(es) used in qualification."],
    ["- If multiple processes used, only the exact sequence is qualified."],
    ["- If using only one process from a multi-process WPQR, it must be separately qualified."]
]

# ✅ Final qualified outcome to be recorded
output_table = [
    ["Welding Process Qualification", qualified_for]
]

# 📌 Append to global storage lists used for final Excel export
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# 📤 Display all outputs in formatted table view
print()
print(tabulate(essential_variable_table, headers=["Essential Variable", ""], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", ""], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", ""], tablefmt="grid"))




+------------------------+----+
| Essential Variable     |    |
+========================+====+
| 1. Welding Process(es) |    |
+------------------------+----+

+------------------------+-----------+
| Input Considered       |           |
+========================+===========+
| 1. Welding Process(es) | MAG (135) |
+------------------------+-----------+

+-----------------------------------------------------------------------------------------+
| Clause Interpretation                                                                   |
+=========================================================================================+
| Based on ISO 15614-1 Clause 8.4.1:                                                      |
+-----------------------------------------------------------------------------------------+
| - WPS is only valid for the process(es) used in qualification.                          |
+-----------------------------------------------------------------------------------------

In [6]:
# ------------------------------------------------------------------------------
# 📘 ISO 15614-1 — Clause 8.3.2.2: Thickness Qualification — Level 2
# ------------------------------------------------------------------------------
# ✅ ESSENTIAL VARIABLES:
# 1. Parent Material Thickness (t)
# 2. Deposited Weld Metal Thickness (s)
# 3. Throat Thickness (a) — for fillet welds

# ----------------------------------------------------------------------
# GENERAL RULE:
# The welding procedure test conducted on a plate or pipe of thickness `t`
# qualifies a range of parent material thickness and deposited weld metal thickness
# as defined in Table 7 (for butt welds) and Table 8 (for fillet welds).
# ----------------------------------------------------------------------

# ----------------------------------------------------------------------
# 📊 TABLE 7 — Butt Welds: Range of Qualification for Parent Material & Deposited Metal Thickness
# ----------------------------------------------------------------------
# | Thickness of Test Piece (t) | Parent Material Thickness Qualified | Deposited Weld Thickness (s) Qualified (Level 2)       |
# |-----------------------------|--------------------------------------|---------------------------------------------------------|
# | t ≤ 3 mm                    | 0.5 × t to 2 × t                     | Single run: 0.5 × t to 1.3 × t                          |
# |                             |                                      | Multi-run: 3 mm to 2 × s (max. 2 × a if impact not done)|
# | 3 < t ≤ 12 mm               | 1.5 mm to 2 × t                      | 0.5 × t to 1.3 × t (min 3 mm)                           |
# | 12 < t ≤ 20 mm              | 5 mm to 2 × t                        | 0.5 × t to 2 × t (max. 2 × s)                           |
# | 20 < t ≤ 40 mm              | 5 mm to 2 × t                        | Same as above                                           |
# | 40 < t ≤ 100 mm             | 5 mm to 200 mm                       | Same as above or max. 200 mm if s ≥ 20 mm              |
# | 100 < t ≤ 150 mm            | 5 mm to 200 mm                       | 50 mm to 2 × t or max. 2 × s / 300 mm                  |
# | t > 150 mm                  | 5 mm to 1.33 × t                     | 50 mm to 2 × t or max. 1.33 × t                        |
# ----------------------------------------------------------------------

# 📝 NOTE:
# - If impact tests are NOT performed but are required, maximum qualified thickness is limited to 12 mm.
# - For multipass welds, qualification is broader, but must meet energy control requirements.

# ----------------------------------------------------------------------
# 📊 TABLE 8 — Fillet Welds: Qualification Range for Material & Throat Thickness (Level 2)
# ----------------------------------------------------------------------
# | Thickness of Test Piece (t) | Material Thickness Qualified     | Throat Thickness Qualified (a)                          |
# |-----------------------------|----------------------------------|----------------------------------------------------------|
# | t ≤ 3 mm                    | 0.7 × t to 2 × t                  | Single Run: 0.75 × a to 1.5 × a                         |
# |                             |                                  | Multi Run: No restriction                                |
# | 3 < t < 30 mm               | 3 mm to 2 × t                     | No restriction                                           |
# | t ≥ 30 mm                   | ≥ 5 mm                            | No restriction                                           |
# ----------------------------------------------------------------------

# 🧠 INTERPRETATION:
# - A weld test done at a specific base metal thickness (t) does not qualify unlimited thicknesses.
# - Clause 8.3.2.2 introduces **bounded thickness ranges** to control how procedures scale to thinner or thicker materials.
# - Throat thickness for fillet welds is only restricted in single-run conditions.
# - If a butt weld is used to qualify fillet welds, the throat thickness range must be based on the deposited weld thickness (s).

# 📌 USE IN CODE:
# These values are extracted from your WPQR essential variables table.
# Based on actual `t`, `s`, and `a`, the code applies the lookup logic to determine:
# - Qualified parent material thickness
# - Qualified deposited weld thickness
# - Qualified throat thickness range (if applicable)

# ----------------------------------------------------------------------


# ------------------------------------------------------------------------
# 📘 ISO 15614-1 — Clause 8.3.2.2: Thickness Qualification (Level 2)
# ------------------------------------------------------------------------

# ✅ Essential Variable: 1. Parent material thickness (t)
# ✅ Essential Variable: 2. Deposited weld metal thickness (s)
# ✅ Essential Variable: 3. Throat thickness (a)

# 🧮 The following logic has been considered in the code:

# -----------------------------
# ✅ PARENT MATERIAL THICKNESS (t)
# -----------------------------
# - If t ≤ 3 mm:
#     ➤ Qualified range = 0.5 × t to 2 × t
# - If 3 < t ≤ 12 mm:
#     ➤ Qualified range = 1.5 mm (min) or 0.5 × t to 2 × t
# - If 12 < t ≤ 20 mm:
#     ➤ Qualified range = 5 mm to 2 × t
# - If 20 < t ≤ 40 mm:
#     ➤ Qualified range = 5 mm to 2 × t
# - If 40 < t ≤ 100 mm:
#     ➤ Qualified range = 5 mm to 200 mm (fixed max)
# - If 100 < t ≤ 150 mm:
#     ➤ Qualified range = 5 mm to 200 mm (as per code simplification)
# - If t > 150 mm:
#     ➤ Qualified range = 5 mm to 1.33 × t

# -----------------------------
# ✅ DEPOSITED METAL THICKNESS (s)
# -----------------------------
# - Qualified range is determined using:
#     ➤ Minimum = 0.5 × t
#     ➤ Maximum = 2 × s or 2 × t (whichever is lower)
#     ➤ Max = 200 mm if s ≥ 20 mm
# - Used ISO Table 7 logic for Level 2 — multi-run condition

# -----------------------------
# ✅ THROAT THICKNESS (a)
# -----------------------------
# - If a is None and s is available (i.e., full-penetration welds used for fillet weld qualification):
#     ➤ Qualified range = "No restriction"
# - If a is known and t ≤ 3 mm:
#     ➤ Qualified range = 0.75 × a to 1.5 × a
# - If a is known and t > 3 mm:
#     ➤ Qualified range = "No restriction"
# - Based on Table 8 for fillet welds qualified by butt welds

# 📌 Interpretation logic assumes the weld type is butt or fillet (based on context), and ranges are estimated for WPS/WPQR use.


import re
from tabulate import tabulate

# --- Helper to extract numeric value from text ---
def extract_numeric(value):
    nums = [float(n) for n in re.findall(r"\d+\.?\d*", str(value))]
    return max(nums) if nums else None

# --- Extract values from dataframe ---
t_val = extract_val(essential_variables_df, "parent material thickness")
s_val = extract_val(essential_variables_df, "deposited metal thickness")
a_val = extract_val(essential_variables_df, "throat thickness")

t = extract_numeric(t_val)
s = extract_numeric(s_val)
a = extract_numeric(a_val)

# --- Calculate Parent Material Thickness Range (t) ---
if t is not None:
    if t <= 3:
        t_range = f"{round(0.5 * t, 2)} mm to {round(2 * t, 2)} mm"
    elif 3 < t <= 12:
        t_range = f"{max(1.5, round(0.5 * t, 2))} mm to {round(2 * t, 2)} mm"
    elif 12 < t <= 20:
        t_range = f"5 mm to {round(2 * t, 2)} mm"
    elif 20 < t <= 40:
        t_range = f"5 mm to {round(2 * t, 2)} mm"
    elif 40 < t <= 100:
        t_range = "5 mm to 200 mm"
    elif 100 < t <= 150:
        t_range = "5 mm to 200 mm"
    else:
        t_range = f"5 mm to {round(1.33 * t, 2)} mm"
else:
    t_range = "Manual review required"

# --- Calculate Deposited Weld Thickness Range (s) ---
if s is not None and t is not None:
    s_min = round(0.5 * t, 2)
    s_max = 200 if s >= 20 else round(min(2 * s, 2 * t), 2)
    s_range = f"{s_min} mm to {s_max} mm"
else:
    s_range = "Manual review required"

# --- Calculate Throat Thickness Range (a) ---
if a is None and s is not None:
    a_range = "No restriction"
elif a is not None and t is not None:
    if t <= 3:
        a_range = f"{round(0.75 * a, 2)} mm to {round(1.5 * a, 2)} mm"
    else:
        a_range = "No restriction"
else:
    a_range = "Manual review"

# --- Tabulate outputs ---
essential_vars = [
    ["1. Parent material thickness (mm)", ""],
    ["2. Deposited metal thickness (mm)", ""],
    ["3. Throat thickness (mm)", ""]
]

input_data = [
    ["1. Parent material thickness (t)", t_val],
    ["2. Deposited metal thickness (s)", s_val],
    ["3. Throat thickness (a)", a_val]
]

interpretation = [
    ["Based on ISO 15614-1 Clause 8.3.2.2:"],
    ["- Qualified parent thickness from Table 7 based on test piece thickness."],
    ["- Deposited metal qualified up to 2s or 2t, max 200 mm depending on actual s."],
    ["- Throat thickness in fillet welds generally has no restriction if multi-run or t > 3 mm."]
]

output_table = [
    ["Parent Material Thickness (t)", t_range],
    ["Deposited Weld Thickness (s)", s_range],
    ["Throat Thickness (a)", a_range]
]

# --- Append to global output lists ---
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# --- Display formatted tables ---
print()
print(tabulate(essential_vars, headers=["Essential Variable", ""], tablefmt="grid"))
print()
print(tabulate(input_data, headers=["Input Considered", ""], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", ""], tablefmt="grid"))



+-----------------------------------+----+
| Essential Variable                |    |
+===================================+====+
| 1. Parent material thickness (mm) |    |
+-----------------------------------+----+
| 2. Deposited metal thickness (mm) |    |
+-----------------------------------+----+
| 3. Throat thickness (mm)          |    |
+-----------------------------------+----+

+----------------------------------+----------------+
| Input Considered                 |                |
+==================================+================+
| 1. Parent material thickness (t) | 1 - 12 mm      |
|                                  | 2  - 12 mm     |
+----------------------------------+----------------+
| 2. Deposited metal thickness (s) | 12 mm          |
+----------------------------------+----------------+
| 3. Throat thickness (a)          | Not Applicable |
+----------------------------------+----------------+

+---------------------------------------------------------------------

In [7]:
# ------------------------------------------------------------------------
# 📘 ISO 15614-1 — Clause 8.4.3: Type of Joint and Weld (Level 2)
# ------------------------------------------------------------------------

# For level 2: The range of qualification for the type of welded joints
# is as used in the welding procedure test, subject to limitations given
# in other clauses (e.g., thickness) and additionally:

# a) Butt welds qualify full and partial penetration butt welds and fillet welds,
#    while fillet weld tests are required where T-joints are performed by fillet welds
#    or partial penetration butt welds in the predominant form of welded connections
#    in relation to the design and production welding.

# b) Full penetration butt welds qualify full and partial penetration butt welds
#    and fillet welds in any type of joints.

# c) Butt joints in pipe qualify branch connections with an angle α ≥ 60°
#    (see Figure 4 for α).

# d) Butt welds in T-joints with full penetration qualify full and partial penetration
#    butt welds in T-joints and fillet welds — but not vice versa.

# e) Fillet welds qualify fillet welding only.

# f) Welds made from one side without backing qualify welds made from both sides
#    and welds with backing.

# g) Welds made with backing qualify welds made from both sides.

# h) Welds made from both sides without removing the root qualify welds made
#    from both sides with removing the root (except thermal gouging).

# i) Welds made from both sides with or without gouging qualify welds made
#    from one side with backing.

# j) When impact or hardness requirements apply, it is not permitted to change
#    a multi-run deposit into a single run deposit (or single run on each side)
#    or vice versa for a given process.

# k) Build-up welding is qualified by butt weld test piece.

# l) Buttering shall be performed by a separate test piece in combination
#    with the butt weld.


# ------------------------------------------------------------------------
# 📘 ISO 15614-1 — Clause 8.4.3: Type of Joint and Weld (Level 2)
# ------------------------------------------------------------------------

# ✅ Essential Variables:
# 1. Type of joint and weld
# 2. Weld Details
# 3. Single Layer / Multi run

# 🧮 This clause defines what joint types and welding configurations are qualified
#     when a specific type of weld is used in the procedure qualification test (PQT).

# 🔍 Interpretation Logic Applied in Code:

# a) Butt welds qualify:
#    → Full penetration butt welds
#    → Partial penetration butt welds
#    → Fillet welds
#    ⚠️ But if fillet welds (in T-joints) are predominant in design, a separate test is required.

# b) Full penetration butt welds qualify:
#    → All types of butt welds (full and partial penetration)
#    → Fillet welds
#    → In any type of joint

# c) Butt joints in pipe qualify:
#    → Branch connections with an angle α ≥ 60°

# d) Butt welds in T-joints (full penetration) qualify:
#    → Full/partial penetration butt welds in T-joints
#    → Fillet welds
#    ⚠️ But not the reverse

# e) Fillet welds only qualify:
#    → Fillet welds (do not qualify butt welds)

# f) Welds made from one side without backing qualify:
#    → Welds made from both sides
#    → Welds with backing

# g) Welds made with backing qualify:
#    → Welds made from both sides

# h) Welds from both sides without root removal qualify:
#    → Welds from both sides with root removal (excluding thermal gouging)

# i) Welds from both sides (with/without gouging) qualify:
#    → Welds from one side with backing

# j) When impact or hardness testing is required:
#    → Must maintain run type (multi-run or single-run) as qualified
#    → Switching is not allowed

# k) Build-up welding:
#    → Qualified via butt weld test piece

# l) Buttering:
#    → Requires a separate test piece alongside butt weld qualification

# ------------------------------------------------------------------------
# 📊 Summary Table — Qualification Matrix (Clause 8.4.3)
# ------------------------------------------------------------------------
# | Qualification Test Performed                        | Qualified For                                     |
# |-----------------------------------------------------|---------------------------------------------------|
# | Butt weld                                           | Butt (full/partial) + Fillet                      |
# | Full penetration butt weld                          | All joints + Fillet                              |
# | Butt joint in pipe                                  | Branch with α ≥ 60°                              |
# | Butt weld in T-joint (full penetration)             | Butt (full/partial) in T-joint + Fillet          |
# | Fillet weld                                         | Fillet only                                      |
# | One-side weld (no backing)                          | Welds both sides + with backing                  |
# | Weld with backing                                   | Welds both sides                                 |
# | Both sides (no root removal)                        | Both sides with root removal (except gouging)    |
# | Both sides (with/without gouging)                   | One side with backing                            |
# | Multi-run (when impact/hardness required)           | Only multi-run allowed (no switch to single-run) |
# | Build-up welding                                    | Qualified by butt weld test                      |
# | Buttering                                           | Needs separate test + butt weld qualification    |
# ------------------------------------------------------------------------





import re
from tabulate import tabulate

# ------------------------------------------------------------
# 📌 Essential Variables:
# 1. Type of joint and weld
# 2. Weld Details
# 3. Single Layer / Multi run
# Clause: ISO 15614-1 → 8.4.3 Type of joint/weld
# ------------------------------------------------------------

# 🔍 Extract combined "Type of joint and weld" value
joint_and_weld_val = extract_val(essential_variables_df, "type of joint and weld")

# 🎯 Try to split it into joint type and weld type
joint_type_val, weld_type_val = ("", "")
if isinstance(joint_and_weld_val, str) and "," in joint_and_weld_val:
    parts = [p.strip() for p in joint_and_weld_val.split(",", maxsplit=1)]
    if len(parts) == 2:
        joint_type_val, weld_type_val = parts
    else:
        joint_type_val = parts[0]
else:
    print("⚠️ Warning: 'Type of joint and weld' format not as expected.")

# 🔍 Extract remaining essential inputs
weld_detail_val = extract_val(essential_variables_df, "weld details")
layer_val = extract_val(essential_variables_df, "single layer")
impact_val = extract_val(essential_variables_df, "impact")

# ✅ Normalize for comparison
joint_type = str(joint_type_val).strip().lower()
weld_type = str(weld_type_val).strip().lower()
weld_detail = str(weld_detail_val).strip().lower()
layer_type = str(layer_val).strip().lower()
impact_required = str(impact_val).strip().lower() == "yes"

# ------------------------------------------------------------
# 🔎 Clause 8.4.3 Logic for Interpretation
# ------------------------------------------------------------

# 🧠 Determine joint type qualification
if "butt" in weld_type and "full" in weld_type:
    joint_qualification = "full/partial butt welds and fillet welds"
elif "butt" in weld_type:
    joint_qualification = "partial butt welds and fillet welds"
elif "fillet" in weld_type:
    joint_qualification = "fillet welds only"
else:
    joint_qualification = "Manual review required"

# 🧠 Determine backing qualification
if "single side" in weld_detail and "without backing" in weld_detail:
    backing_qualification = "both-side welds and welds with backing"
elif "single side" in weld_detail and "backing" in weld_detail:
    backing_qualification = "both-side and backing welds only"
elif "both side" in weld_detail:
    backing_qualification = "both-side and backing welds"
else:
    backing_qualification = "Manual review"

# 🧠 Determine restriction for run type (single vs multi-run)
if impact_required:
    if "multi" in layer_type:
        run_qualification = "Multi-run only (cannot switch to single-run)"
    elif "single" in layer_type:
        run_qualification = "Single-run only (cannot switch to multi-run)"
    else:
        run_qualification = "Run type unclear — check manually"
else:
    run_qualification = "No restriction — impact not required"

# ------------------------------------------------------------
# 📋 Tabulate Output
# ------------------------------------------------------------

essential_vars = [
    ["1. Type of joint and weld", ""],
    ["2. Weld Details", ""],
    ["3. Single Layer / Multi run", ""]
]

input_data = [
    ["1. Type of joint", joint_type_val],
    ["2. Type of weld", weld_type_val],
    ["3. Weld Details", weld_detail_val],
    ["4. Single Layer / Multi run", layer_val],
    ["5. Impact", impact_val]
]

interpretation = [
    ["Based on ISO 15614-1 Clause 8.4.3:"],
    ["- Full penetration butt weld qualifies butt and fillet welds."],
    ["- Fillet welds qualify fillet welds only."],
    ["- Single-side welding without backing qualifies both-side and backed welds."],
    ["- Impact requirements restrict switching between single and multi-run."]
]

output_table = [
    ["Type of joint and weld", joint_qualification],
    ["Weld Details", backing_qualification],
    ["Single Layer / Multi run", run_qualification]
]

# 🧩 Add results to global collectors (used in Excel filling later)
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# 📤 Display Final Output
print()
print(tabulate(essential_vars, headers=["Essential Variable", ""], tablefmt="grid"))
print()
print(tabulate(input_data, headers=["Input Considered", ""], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", ""], tablefmt="grid"))



+-----------------------------+----+
| Essential Variable          |    |
+=============================+====+
| 1. Type of joint and weld   |    |
+-----------------------------+----+
| 2. Weld Details             |    |
+-----------------------------+----+
| 3. Single Layer / Multi run |    |
+-----------------------------+----+

+-----------------------------+------------------------------+
| Input Considered            |                              |
+=============================+==============================+
| 1. Type of joint            | Butt -Joint                  |
+-----------------------------+------------------------------+
| 2. Type of weld             | Butt Weld – Full Penetration |
+-----------------------------+------------------------------+
| 3. Weld Details             | Both Side – with gauging,    |
+-----------------------------+------------------------------+
| 4. Single Layer / Multi run | Multi Layer                  |
+-----------------------------+----

In [8]:
# ------------------------------------------------------------------------
# 📘 ISO 15614-1 — Clause 8.4.4: Filler material, manufacturer/trade name, designation
# ------------------------------------------------------------------------

# ✅ Essential Variables:
# 1. Filler Material Make (Manufacturer / Trade Name)
# 2. Filler Material Designation

# For level 2:
# Filler materials cover other filler materials as long as, according to the designation
# in the appropriate international standard for the filler material, they have:
# - Equivalent mechanical properties,
# - Same type of covering or flux core,
# - Same nominal chemical composition, and
# - The same or lower hydrogen content.

# When impact testing is required by the application standard at temperatures less than −20 °C,
# for processes:
#     - 111 (Manual Metal Arc),
#     - 114 (Self-shielded flux-cored arc welding),
#     - 12 (Submerged arc welding),
#     - 136 (Tubular cored arc welding with active gas), and
#     - 132 (Metal inert gas welding),
# according to ISO 4063, the range of validity is restricted to the **manufacturer trade name**
# of the filler material used in the procedure test.

# In this case, it is also permissible to change the manufacturer of filler material to another
# with the same **compulsory part of the designation** provided:
#     - One additional test piece is welded using the **maximum heat input** qualified, and
#     - Only **weld metal impact test specimens** shall be tested.

# This does **not** apply to **solid wire and rods** with the same designation
# and nominal chemical composition.




# 📌 Interpretation Logic (Clause 8.4.4):

# ------------------------------------------------------------
# 📖 INTERPRETATION OF ISO 15614-1 CLAUSE 8.4.4 (in code comments)
# ------------------------------------------------------------

# 🧾 General Rule for Substitution of Filler Materials:
# - A filler material can be substituted with another only if the following are equivalent:
#     1. Mechanical properties (e.g., tensile/yield strength)
#     2. Type of covering (e.g., basic, rutile) or flux core
#     3. Nominal chemical composition (main alloy elements)
#     4. Hydrogen content (must be the same or lower)

# ✅ These equivalencies are based on international standards like ISO, AWS, etc.
# ✅ If these match, substitution is generally allowed without further testing.

# 🔒 RESTRICTION WHEN IMPACT TEST IS REQUIRED AT TEMPERATURES < -20 °C:
# - Applies only when:
#     a. Impact testing is required (impact = "yes")
#     b. The impact test temperature is below -20 °C
#     c. The welding process is any of the following ISO 4063 codes:
#        → 111 (Manual Metal Arc), 114, 12 (SAW), 132, or 136

# 🚫 In this case:
#     → Substitution is **not allowed**
#     → Approval is limited strictly to the same *manufacturer or trade name* used during testing

# 🧪 EXCEPTION WITH REQUALIFICATION:
# - If the designation (code) is the same and only the *manufacturer* is changing,
#   then substitution is allowed *only* if:
#     → An additional test piece is welded using the *maximum qualified heat input*
#     → Only weld metal impact specimens are tested

# ❌ This exception does NOT apply in this script's logic. We are assuming strict restriction
#    without performing requalification (no new testing is coded here).

# ✅ EXEMPTION FOR SOLID WIRES AND RODS:
# - Solid wires and rods with the same designation and nominal chemical composition
#   are **not** subject to this restriction.
# - This code assumes that solid wires are allowed, but identification of solid wires
#   is not automated here — manual validation may be needed.

# ⚙️ Code Decision Output:
# - If impact test < -20°C and restricted process is used → Requalification required
# - Otherwise → No restriction (substitution allowed if standard equivalencies hold)

# ✅ OUTPUT VALUES:
#     → Filler Material Make = "No substitution — Requalification required"
#     → Filler Designation   = "No substitution — Requalification required"
#   OR
#     → Filler Material Make = "No restriction"
#     → Filler Designation   = original value


# ------------------------------------------------------------------------
# 📊 Summary Table — Clause 8.4.4 Validity Rules
# ------------------------------------------------------------------------
# | Case                                                        | Qualification Valid For                             |
# |-------------------------------------------------------------|-----------------------------------------------------|
# | Filler with same mech. props, flux/core, chem, H₂ content   | Equivalent fillers from other makes                 |
# | Impact testing < -20°C + process 111/114/12/136/132         | Only same manufacturer qualified                    |
# | Changing filler (same designation) in above case            | Requires 1 extra test piece with weld metal impact  |
# | Solid wire/rod, same designation/chemistry                  | No restriction on manufacturer                      |
# ------------------------------------------------------------------------



from tabulate import tabulate

# ------------------------------------------------------------
# 🔧 Helper Functions for Safe Data Extraction
# ------------------------------------------------------------

# Extracts string values from the dataframe
def extract_str(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return str(row.iloc[0]["Actual Value"]).strip() if not row.empty else ""

# Extracts numeric (float) values from the dataframe
def extract_float(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    try:
        return float(row.iloc[0]["Actual Value"]) if not row.empty else None
    except (ValueError, TypeError):
        return None

# ------------------------------------------------------------
# 📌 Essential Variables:
# 1. Filler Material Make (Manufacturer / Trade Name)
# 2. Filler Material Designation
# Clause: ISO 15614-1 → 8.4.4 Filler Material Substitution Rules
# ------------------------------------------------------------

# 🔍 Extract input values from dataframe using safe functions
filler_make = extract_str(essential_variables_df, "filler material make")
filler_designation = extract_str(essential_variables_df, "filler material designation")
welding_process = extract_str(essential_variables_df, "welding process").lower()
impact_required = extract_str(essential_variables_df, "impact").lower()
impact_temp_val = extract_float(essential_variables_df, "impact temperature")

# ------------------------------------------------------------
# 🔎 Clause 8.4.4 Logic for Interpretation
# ------------------------------------------------------------

# 🧾 General Rule:
# Filler materials can be substituted if mechanical properties,
# covering/core type, chemical composition, and hydrogen content match.

# 🚫 Restriction if all three conditions are true:
#   1. Impact testing is required ("yes")
#   2. Temperature < -20°C
#   3. Process is one of ISO 4063 restricted codes
restricted_processes = ["111", "114", "12", "132", "136"]

is_restricted_process = any(code in welding_process for code in restricted_processes)
is_impact_low = impact_temp_val is not None and impact_temp_val < -20
restriction_applies = impact_required == "yes" and is_restricted_process and is_impact_low

# 🧾 Determine final output based on restriction
if restriction_applies:
    output_make = "No substitution — Requalification required"
    output_designation = "No substitution — Requalification required"
else:
    output_make = "No restriction"
    output_designation = filler_designation

# ------------------------------------------------------------
# 📋 Tabulate Output
# ------------------------------------------------------------

essential_var_table = [
    ["1. Filler Material Make", filler_make],
    ["2. Filler Material Designation", filler_designation]
]

input_table = [
    ["1. Filler Material Make", filler_make],
    ["2. Filler Material Designation", filler_designation],
    ["3. Welding Process", welding_process],
    ["4. Impact", "Yes" if impact_required == "yes" else "No"],
    ["5. Impact Temperature", f"{impact_temp_val} °C" if impact_temp_val is not None else "Not Provided"]
]

interpretation = [
    ["Based on ISO 15614-1 Clause 8.4.4:"],
    ["- Substitution allowed if properties, covering, chemistry, and hydrogen match."],
    ["- For processes 111, 114, 12, 132, 136 with impact tests below -20°C:"],
    ["  substitution is prohibited unless requalification is performed."]
]

output_table = [
    ["Filler Material Make", output_make],
    ["Filler Material Designation", output_designation]
]

# 🧩 Append to global result lists (assumed to exist)
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# 📤 Display Output Tables
print()
print(tabulate(essential_var_table, headers=["Essential Variable", "Value"], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", "Value"], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", "Result"], tablefmt="grid"))





+--------------------------------+-----------------------------+
| Essential Variable             | Value                       |
+================================+=============================+
| 1. Filler Material Make        | Bohler                      |
+--------------------------------+-----------------------------+
| 2. Filler Material Designation | ISO 14341-A-G 42 3 M21 3Si1 |
+--------------------------------+-----------------------------+

+--------------------------------+-----------------------------+
| Input Considered               | Value                       |
+================================+=============================+
| 1. Filler Material Make        | Bohler                      |
+--------------------------------+-----------------------------+
| 2. Filler Material Designation | ISO 14341-A-G 42 3 M21 3Si1 |
+--------------------------------+-----------------------------+
| 3. Welding Process             | mag (135)                   |
+----------------------

In [9]:
# ------------------------------------------------------------
# 📌 Next Essential Variable
# 1. Filler Material Size (Diameter)
# 8.4.5 Filler material size
# ------------------------------------------------------------

# It is permitted to change the size of filler material provided that the requirements of 8.4.7 are satisfied.

# NOTE When neither impact nor hardness testing is required, there is no limitation on filler material size.

# ------------------------------------------------------------
# 📋 Table (if present in standard context)
# ------------------------------------------------------------

# | Impact or Hardness Testing Required | Filler Size Change Allowed?      |
# |------------------------------------|----------------------------------|
# | No                                 | Yes, unrestricted                |
# | Yes                                | Yes, only if Clause 8.4.7 met    |



# ------------------------------------------------------------
# 📖 INTERPRETATION OF ISO 15614-1 CLAUSE 8.4.5 (in code)
# ------------------------------------------------------------

# 🧾 Clause Context:
# This clause relates to the qualification of changes in the filler material size (diameter)
# used during welding procedure qualification.

# 🔎 Main Interpretation Logic (as implemented in the code):

# Rule 1: When neither IMPACT nor HARDNESS testing is required:
# - There is NO limitation on the change of filler material size.
# - Filler diameter can be freely changed without any qualification restrictions.

# Rule 2: When either IMPACT or HARDNESS testing is required:
# - Changing the filler material size is only permitted if the HEAT INPUT remains
#   within the range qualified in the original procedure test (Clause 8.4.7).

# In other words:
# - The presence of mechanical testing (impact or hardness) activates a dependency
#   on compliance with the heat input limits.
# - Otherwise, filler size changes are unrestricted.

# ✅ Code Implementation Summary:
# - Extract 'impact' and 'hardness' values from input.
# - If either is "yes", mark mechanical_test_required = True.
# - If True → output: "No restriction (if heat input qualified as per Clause 8.4.7)"
# - Else → output: "No restriction on filler size"

# 🔔 This logic assumes that verification of Clause 8.4.7 (heat input) compliance
#     is handled elsewhere or manually confirmed.




from tabulate import tabulate

# ------------------------------------------------------------
# 📌 Essential Variable:
# 1. Filler Material Size (Diameter)
# Clause: ISO 15614-1 → 8.4.5 Filler Material Size
# ------------------------------------------------------------

# 🔧 Helper function to safely extract string values
def extract_str(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return str(row.iloc[0]["Actual Value"]).strip().lower() if not row.empty else ""

# 🔍 Extract input values
impact_val = extract_str(essential_variables_df, "impact")
hardness_val = extract_str(essential_variables_df, "hardness")

# ✅ Normalize display values for table output
impact_display = "Yes" if impact_val == "yes" else "No"
hardness_display = "Yes" if hardness_val == "yes" else "No"

# ------------------------------------------------------------
# 🔎 Clause 8.4.5 Logic for Interpretation
# ------------------------------------------------------------

# 🧾 Interpretation:
# - If neither impact nor hardness test is required → unrestricted filler size
# - If either test is required → filler size change allowed only if heat input stays within qualified limits

mechanical_test_required = impact_val == "yes" or hardness_val == "yes"

if mechanical_test_required:
    filler_size_output = "No restriction (If Heat Input within qualified limit)"
else:
    filler_size_output = "No restriction on filler size"

# ------------------------------------------------------------
# 📋 Tabulate Output
# ------------------------------------------------------------

essential_table = [
    ["1. Filler Material Size (Diameter)", ""]
]

input_table = [
    ["1. Impact", impact_display],
    ["2. Hardness", hardness_display]
]

interpretation_table = [
    ["Based on ISO 15614-1 Clause 8.4.5:"],
    ["- Filler diameter may be changed if Clause 8.4.7 (Heat Input) is satisfied."],
    ["- If neither impact nor hardness test is required, diameter may be freely changed."]
]

output_table = [
    ["Filler Material Size (Diameter)", filler_size_output]
]

# 🧩 Append to global result collectors
interpretation_rows.extend(interpretation_table)
essential_output_rows.extend(output_table)

# 📤 Display Output Tables
print()
print(tabulate(essential_table, headers=["Essential Variable", "Value"], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", "Value"], tablefmt="grid"))
print()
print(tabulate(interpretation_table, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", "Result"], tablefmt="grid"))




+------------------------------------+---------+
| Essential Variable                 | Value   |
+====================================+=========+
| 1. Filler Material Size (Diameter) |         |
+------------------------------------+---------+

+--------------------+---------+
| Input Considered   | Value   |
+====================+=========+
| 1. Impact          | Yes     |
+--------------------+---------+
| 2. Hardness        | Yes     |
+--------------------+---------+

+------------------------------------------------------------------------------------+
| Clause Interpretation                                                              |
+====================================================================================+
| Based on ISO 15614-1 Clause 8.4.5:                                                 |
+------------------------------------------------------------------------------------+
| - Filler diameter may be changed if Clause 8.4.7 (Heat Input) is satisfied.        |

In [10]:
# ------------------------------------------------------------
# 📌 Essential Variable
# 1. Type of Current and Polarity
# Clause: ISO 15614-1 → 8.4.6 Type of current
# ------------------------------------------------------------

# The qualification is given for the type of current [alternating current (AC), direct current (DC), pulsed
# current] and polarity used in the welding procedure test.

# For process 111, alternating current also qualifies direct current (both polarities) unless impact testing is required.

# ------------------------------------------------------------
# 📖 INTERPRETATION OF ISO 15614-1 CLAUSE 8.4.6 (in code)
# ------------------------------------------------------------

# 🧾 Clause Context:
# This clause relates to the type of welding current and polarity used
# during welding procedure qualification testing (WPQR).

# 🔎 Main Interpretation Logic (as implemented in the code):

# Rule 1: Default Behavior
# - The qualification only applies to the **exact type of current** (AC, DC, pulsed DC)
#   and **polarity** (DCEN, DCEP) used during the original test.
# - No other current or polarity is automatically covered or transferable.

# Rule 2: Exception for Process 111 (Manual Metal Arc Welding)
# - If the welding process is "111" (MMAW / Shielded Metal Arc Welding),
# - And if **alternating current (AC)** was used in the procedure test,
# - And if **impact testing is not required**,
# THEN:
#   → The qualification **also covers** DCEN and DCEP (both DC polarities),
#     in addition to AC.

# Rule 3: When Impact Testing is Required
# - If impact testing *is* required, then **no extension** is permitted.
#   → Only the current/polarity used in the original WPQR is qualified.

# ✅ Code Output Summary:
# - If (process 111 + AC used + no impact) → Output = "AC, DCEN, DCEP"
# - If (process 111 + AC used + impact = yes) → Output = "AC only"
# - Else → Output = current used in WPQR (as provided in input)





from tabulate import tabulate

# ------------------------------------------------------------
# 📌 Essential Variable:
# 1. Type of Current and Polarity
# Clause: ISO 15614-1 → 8.4.6 Type of Current
# ------------------------------------------------------------

# 🔧 Helper function to extract string values
def extract_str(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return str(row.iloc[0]["Actual Value"]).strip() if not row.empty else ""

# 🔍 Extract input values
welding_process = extract_str(essential_variables_df, "welding process").lower()
current_polarity = extract_str(essential_variables_df, "current and polarity").upper()
impact_required = extract_str(essential_variables_df, "impact").lower()

# ------------------------------------------------------------
# 🔎 Clause 8.4.6 Logic for Interpretation
# ------------------------------------------------------------

# 🧾 Rule:
# - Only the current and polarity used in the WPQR is qualified
# - Exception:
#   → If welding process is 111 (MMAW)
#   → And AC is used
#   → And impact test is NOT required
#   → Then DCEN and DCEP are also considered qualified

if "111" in welding_process and "AC" in current_polarity:
    if impact_required == "no":
        output_current = "AC, DCEN, DCEP"
    else:
        output_current = "AC only"
else:
    output_current = current_polarity

# ------------------------------------------------------------
# 📋 Tabulate Output
# ------------------------------------------------------------

essential_var_table = [
    ["1. Type of Current and Polarity", current_polarity]
]

input_table = [
    ["1. Welding Process", welding_process],
    ["2. Current and Polarity", current_polarity],
    ["3. Impact", "Yes" if impact_required == "yes" else "No"]
]

interpretation = [
    ["Based on ISO 15614-1 Clause 8.4.6:"],
    ["- Only the current/polarity used during testing is qualified."],
    ["- Exception: For Process 111 with AC (and no impact test), AC, DCEN, and DCEP are all qualified."]
]

output_table = [
    ["Type of Current and Polarity", output_current]
]

# 🧩 Append to global result collectors
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# 📤 Display Output Tables
print()
print(tabulate(essential_var_table, headers=["Essential Variable", "Value"], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", "Value"], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", "Result"], tablefmt="grid"))



+---------------------------------+-------------------------+
| Essential Variable              | Value                   |
+=================================+=========================+
| 1. Type of Current and Polarity | DCEP (REVERSE POLARITY) |
+---------------------------------+-------------------------+

+-------------------------+-------------------------+
| Input Considered        | Value                   |
+=========================+=========================+
| 1. Welding Process      | mag (135)               |
+-------------------------+-------------------------+
| 2. Current and Polarity | DCEP (REVERSE POLARITY) |
+-------------------------+-------------------------+
| 3. Impact               | Yes                     |
+-------------------------+-------------------------+

+--------------------------------------------------------------------------------------------------+
| Clause Interpretation                                                                            |

In [11]:
# ------------------------------------------------------------
# 📌 Essential Variables
# 1. Mode of Metal Transfer
# 2. Power Source / Waveform Control
# Clause: ISO 15614-1 → 8.5.2.3 Transfer mode
# ------------------------------------------------------------

# 🔹 8.5.2.3.1 General
# For solid and metal cored wires, the qualification using short circuiting transfer qualifies only short
# circuiting transfer. Qualification using spray, pulse or globular transfer qualifies spray, pulse and
# globular transfer.

# 🔹 8.5.2.3.2 For level 2 — Waveform controlled welding
# When a power source with waveform control (see ISO/TR 18491) is used, the power source
# manufacturer and waveform control mode shall be recorded on the WPQR including all other pertinent
# information.
# Change from one manufacturer to another manufacturer of the power source or a change in the
# waveform control mode requires a new qualification test.

# 🔹 8.5.2.3.3 For level 2 — Welding with pulsed mode (without waveform control)
# When a power source with pulsed mode is used, the identification of the power source manufacturer
# shall be recorded on the WPQR including all pertinent information.
# Change from one manufacturer to another manufacturer of the power source does not require a new
# qualification test.

# 🔹 8.5.2.3.4 For level 2 — Welding without pulsed mode and without waveform control
# When the power source used to qualify a WPQR has been used without waveform control, an
# identification of the power source manufacturer is required.
# Change from one manufacturer to another manufacturer of the power source does not require a new
# qualification test.


# ------------------------------------------------------------
# 📖 INTERPRETATION OF ISO 15614-1 CLAUSE 8.5.2.3 (in code)
# ------------------------------------------------------------

# 🔹 8.5.2.3.1 — Transfer Mode Qualification Rules:
# - If short-circuiting metal transfer is used:
#     → Only short-circuiting mode is qualified.
# - If spray, globular, or pulse transfer is used:
#     → These three modes qualify each other (spray ↔ pulse ↔ globular).

# 🔹 8.5.2.3.2 — Waveform Controlled Welding (Level 2):
# - If a waveform-controlled power source is used:
#     → The waveform mode and manufacturer **must be recorded** on the WPQR.
#     → Any change to either (manufacturer or waveform mode) requires **requalification**.

# 🔹 8.5.2.3.3 — Pulsed Mode Without Waveform Control (Level 2):
# - If pulsed transfer is used **without waveform control**:
#     → The manufacturer must be recorded.
#     → A change in manufacturer **does NOT require** requalification.

# 🔹 8.5.2.3.4 — Non-Pulsed, Non-Waveform Welding (Level 2):
# - If the power source does **not** use waveform control or pulse:
#     → The manufacturer must still be recorded.
#     → A change in manufacturer **does NOT require** requalification.

# ✅ Summary of Coded Interpretation:
# - If "short" appears in transfer mode:
#     → Output: "Short Circuiting Only"
# - If "spray", "pulse", or "globular" appears:
#     → Output: "Spray / Pulse / Globular"

# - If "waveform" appears in transfer mode string:
#     → Output: manufacturer + "(Waveform Controlled – No Change Allowed)"
# - Else if "pulse" appears (but no waveform):
#     → Output: manufacturer + "(Pulse – Change Allowed)"
# - Else:
#     → Output: manufacturer + "(Standard – Change Allowed)"

# 🔔 Assumption:
# - The field 'mode of metal transfer' contains sufficient information to infer both:
#     1. Transfer mode (short/pulse/spray/globular)
#     2. Whether waveform control is used (based on keyword "waveform")




from tabulate import tabulate

# ------------------------------------------------------------
# 📌 Essential Variables
# 1. Mode of Metal Transfer
# 2. Power Source / Waveform Control
# Clause: ISO 15614-1 → 8.5.2.3 Transfer Mode
# ------------------------------------------------------------

# 🔧 Helper function for string extraction
def extract_str(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return str(row.iloc[0]["Actual Value"]).strip().lower() if not row.empty else ""

# 🔍 Extract input values
transfer_mode = extract_str(essential_variables_df, "mode of metal transfer")
power_source = extract_str(essential_variables_df, "machine make").title()

# For now, assume waveform info is part of 'transfer_mode' field (can be extracted separately if needed)
waveform_info = transfer_mode

# ------------------------------------------------------------
# 🔎 Transfer Mode Qualification Logic (8.5.2.3.1)
# ------------------------------------------------------------

# - Short circuiting only qualifies short circuiting
# - Spray, pulse, globular qualify each other

if "short" in transfer_mode:
    transfer_output = "Short Circuiting Only"
elif any(mode in transfer_mode for mode in ["spray", "pulse", "globular"]):
    transfer_output = "Spray / Pulse / Globular"
else:
    transfer_output = "Manual Review Required"

# ------------------------------------------------------------
# 🔎 Power Source / Waveform Interpretation (8.5.2.3.2 to 8.5.2.3.4)
# ------------------------------------------------------------

uses_waveform = "waveform" in waveform_info
uses_pulse = "pulse" in transfer_mode

if uses_waveform:
    power_source_output = f"{power_source} (Waveform Controlled – No Change Allowed)"
elif uses_pulse:
    power_source_output = f"{power_source} (Pulse – Change Allowed)"
else:
    power_source_output = f"{power_source} (Standard – Change Allowed)"

# ------------------------------------------------------------
# 📋 Tabulate Output
# ------------------------------------------------------------

essential_var_table = [
    ["1. Mode of Metal Transfer", ""],
    ["2. Power Source / Waveform Control", ""]
]

input_table = [
    ["1. Mode of Metal Transfer", transfer_mode.title()],
    ["2. Power Source", power_source]
]

interpretation = [
    ["Based on ISO 15614-1 Clause 8.5.2.3:"],
    ["- Short-circuiting qualifies only short circuit."],
    ["- Spray, pulse, and globular modes qualify each other."],
    ["- Waveform-controlled welding: power source/waveform change not allowed."],
    ["- Pulse without waveform: power source change allowed."],
    ["- No pulse/waveform: power source manufacturer may be changed."]
]

output_table = [
    ["Mode of Metal Transfer", transfer_output],
    ["Power Source / Waveform Control", power_source_output]
]

# 🧩 Append to global lists
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# 📤 Display Tables
print()
print(tabulate(essential_var_table, headers=["Essential Variable", "Value"], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", "Value"], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", "Result"], tablefmt="grid"))



+------------------------------------+---------+
| Essential Variable                 | Value   |
+====================================+=========+
| 1. Mode of Metal Transfer          |         |
+------------------------------------+---------+
| 2. Power Source / Waveform Control |         |
+------------------------------------+---------+

+---------------------------+---------+
| Input Considered          | Value   |
+===========================+=========+
| 1. Mode of Metal Transfer | Pulse   |
+---------------------------+---------+
| 2. Power Source           |         |
+---------------------------+---------+

+--------------------------------------------------------------------------+
| Clause Interpretation                                                    |
+==========================================================================+
| Based on ISO 15614-1 Clause 8.5.2.3:                                     |
+-----------------------------------------------------------------

In [12]:
# ------------------------------------------------------------
# 📌 Next Essential Variable
# 1. Heat Input (kJ/mm) or Arc Energy
# Clause: ISO 15614-1 → 8.4.7 Heat input (arc energy)
# ------------------------------------------------------------

# The heat input can be replaced by arc energy (J/mm). The arc energy shall be calculated in accordance
# with ISO/TR 18491. When using the calculation for the heat input, the k-factor according to
# ISO/TR 17671-1 shall be considered. The kind of calculation, either heat input or arc energy, shall be
# documented.

# For level 2: When impact requirements apply, the
# upper limit of the heat input qualified is 25 % greater than used in welding the test piece.

# For covered electrode, the heat input average shall be calculated for each used diameter in order to
# define the qualified heat input.

# For process 111, the heat input may also be measured by the run out length per unit length of electrode.

# When the welding time is too short and when the length of the weld is not significant (e.g. for small
# repair, for tack welds), the heat input need not to be verified; only the adjustable parameters should be
# checked like amperage and/or voltage.

# Arc energy and heat input are measures of the heat generated by the arc. Whereas, in the past, these
# were different terms for the same measure, they are now calculated in different ways. Either arc energy
# or heat input may be used for welding control, calculated in accordance with ISO/TR 18491.

# When hardness requirements apply, the lower limit of the
# heat input qualified is 25 % lower than that used in welding the test piece.

# If welding procedure test has been performed at both a high and a low heat
# input level, then all intermediate heat input levels are also qualified. It is not necessary to calculate
# every run.

# ------------------------------------------------------------
# 📖 INTERPRETATION OF ISO 15614-1 CLAUSE 8.4.7 (in code)
# ------------------------------------------------------------

# 🔹 Clause Scope:
# This clause defines how **heat input (kJ/mm)** or **arc energy (J/mm)** affects the
# qualified range of welding procedures, especially when mechanical testing applies.

# 🔹 Accepted Calculation Methods:
# - Heat input or arc energy may be used for qualification (they are now calculated differently).
# - Arc energy must follow ISO/TR 18491.
# - Heat input calculation must consider the k-factor from ISO/TR 17671-1.
# - The method used (heat input vs arc energy) must be clearly documented on the WPQR.

# 🔹 Code Logic (Test-Based Qualification Ranges):
# 1️⃣ If **Impact testing** is required:
#     → The **maximum heat input** is limited to **+25%** above the value used in the test piece.
#
# 2️⃣ If **Hardness testing** is required:
#     → The **minimum heat input** is limited to **−25%** below the value used in the test piece.
#
# 3️⃣ If **both** Impact and Hardness are required:
#     → The valid range is from **−25% to +25%** of the test value.
#
# 4️⃣ If **neither** test is required:
#     → There is **no restriction** on the heat input.

# 🔹 Special Notes (not implemented in logic but relevant):
# - For process 111 (MMAW), run-out length may also be used to determine heat input.
# - For short welds or repairs where time/length are not significant:
#     → Heat input does not need to be verified.
#     → Only adjustable parameters (amperage/voltage) must be controlled.

# ✅ Code Output Summary:
# - If heat input is known and testing applies:
#     → Outputs calculated qualified range (min/max or both).
# - If neither test applies:
#     → Outputs "No restriction".
# - If input is missing:
#     → Outputs "Manual review required".




from tabulate import tabulate
import re

# ------------------------------------------------------------
# 📌 Essential Variable
# 1. Heat Input (kJ/mm) or Arc Energy (J/mm)
# Clause: ISO 15614-1 → 8.4.7 Heat Input
# ------------------------------------------------------------

# 🔧 Helper: Extract string value from dataframe
def extract_str(df, keyword):
    row = df[df["Essential Variable"].str.contains(re.escape(keyword), case=False, na=False, regex=True)]
    return str(row.iloc[0]["Actual Value"]).strip() if not row.empty else ""

# 🔧 Helper: Extract first number from a string
def extract_numeric(val):
    nums = re.findall(r"\d+\.?\d*", str(val))
    return float(nums[0]) if nums else None

# 🔍 Extract input values
heat_input_raw = extract_str(essential_variables_df, "heat input (kJ/mm)")
impact_required = extract_str(essential_variables_df, "impact").lower()
hardness_required = extract_str(essential_variables_df, "hardness").lower()

heat_input_val = extract_numeric(heat_input_raw)

# ✅ Display values for table output
impact_display = "Yes" if impact_required == "yes" else "No"
hardness_display = "Yes" if hardness_required == "yes" else "No"

# ------------------------------------------------------------
# 🔎 Clause 8.4.7 Interpretation Logic
# ------------------------------------------------------------

# Impact → upper limit = +25%
# Hardness → lower limit = −25%
# Both → range must stay within ±25%
# Neither → unrestricted

if heat_input_val:
    upper = round(heat_input_val * 1.25, 3) if impact_required == "yes" else None
    lower = round(heat_input_val * 0.75, 3) if hardness_required == "yes" else None

    if upper and lower:
        heat_input_output = f"{lower} – {upper} kJ/mm"
    elif upper:
        heat_input_output = f"≤ {upper} kJ/mm"
    elif lower:
        heat_input_output = f"≥ {lower} kJ/mm"
    else:
        heat_input_output = "No restriction"
else:
    heat_input_output = "Manual review required"

# ------------------------------------------------------------
# 📋 Tabulate Output
# ------------------------------------------------------------

essential_var_table = [
    ["1. Heat Input (kJ/mm) or Arc Energy", ""]
]

input_table = [
    ["1. Heat Input", heat_input_raw],
    ["2. Impact", impact_display],
    ["3. Hardness", hardness_display]
]

interpretation = [
    ["Based on ISO 15614-1 Clause 8.4.7:"],
    ["- Impact testing requires a max limit: +25% of WPQR heat input."],
    ["- Hardness testing requires a min limit: −25% of WPQR heat input."],
    ["- If both are required, heat input must stay within −25% to +25%."],
    ["- If neither is required, there is no heat input restriction."]
]

output_table = [
    ["Heat Input / Arc Energy", heat_input_output]
]

# 🧩 Append results to global lists
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# 📤 Display Tables
print()
print(tabulate(essential_var_table, headers=["Essential Variable", "Value"], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", "Value"], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", "Result"], tablefmt="grid"))



+-------------------------------------+---------+
| Essential Variable                  | Value   |
+=====================================+=========+
| 1. Heat Input (kJ/mm) or Arc Energy |         |
+-------------------------------------+---------+

+--------------------+---------+
| Input Considered   | Value   |
+====================+=========+
| 1. Heat Input      | 0.61    |
+--------------------+---------+
| 2. Impact          | Yes     |
+--------------------+---------+
| 3. Hardness        | Yes     |
+--------------------+---------+

+-------------------------------------------------------------------+
| Clause Interpretation                                             |
+===================================================================+
| Based on ISO 15614-1 Clause 8.4.7:                                |
+-------------------------------------------------------------------+
| - Impact testing requires a max limit: +25% of WPQR heat input.   |
+-----------------------------

In [13]:
# ------------------------------------------------------------
# 📌 Next Essential Variable
# 1. Welding Position
# Clause: ISO 15614-1 → 8.4.2 Welding Positions
# ------------------------------------------------------------

# When neither impact nor hardness requirements are specified, welding of the test piece in any position
# (pipe or plate) qualifies for welding in all positions (pipe or plate).

# For qualification of all welding positions, the following requirements shall be fulfilled:
# — specimens for impact test shall be taken from the weld in the highest heat input position;
# — specimens for hardness test shall be taken from the weld in the lowest heat input position.

# To satisfy both hardness and impact requirements, two test pieces in different welding positions are
# required, unless a single position qualification is required or in case of when a fixed pipe is used for the
# qualification. Where qualification is required for all positions, both test pieces shall be subjected to full
# visual testing and further non-destructive methods.

# Vertical down welding (welding positions PG, PJ and J-L045) shall be qualified by a specific test piece.

# For material of group 10, the lowest and highest heat input positions shall be subjected to impact testing.

# NOTE For example, for butt welds in plate, the highest heat input position is normally PF and PA and the
# lowest heat input position is PC and PE.


# ------------------------------------------------------------
# 📖 INTERPRETATION OF ISO 15614-1 CLAUSE 8.4.2 (in code)
# ------------------------------------------------------------

# 🔹 Clause Scope:
# This clause defines how welding positions used during WPQR (procedure qualification)
# affect the range of qualified welding positions.

# 🔎 Interpretation Logic (as coded):

# 1️⃣ If **neither impact nor hardness testing** is required:
#     → Welding in **any position** (pipe or plate) during the test
#        qualifies the welder/procedure for **all positions** (pipe and plate).
#     ✅ Output: "All welding positions (Pipe & Plate)"

# 2️⃣ If **impact or hardness testing is required**:
#     → To qualify for all positions, the test must include:
#        - One position with **highest heat input** (for impact testing)
#        - One position with **lowest heat input** (for hardness testing)
#     → If only one position is tested:
#        ⚠️ Only that specific position is qualified.
#     ✅ Output varies:
#         - Single position → only that one is qualified
#         - Two or more → all listed are qualified if covering high/low positions

# 3️⃣ If **vertical-down positions** are tested (PG, PJ, J-L045):
#     → These require a **separate qualification** regardless of other positions.
#     ✅ Output: "Tested vertical-down positions (PG, PJ, J-L045)"

# 4️⃣ If **input is unclear** or cannot be parsed:
#     → A **manual review is required** to confirm what's qualified.

# 🔔 Additional Notes from Clause:
# - When qualifying all positions with both hardness and impact tests,
#   two test pieces are needed (unless it's a fixed pipe qualification).
# - For material group 10, both the highest and lowest heat input positions must be impact tested.
# - Example heat input order for plate:
#     - Highest: PF, PA
#     - Lowest:  PC, PE

# ✅ Code Output Summary:
# - Based on number of positions found and testing requirements,
#   the script outputs what welding positions are considered qualified.





from tabulate import tabulate
import re

# ------------------------------------------------------------
# 📌 Essential Variable
# 1. Welding Position
# Clause: ISO 15614-1 → 8.4.2 Welding Positions
# ------------------------------------------------------------

# 🔧 Helper to extract string value from dataframe
def extract_str(df, keyword):
    row = df[df["Essential Variable"].str.contains(re.escape(keyword), case=False, na=False, regex=True)]
    return str(row.iloc[0]["Actual Value"]).strip() if not row.empty else ""

# 🔍 Extract input values
weld_position_raw = extract_str(essential_variables_df, "welding position")
impact_required = extract_str(essential_variables_df, "impact").lower()
hardness_required = extract_str(essential_variables_df, "hardness").lower()

# ✅ Normalize and parse positions
positions_found = re.findall(r"\b[A-Z]{2,3}(?:-L\d+)?\b", weld_position_raw.upper())
vertical_down_positions = {"PG", "PJ", "J-L045"}
notes = []

# ------------------------------------------------------------
# 🔎 Clause 8.4.2 Interpretation Logic
# ------------------------------------------------------------

# - No impact/hardness → all positions are qualified
# - One position → only that position qualified
# - Two or more → may qualify all if high & low heat input are covered
# - PG, PJ, J-L045 require separate qualification

if any(pos in vertical_down_positions for pos in positions_found):
    qualified_scope = "Tested vertical-down positions (PG, PJ, J-L045)"
    notes.append("✅ Vertical-down positions require separate qualification.")
elif impact_required != "yes" and hardness_required != "yes":
    qualified_scope = "All welding positions (Pipe & Plate)"
    notes.append("✅ No impact/hardness required — all positions are qualified.")
elif len(positions_found) == 1:
    qualified_scope = f"{weld_position_raw}"
    notes.append("⚠️ One position tested — only that position is qualified.")
elif len(positions_found) >= 2:
    qualified_scope = f"All tested positions: {', '.join(positions_found)}"
    notes.append("✅ Two or more positions tested — check if they cover high/low heat input.")
else:
    qualified_scope = "Manual Review Required"
    notes.append("❌ Welding position unclear — review needed.")

# ------------------------------------------------------------
# 📋 Tabulate Output
# ------------------------------------------------------------

essential_var_table = [["1. Welding Position", ""]]

input_table = [
    ["1. Welding Positions", weld_position_raw],
    ["2. Impact", "Yes" if impact_required == "yes" else "No"],
    ["3. Hardness", "Yes" if hardness_required == "yes" else "No"]
]

interpretation = [
    ["Based on ISO 15614-1 Clause 8.4.2:"],
    ["- No impact/hardness → all positions qualified."],
    ["- With mechanical tests → two positions (high/low heat) needed to qualify all."],
    ["- Vertical-down (PG, PJ, J-L045) must be separately qualified."]
]

output_table = [["Welding Position", qualified_scope]]

# 🧩 Append results to global collectors
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# 📤 Display Final Output
print()
print(tabulate(essential_var_table, headers=["Essential Variable", "Value"], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", "Value"], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", "Result"], tablefmt="grid"))

# 📝 Show additional notes
if notes:
    print("\n🔍 Notes:")
    for note in notes:
        print(f"- {note}")



+----------------------+---------+
| Essential Variable   | Value   |
+======================+=========+
| 1. Welding Position  |         |
+----------------------+---------+

+----------------------+---------+
| Input Considered     | Value   |
+======================+=========+
| 1. Welding Positions | PA (1G) |
+----------------------+---------+
| 2. Impact            | Yes     |
+----------------------+---------+
| 3. Hardness          | Yes     |
+----------------------+---------+

+--------------------------------------------------------------------------------+
| Clause Interpretation                                                          |
+================================================================================+
| Based on ISO 15614-1 Clause 8.4.2:                                             |
+--------------------------------------------------------------------------------+
| - No impact/hardness → all positions qualified.                                |
+--------

In [14]:
# ------------------------------------------------------------
# 📌 Essential Variable
# 1. Preheat Temperature Min
# Clause: ISO 15614-1 → 8.4.8 Preheat Temperature
# ------------------------------------------------------------

# A decrease of more than 50 °C from the recorded preheating temperature on the WPQR requires a requalification.

# A decrease of the preheating temperature is permitted only if the requirements concerning preheating
# (especially the combined thickness) are fulfilled, e.g. ISO/TR 17671-2.

# The pre-heat temperature may be specified, e.g. by a material data sheet, and will depend on the
# material thickness.


# ------------------------------------------------------------
# 📖 INTERPRETATION OF ISO 15614-1 CLAUSE 8.4.8 (in code)
# ------------------------------------------------------------

# 🔹 Clause Context:
# This clause governs how much reduction in preheat temperature is permitted
# when qualifying or applying a welding procedure based on WPQR data.

# 🔎 Code Interpretation Logic:

# 1️⃣ The preheat temperature recorded during the welding procedure qualification (WPQR)
#     is considered the **baseline** value.

# 2️⃣ A **reduction** of **up to 50°C** from the recorded WPQR preheat temperature
#     is allowed **without requalification**.

# 3️⃣ However, under **no circumstances** shall the resulting preheat temperature
#     fall **below 30°C**. This is a hard lower bound, regardless of original WPQR value.

#     ✅ Example:
#         - If WPQR preheat was 100°C → Min qualified = 50°C
#         - If WPQR preheat was 60°C → Min qualified = 30°C (not 10°C)

# 4️⃣ The calculated result is:
#     → `max(preheat_val - 50, 30.0)`

# 5️⃣ If the preheat value is missing or cannot be extracted:
#     → Output is set to "Manual Review Required"

# 🔔 Additional Note (not implemented but clause-referenced):
# - If preheat is specified by a material data sheet, that guidance must be followed.
# - Preheat requirements may vary based on combined thickness (see ISO/TR 17671-2).

# ✅ Output Summary:
# - If value is valid → returns adjusted minimum preheat as qualified limit
# - If value is missing → prompts user for manual check




import re
from tabulate import tabulate

# ------------------------------------------------------------
# 📌 Essential Variable
# 1. Preheat Temperature Min
# Clause: ISO 15614-1 → 8.4.8 Preheat Temperature
# ------------------------------------------------------------

# 🔧 Helper: Extract string value from dataframe
def extract_str(df, keyword):
    row = df[df["Essential Variable"].str.contains(re.escape(keyword), case=False, na=False, regex=True)]
    return str(row.iloc[0]["Actual Value"]).strip() if not row.empty else ""

# 🔍 Extract raw input value
preheat_raw = extract_str(essential_variables_df, "preheat temperature min")

# 🔢 Extract numeric value (first match)
match = re.findall(r"\d+\.?\d*", preheat_raw)
preheat_val = float(match[0]) if match else None

# ------------------------------------------------------------
# 🔎 Clause 8.4.8 Interpretation Logic
# ------------------------------------------------------------

# - A reduction of up to 50°C is permitted from the WPQR recorded value.
# - However, preheat shall not go below 30°C under any condition.

if preheat_val:
    qualified_min = max(preheat_val - 50.0, 30.0)
    interpretation = [
        ["Based on ISO 15614-1 Clause 8.4.8:"],
        ["- A reduction of up to 50°C from WPQR value is permitted."],
        ["- Minimum preheat must not fall below 30°C."],
        [f"- Calculated qualified minimum preheat: {qualified_min} °C"]
    ]
    output_table = [["Preheat Temperature Min", f"{qualified_min} °C"]]
else:
    interpretation = [["Based on ISO 15614-1 Clause 8.4.8:"]]
    interpretation.append(["- Preheat temperature could not be extracted — manual review required."])
    output_table = [["Preheat Temperature Min", "Manual Review Required"]]

# ------------------------------------------------------------
# 📋 Tabulate Output
# ------------------------------------------------------------

essential_var_table = [["1. Preheat Temperature Min", ""]]

input_table = [
    ["1. Preheat Temperature Min", preheat_raw]
]

# 🧩 Append results to global collectors
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# 📤 Display Output Tables
print()
print(tabulate(essential_var_table, headers=["Essential Variable", "Value"], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", "Value"], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", "Result"], tablefmt="grid"))



+----------------------------+---------+
| Essential Variable         | Value   |
+============================+=========+
| 1. Preheat Temperature Min |         |
+----------------------------+---------+

+----------------------------+---------+
| Input Considered           | Value   |
+============================+=========+
| 1. Preheat Temperature Min | 30°C    |
+----------------------------+---------+

+-----------------------------------------------------------+
| Clause Interpretation                                     |
+===========================================================+
| Based on ISO 15614-1 Clause 8.4.8:                        |
+-----------------------------------------------------------+
| - A reduction of up to 50°C from WPQR value is permitted. |
+-----------------------------------------------------------+
| - Minimum preheat must not fall below 30°C.               |
+-----------------------------------------------------------+
| - Calculated qualified mini

In [15]:
# ------------------------------------------------------------
# 📌 Essential Variable
# 1. Interpass Temperature
# Clause: ISO 15614-1 → 8.4.9 Interpass Temperature
# ------------------------------------------------------------

# An increase of more than 50 °C in the maximum interpass temperature reached in the welding
# procedure test shall require re-qualification.

# An increased preheat temperature intentionally applied during welding of the capping passes to reduce
# hardness in HAZ of a welding procedure test shall be considered as an essential variable. Both minimum
# preheat temperature applied and the preheating temperatures applied during welding of the capping
# passes shall be reported.

# For level 2: The upper limit of the qualification is
# the highest interpass temperature reached in the
# welding procedure test for material groups 8, 10
# and 41 to 48.

# This limitation does not apply when a WPS qualified with a PWHT above the upper transformation
# temperature or when an austenitic material is solution annealed after welding.


# ------------------------------------------------------------
# 📖 INTERPRETATION OF ISO 15614-1 CLAUSE 8.4.9 (in code)
# ------------------------------------------------------------

# 🔹 Clause Scope:
# This clause sets limitations on interpass temperature qualification
# based on material group and post-weld heat treatment (PWHT) conditions.

# 🔎 Code Logic Interprets the Clause as Follows:

# 1️⃣ General Rule (applies to most material groups):
#     → The maximum interpass temperature qualified = value used during WPQR + 50°C.
#     ✅ This provides flexibility for production welding if not otherwise restricted.

# 2️⃣ Special Rule for Restricted Material Groups:
#     → If the parent material belongs to group 8, 10, or 41 to 48:
#         - The upper limit of the interpass temperature is fixed at the **value recorded in WPQR**.
#         - No increase is allowed beyond this value **unless an exception applies**.

# 3️⃣ Exception Rule:
#     → If any of the following are true, the special restriction is lifted:
#         - PWHT was performed above the **upper transformation temperature**, OR
#         - The material was **solution annealed** after welding.
#     ✅ In these cases, the general +50°C rule applies even for restricted groups.

# 4️⃣ Fallback:
#     → If interpass temperature value is missing or unreadable:
#         - The system will return **"Manual Review Required"**.

# ✅ Output Behavior Summary:
# - If general group → outputs qualified max as WPQR + 50°C
# - If restricted group without PWHT/anneal → max = WPQR value only
# - If exception applies → outputs WPQR + 50°C
# - If no value found → triggers manual review





import re
from tabulate import tabulate

# ------------------------------------------------------------
# 📌 Essential Variable
# 1. Interpass Temperature
# Clause: ISO 15614-1 → 8.4.9 Interpass Temperature
# ------------------------------------------------------------

# 🔧 Helper: Extract string value from dataframe
def extract_str(df, keyword):
    row = df[df["Essential Variable"].str.contains(re.escape(keyword), case=False, na=False, regex=True)]
    return str(row.iloc[0]["Actual Value"]).strip() if not row.empty else ""

# 🔍 Extract Inputs
interpass_raw = extract_str(essential_variables_df, "Interpass temperature Max")
material_group = extract_str(essential_variables_df, "Parent material group")
pwht_status = extract_str(essential_variables_df, "Post Weld Heat Treatement (PWHT)").lower()

# 🔢 Parse numeric interpass temperature
interpass_match = re.findall(r"\d+\.?\d*", interpass_raw)
interpass_val = float(interpass_match[0]) if interpass_match else None

# ------------------------------------------------------------
# 🔎 Clause 8.4.9 Logic for Evaluation
# ------------------------------------------------------------

# Rule 1: For general cases → max allowed = WPQR + 50°C
# Rule 2: For material groups 8, 10, and 41–48:
#   - No increase allowed unless PWHT > upper transformation temp or solution annealing is done.

restricted_groups = {"8", "10"} | {str(i) for i in range(41, 49)}
group_list = set(re.findall(r"\d+", material_group))  # handles multiple group values
is_restricted_group = bool(restricted_groups & group_list)

exception_applies = (
    "above upper transformation" in pwht_status or
    "solution anneal" in pwht_status
)

# ------------------------------------------------------------
# 📊 Determine Qualified Interpass Range
# ------------------------------------------------------------

if interpass_val is not None:
    if is_restricted_group and not exception_applies:
        qualified_max = f"{interpass_val:.1f} °C"
        interpretation = [
            ["Based on ISO 15614-1 Clause 8.4.9:"],
            ["- Material group is restricted (8, 10, 41–48)."],
            ["- PWHT exemption does NOT apply."],
            ["→ Maximum interpass temperature is limited to value used in WPQR."],
            [f"→ Qualified Max Interpass = {qualified_max}"]
        ]
    else:
        qualified_max = f"{interpass_val + 50:.1f} °C"
        interpretation = [
            ["Based on ISO 15614-1 Clause 8.4.9:"],
            ["- General rule: up to +50°C above WPQR interpass is allowed."],
            [f"→ Qualified Max Interpass = {qualified_max}"]
        ]
else:
    qualified_max = "Manual review required"
    interpretation = [
        ["Based on ISO 15614-1 Clause 8.4.9:"],
        ["- Interpass temperature not found — cannot apply qualification limits."]
    ]

# ------------------------------------------------------------
# 📋 Tabulate Output
# ------------------------------------------------------------

essential_var_table = [["1. Interpass Temperature", ""]]

input_table = [
    ["1. Interpass Temperature", interpass_raw],
    ["2. Parent Material Group", material_group],
    ["3. PWHT", pwht_status.title()]
]

output_table = [["Interpass Temperature", qualified_max]]

# 🧩 Append to global output lists
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# 📤 Display Output Tables
print()
print(tabulate(essential_var_table, headers=["Essential Variable", "Value"], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", "Value"], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", "Result"], tablefmt="grid"))




+--------------------------+---------+
| Essential Variable       | Value   |
+==========================+=========+
| 1. Interpass Temperature |         |
+--------------------------+---------+

+--------------------------+---------------+
| Input Considered         | Value         |
+==========================+===============+
| 1. Interpass Temperature | 250°C         |
+--------------------------+---------------+
| 2. Parent Material Group | 1.4 to 1.4    |
+--------------------------+---------------+
| 3. PWHT                  | Not Available |
+--------------------------+---------------+

+--------------------------------------------------------------+
| Clause Interpretation                                        |
+==============================================================+
| Based on ISO 15614-1 Clause 8.4.9:                           |
+--------------------------------------------------------------+
| - General rule: up to +50°C above WPQR interpass is allowed. |
+------

In [16]:
# ------------------------------------------------------------
# 📌 Essential Variable
# 1. Post-heating
# Clause: ISO 15614-1 → 8.4.10 Post-heating for hydrogen release
# ------------------------------------------------------------

# For level 2: The temperature and duration of
# post-heating for hydrogen release shall not be
# reduced. Post-heating shall not be omitted, but may
# be added.


# ------------------------------------------------------------
# 📖 INTERPRETATION OF ISO 15614-1 CLAUSE 8.4.10 (in code)
# ------------------------------------------------------------

# 🔹 Clause Scope:
# Post-heating for hydrogen release is an essential variable
# specifically addressed under Level 2 requirements.

# 🔎 Code Logic Interprets the Clause as Follows:

# 1️⃣ If the WPQR does NOT specify any post-heating (i.e., value is empty,
#     "NA", "None", etc.):
#     → Post-heating may be added in production without requalification.
#     ✅ This is permissible and does not invalidate the WPQR.

# 2️⃣ If the WPQR DOES include post-heating details:
#     → The recorded temperature and duration must NOT be reduced or omitted.
#     ✅ Post-heating is mandatory under the same conditions as tested.

# 3️⃣ Increase in post-heating temperature or duration:
#     → Allowed without needing requalification.
#     ✅ This aligns with clause allowance to "add" post-heating.

# 4️⃣ Practical Interpretation Implemented:
#     - Values like "NA", "none", "not applicable", and blank entries are treated
#       as absence of post-heating in WPQR.
#     - Any other value is treated as specified post-heating that must be respected.

# ✅ Output Behavior Summary:
# - If no post-heating → Output: "Not Applicable (may be added if needed)"
# - If post-heating exists → Output: "Must not reduce or omit"



import re
from tabulate import tabulate

# ------------------------------------------------------------
# 📌 Essential Variable
# 1. Post-heating
# Clause: ISO 15614-1 → 8.4.10 Post-heating for Hydrogen Release
# ------------------------------------------------------------

# 🔧 Helper: Extract string value from dataframe
def extract_str(df, keyword):
    row = df[df["Essential Variable"].str.contains(re.escape(keyword), case=False, na=False, regex=True)]
    return str(row.iloc[0]["Actual Value"]).strip().lower() if not row.empty else ""

# 🔍 Extract post-heating input
post_heat_val = extract_str(essential_variables_df, "post heating")

# ------------------------------------------------------------
# 🔎 Clause 8.4.10 Interpretation Logic
# ------------------------------------------------------------

# Rule 1: If no post-heating is defined in the WPQR
#   → It may be added in production if needed.

# Rule 2: If post-heating is defined in WPQR
#   → It must not be omitted or reduced.
#   → Increase in time/temperature is permitted.

no_postheat_values = {"", "not applicable", "na", "none", "n/a"}

if post_heat_val in no_postheat_values:
    postheat_output = "Not Applicable (may be added if needed)"
    interpretation = [
        ["Based on ISO 15614-1 Clause 8.4.10:"],
        ["- No post-heating was recorded in the WPQR."],
        ["- Post-heating may be added during production if required."]
    ]
else:
    postheat_output = "Must not reduce or omit"
    interpretation = [
        ["Based on ISO 15614-1 Clause 8.4.10:"],
        ["- Post-heating is recorded in the WPQR."],
        ["- It must not be reduced or omitted in production."],
        ["- Increasing duration or temperature is allowed."]
    ]

# ------------------------------------------------------------
# 📋 Tabulate Output
# ------------------------------------------------------------

essential_var_table = [["1. Post-heating", ""]]

input_table = [
    ["1. Post-heating", post_heat_val if post_heat_val else "Not Provided"]
]

output_table = [["Post-heating", postheat_output]]

# 🧩 Append to global output lists
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# 📤 Display Output Tables
print()
print(tabulate(essential_var_table, headers=["Essential Variable", "Value"], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", "Value"], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", "Result"], tablefmt="grid"))



+----------------------+---------+
| Essential Variable   | Value   |
+======================+=========+
| 1. Post-heating      |         |
+----------------------+---------+

+--------------------+----------------+
| Input Considered   | Value          |
+====================+================+
| 1. Post-heating    | not applicable |
+--------------------+----------------+

+------------------------------------------------------------+
| Clause Interpretation                                      |
+============================================================+
| Based on ISO 15614-1 Clause 8.4.10:                        |
+------------------------------------------------------------+
| - No post-heating was recorded in the WPQR.                |
+------------------------------------------------------------+
| - Post-heating may be added during production if required. |
+------------------------------------------------------------+

+-----------------------------+-----------------------

In [17]:
# 11 Designation
#
# The designation of a welder qualification shall comprise the following items in the order given
# (the system is arranged so that it can be used for computerization):
#
# a) The number of this part of ISO 9606:
#    - ISO 9606-1
#
# b) The essential variables:
#
# 1) Welding Processes
#    - Refer to sections 4.2, 5.2, and ISO 4063
#
# 2) Product Type
#    - Plate (P)
#    - Pipe (T)
#    - Refer to 4.3.1 and 5.3
#
# 3) Type of Weld
#    - Butt Weld (BW)
#    - Fillet Weld (FW)
#    - Refer to 5.4
#
# 4) Filler Material Group or Parent Material Group (Autogenous Welding)
#    - Refer to 5.5
#
# 5) Filler Material Types
#    - Refer to 5.6
#
# 6) Dimensions of the Test Piece
#    - Deposited thickness: s
#    - Material thickness: t
#    - Outside pipe diameter: D
#    - Refer to 5.7
#
# 7) Welding Positions
#    - Refer to 5.8 and ISO 6947
#
# 8) Weld Details
#    - Refer to 5.9




# 4.2 Reference Numbers of Welding Processes
#
# This part of ISO 9606 covers the following manual or partly mechanized welding processes.
# Reference numbers of welding processes for symbolic representations are listed in ISO 4063:
#
# 111 - Manual Metal Arc Welding (MMA)
# 114 - Self-shielded Tubular Cored Arc Welding
# 121 - Submerged Arc Welding with Solid Wire Electrode (partly mechanized)
# 125 - Submerged Arc Welding with Tubular Cored Electrode (partly mechanized)
# 131 - MIG Welding with Solid Wire Electrode
# 135 - MAG Welding with Solid Wire Electrode
# 136 - MAG Welding with Flux Cored Electrode
# 138 - MAG Welding with Metal Cored Electrode
# 141 - TIG Welding with Solid Filler Material (wire/rod)
# 142 - Autogenous TIG Welding
# 143 - TIG Welding with Tubular Cored Filler Material (wire/rod)
# 145 - TIG Welding using Reducing Gas and Solid Filler Material (wire/rod)
# 15  - Plasma Arc Welding
# 311 - Oxyacetylene Welding




# 4.3.1 For Test Pieces
#
# a   - design throat thickness
# BW  - butt weld
# D   - outside pipe diameter
# FW  - fillet weld
# l1  - length of test piece
# l2  - half-width of test piece
# lf  - examination length
# P   - plate
# s   - deposited thickness or fused metal thickness in butt welds
# t   - material thickness of test piece (plate or wall thickness)
# s1  - deposited thickness of test piece for welding process 1
# s2  - deposited thickness of test piece for welding process 2
# T   - pipe
# z   - leg length of fillet weld



# 5.4 Type of Weld
#
# The qualification test shall be carried out as butt or fillet welding.
# The following criteria are applicable:
#
# a) Butt welds cover butt welds in any type of joint except branch connections [see also c)].
#
# b) Butt welds do not qualify fillet welds or vice versa. It is, however, permissible to qualify a fillet weld
#    in combination with a butt weld, e.g. single bevel joint preparation with permanent material backing
#    (a minimum test piece thickness of 10 mm shall be used). See Annex C.
#    For this combination test, all testing requirements specified in this part of ISO 9606 shall be fulfilled
#    and associated ranges of qualification shall be given based on the test conditions.
#
# c) Butt welds in pipes qualify branch joints with an angle ≥ 60° and the same range of qualification as
#    in Tables 1 to 12. For a branch weld, the range of qualification is based on the outside diameter of the branch.
#
# d) For applications where the type of weld cannot be qualified by means of either a butt or fillet or for
#    branch connections of less than 60°, a specific test piece should be used to qualify the welder,
#    when specified (e.g. by the product standard).
#
# e) Butt welds may qualify fillet welds if a supplementary fillet weld test piece (see Figure 3) is welded
#    with each process, filler material (FM) group and electrode covering/core, in accordance with
#    Tables 3, 4, and 5.
#    The test piece shall be at least 10 mm thick, or the thickness of the butt weld test piece if the
#    thickness is less, and completed using a single layer in the PB position.
#    For this supplementary test, the welder shall be qualified for all fillet welds as given for the butt weld
#    qualification variables related to the range of qualification for fillet welds (e.g. Tables 7, 8, 9, 10 and 12).



# 5.5 Filler Material Grouping
#
# 5.5.1 General
# The qualification test shall be carried out with filler material from one of the groups listed in Table 2.
# When welding with filler materials outside the filler material grouping in Table 2, a separate test is required.
# The parent material used in a qualification test should be from any suitable material from ISO/TR 15608,
# material groups 1 to 11.
#
# 5.5.2 Range of Qualification
# Filler material groups are defined in Table 2.
#
# Table 2 — Filler Material Grouping
#
# Group  | Filler Material for Welding of              | Examples of Applicable Standards
# -------|---------------------------------------------|--------------------------------------------
# FM1    | Non-alloy and fine grain steels             | ISO 2560, ISO 14341, ISO 636, ISO 14171, ISO 17632
# FM2    | High-strength steels                        | ISO 18275, ISO 16834, ISO 26304, ISO 18276
# FM3    | Creep-resisting steels (Cr < 3.75%)         | ISO 3580, ISO 21952, ISO 24598, ISO 17634
# FM4    | Creep-resisting steels (3.75% ≤ Cr ≤ 12%)   | ISO 3580, ISO 21952, ISO 24598, ISO 17634
# FM5    | Stainless and heat-resisting steels         | ISO 3581, ISO 14343, ISO 17633
# FM6    | Nickel and nickel alloys                    | ISO 14172, ISO 18274




# 5.6 Filler Material Type
#
# Welding with filler material qualifies for welding without filler material, but not vice versa.
#
# NOTE:
# For processes 142 and 311 (which are performed without filler material),
# the parent material group used in the test is the material group that the welder is qualified for.
#
# The ranges of qualification for filler material type are given in Tables 4 and 5.
#
# Table 4 — Range of Qualification for Covered Electrodes (Process 111)
#
# Welding Process: 111
#
# Type of Covering Used in the Test         | Range of Qualification
# ----------------------------------------- | -----------------------
# A, RA, RB, RC, RR, R (03, 13, 14, 19, 20, 24, 27) | Qualified for: A only
# B (15, 16, 18, 28, 45, 48)                        | Qualified for: A and B
# C (10, 11)                                        | Qualified for: C only
#
# Symbol Legend:
# ✓ (shown as '×') - welder is qualified
# —                - welder is not qualified
#
# Additional Notes:
# a) For abbreviations, see 4.3.2.
# b) The type of covering used in the qualification test of welders for root run welding without backing (ss nb)
#    is the type of covering qualified for root run welding in production with no backing (ss nb).




# 5.7 Dimensions
#
# The welder qualification test of butt welds is based on the deposited thickness and outside pipe diameters.
# The ranges of qualification are specified in Tables 6 and 7.
#
# It is not intended that deposited thickness or outside pipe diameters be measured precisely,
# but rather that the general philosophy behind the values given in Tables 6 and 7 be applied.
#
# For fillet welds, the range of qualification for material thicknesses is specified in Table 8.
#
# For test pieces of different outside pipe diameters and deposited thicknesses,
# the welder is qualified for:
# - the thinnest through to the thickest deposited and/or parent metal thickness qualified; and
# - the smallest through to the largest diameter qualified
# (refer to Tables 6 and 7).
#
# Table 6 — Range of Qualification of Deposited Thickness for Butt Welds
# (All dimensions in millimetres)
#
# Deposited Thickness of Test Piece (s) | Range of Qualification
# ------------------------------------- | -----------------------
# s < 3                                 | s to 3 or s to 2s (whichever is greater)
# 3 ≤ s < 12                            | 3 to 2s
# s ≥ 12                                | ≥ 3
#
# Notes:
# a) For single process and the same type of filler material, s is equal to parent material thickness t.
#
# b) For branch joints, the range of qualification for deposited thickness is:
#    - Set-on branch: deposited thickness of the branch
#    - Set-through / Set-in: deposited thickness of the main pipe or shell
#
# c) For oxyacetylene welding (311):
#    - s to 1.5s (if s < 3)
#
# d) For oxyacetylene welding (311):
#    - 3 to 1.5s (if 3 ≤ s < 12)
#
# e) If s ≥ 12, the test piece must be welded in at least 3 layers
#
# f) For multi-processes, s is the deposited thickness for each process



# 5.8 Welding Positions
#
# The range of qualification for each welding position is given in Tables 9 and 10.
# The welding positions and symbols refer to ISO 6947.
#
# The test pieces shall be welded in accordance with the testing positions specified in ISO 6947.
#
# Welding two pipes with the same outside pipe diameter, one in welding position PH and one in welding position PC,
# also covers the range of qualification of a pipe welded in welding position H-L045 using upward welding.
#
# Welding two pipes with the same outside pipe diameter, one in welding position PJ and one in welding position PC,
# also covers the range of qualification of a pipe welded in welding position J-L045 using downward welding.
#
# Outside pipe diameters D ≥ 150 mm can be welded in two welding positions:
# - PH or PJ for 2/3 of the circumference
# - PC for 1/3 of the circumference
# using only one test piece.
# This test covers all positions for the direction of welding used in the test.
#
# NOTE: For welding position symbols, refer to ISO 6947.



# 5.9 Weld Details
#
# Depending on the weld details, the ranges of qualification are shown in Tables 11 and 12.
#
# When welding with process 311 (oxyacetylene welding), a change from rightward welding to leftward welding,
# and vice versa, requires a new qualification test.
#
# Table 11 — Range of Qualification for Backings and Consumable Inserts
#
# Test Condition        | Qualified For:
# ----------------------|---------------------------------------------------------
# No backing (ss,nb)    | ✓ Material backing (ss,mb)
#                      | ✓ Welding from both sides (bs)
#                      | ✓ Gas backing (ss,gb)
#                      | ✗ Consumable insert (ci)
#                      | ✓ Flux backing (ss,fb)
#
# Material backing      | ✗ No backing
# (ss,mb)               | ✓ Material backing
#                      | ✓ Welding from both sides
#                      | ✗ Gas backing
#                      | ✗ Consumable insert
#                      | ✗ Flux backing
#
# Welding from both     | ✗ No backing
# sides (bs)            | ✓ Material backing
#                      | ✓ Welding from both sides
#                      | ✗ Gas backing
#                      | ✗ Consumable insert
#                      | ✗ Flux backing
#
# Gas backing (ss,gb)   | ✗ No backing
#                      | ✓ Material backing
#                      | ✓ Welding from both sides
#                      | ✓ Gas backing
#                      | ✗ Consumable insert
#                      | ✗ Flux backing
#
# Consumable insert     | ✗ No backing
# (ci)                  | ✓ Material backing
#                      | ✓ Welding from both sides
#                      | ✗ Gas backing
#                      | ✓ Consumable insert
#                      | ✗ Flux backing
#
# Flux backing          | ✗ No backing
# (ss,fb)               | ✓ Material backing
#                      | ✓ Welding from both sides
#                      | ✗ Gas backing
#                      | ✗ Consumable insert
#                      | ✓ Flux backing
#
# ✓ = welder is qualified for that condition
# ✗ = welder is NOT qualified for that condition
#
# Table 12 — Range of Qualification of Layer Technique for Fillet Welds
#
# Test Piece           | Range of Qualification
# ---------------------|--------------------------
# Single layer (sl)    | ✓ Single layer only
#                      | ✗ Not qualified for multi-layer
#
# Multi-layer (ml)     | ✓ Multi-layer
#                      | ✓ Single layer
#
# ✓ = welder is qualified
# ✗ = welder is NOT qualified
#
# Note:
# a) During welding of the test piece, the examiner shall perform a visual exam of the first layer
#    in accordance with Clause 7.
#
# b) If a welder is qualified using a multi-layer butt weld and performs the supplementary fillet weld test
#    described in 5.4(e), they are qualified for both multi- and single-layer fillet welds.


# Data Required for WPQR Designation According to ISO 15614-1

# To establish a Welding Procedure Qualification Record (WPQR) designation in accordance with ISO 15614-1,
# the following essential data points must be extracted and documented:

# 1. Welding Process(es):
#    The welding process used, such as MAG (135), TIG (141), MMA (111), etc., as per ISO 4063.

# 2. Type of Joint:
#    The joint configuration being welded:
#    - BW for Butt Weld
#    - FW for Fillet Weld

# 3. Product Form:
#    The form of the material used:
#    - P for Plate
#    - T for Tube
#    - F for Forging, etc.

# 4. Filler Material Designation (ISO):
#    The filler material group as per ISO standards (e.g., ISO 14341 for GMAW wires), labeled as FM1, FM2, etc.

# 5. Parent Material Group(s) and Subgroup(s):
#    Group numbers according to ISO 15608 (e.g., 1.1, 1.2, 8.2, etc.)

# 6. Deposited Metal Thickness (mm):
#    The actual thickness of the weld deposit.

# 7. Parent Material Thickness (mm):
#    The base material thickness used in the test.

# 8. Outside Pipe Diameter (mm):
#    Applicable only for pipe welds; enter the external diameter in mm.

# 9. Welding Position(s):
#    Welding position codes:
#    - PA (Flat), PB (Horizontal), PC (Vertical), PF (Vertical-Up), etc.

# 10. Type of Weld:
#     Indicates characteristics like:
#     - S for Single-Sided
#     - DS for Double-Sided
#     - SS with or without backing

# 11. Single Layer / Multi-Run:
#     Specifies:
#     - sl for single layer
#     - ml for multiple runs

# WPQR Designation Format Example (ISO 15614-1):
# The extracted data should be compiled into a structured designation string as follows:

# ISO15614-1 / [Welding Process] / [Product Form] / [Type of Joint] / [Filler Material Group]
# / [Parent Material Group(s)] / [Type of Weld] / [Deposited Metal Thickness]
# / [Parent Material Thickness] / [Welding Position] / [Backing & Edge Prep] / [Single or Multi-run]

# Example:
# ISO15614-1 / 135 / P / BW / FM1 / 1.4-1.4 / S / s16 / t16 / PB / ssmb / ml

# Where:
# - 135 = MAG welding
# - P = Plate
# - BW = Butt Weld
# - FM1 = Filler Material Group 1
# - 1.4-1.4 = Parent Material Groups/Subgroups
# - S = Single-sided weld
# - s16 = 16 mm deposited weld thickness
# - t16 = 16 mm parent material thickness
# - PB = Horizontal welding position
# - ssmb = Single side, metal backing
# - ml = Multi-run

# Interpretation / Explanation of Logic used in WPQR Designation Generation
# -------------------------------------------------------------------------

# ISO 15614-1 Clause 11 outlines that a WPQR designation must follow a structured format
# using several essential welding variables. This is to standardize how qualifications
# are described and stored, especially for computerized systems.

# The code extracts values from the welding qualification data (typically in a DataFrame),
# then interprets and encodes them into a standardized WPQR string.

# Below are the interpretations of each segment included in the designation:

# 1. Welding Process:
#    - Extracted from 'welding process' field.
#    - Mapped to its corresponding ISO 4063 code (e.g., MAG → 135, TIG → 141, MMA → 111).

# 2. Product Form:
#    - Extracted from 'product form'.
#    - 'Plate' → 'P', 'Pipe' → 'T'.

# 3. Type of Weld:
#    - Derived from the 'type of weld'.
#    - 'Butt' → 'BW', otherwise → 'FW'.

# 4. Filler Material Group:
#    - Identified from the filler material standard (e.g., ISO 14341 → FM1, ISO 18275 → FM2).
#    - Mapped to standard FM group identifiers (FM1, FM2, FM5, etc.).

# 5. Parent Material Group(s):
#    - Extracted and normalized (replaces dashes/spaces with hyphens).
#    - Based on ISO 15608 grouping (e.g., 1.1, 8.1, etc.).

# 6. Weld Side:
#    - Hardcoded as "S" (Single-sided weld) unless customized.

# 7. Deposited Metal Thickness:
#    - Extracts numeric value from 'deposited metal thickness' and formats it as "sX".

# 8. Parent Material Thickness:
#    - Extracts numeric value from 'parent material thickness' and formats it as "tX".

# 9. Welding Position:
#    - Takes the first position mentioned and standardizes it to ISO 6947 symbols (e.g., PA, PB, PF).

# 10. Weld Details (Backing):
#     - Interprets type of backing from descriptive input:
#       "no backing" → "ssnb", "material backing" → "ssmb", "both side" → "bs", etc.

# 11. Single/Multiple Layer:
#     - Interpreted as "sl" (single layer) or "ml" (multi-run), based on keyword matching.

# Final Format:
# ------------------
# The designation string is constructed in the format:
# ISO15614-1 / [Process] / [Product] / [Weld Type] / [Filler Group] /
# [Parent Group] / [Side] / [sX] / [tX] / [Position] / [Backing] / [Layering]

# Example Output:
# ISO15614-1/135/P/BW/FM1/1.1-1.1/S/s10/t10/PA/ssmb/ml





import re
import pandas as pd
from tabulate import tabulate

# Clause Target / Essential Variable:
# WPQR Designation

# Clause Logic:
# Per ISO 15614-1 Clause 11, WPQR designation is constructed using essential welding variables.

# --- Helper functions ---
def extract_val(keyword):
    row = essential_variables_df[essential_variables_df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return str(row.iloc[0]["Actual Value"]).strip() if not row.empty else ""

def extract_numeric(text):
    nums = re.findall(r"[\d.]+", str(text))
    return float(nums[-1]) if nums else None

def normalize_position(pos_text):
    return pos_text.strip().split()[0].upper() if pos_text else "NA"

def normalize_weld_type(text):
    return "BW" if "butt" in text.lower() else "FW"

def normalize_product_form(text):
    return "P" if "plate" in text.lower() else "T"

def normalize_layering(text):
    return "ml" if "multi" in text.lower() else "sl"

def infer_backing(description):
    desc = description.lower()
    if "no backing" in desc:
        return "ssnb"
    elif "material backing" in desc:
        return "ssmb"
    elif "both side" in desc or "welded from both sides" in desc:
        return "bs"
    elif "gas backing" in desc:
        return "ssgb"
    elif "flux backing" in desc:
        return "ssfb"
    elif "consumable insert" in desc:
        return "ci"
    else:
        return "unknown"

# --- Mappings ---
filler_group_map = {
    "ISO 14341": "FM1", "ISO 2560": "FM1",
    "ISO 18275": "FM2", "ISO 3581": "FM5"
}

process_map = {
    "MAG": "135", "MIG": "131", "TIG": "141", "MMA": "111",
    "SAW": "121", "Plasma": "15", "Oxyacetylene": "311"
}

# --- Extract input values ---
weld_process_raw      = extract_val("welding process")
product_form_raw      = extract_val("product form")
weld_type_raw         = extract_val("type of weld")
filler_designation    = extract_val("filler material designation")
parent_group          = extract_val("parent material group")
deposited_thickness   = extract_val("deposited metal thickness")
material_thickness    = extract_val("parent material thickness")
od_raw                = extract_val("outside pipe diameter")
weld_position_raw     = extract_val("welding positions")
layering_raw          = extract_val("single layer")
backing_raw           = extract_val("weld details")

# --- Normalize inputs ---
process_code       = next((v for k, v in process_map.items() if k.lower() in weld_process_raw.lower()), "135")
product_code       = normalize_product_form(product_form_raw)
weld_type_code     = normalize_weld_type(weld_type_raw)
filler_code        = next((group for keyword, group in filler_group_map.items() if keyword.lower() in filler_designation.lower()), "FM1")
parent_group_code  = parent_group.replace(" ", "").replace("\n", "-").replace("–", "-") if parent_group else "NA"

s_val = extract_numeric(deposited_thickness)
t_val = extract_numeric(material_thickness)
s_code = f"s{int(s_val)}" if s_val else "sNA"
t_code = f"t{int(t_val)}" if t_val else "tNA"

position_code = normalize_position(weld_position_raw)
backing_code = infer_backing(backing_raw)
layer_code = normalize_layering(layering_raw)

# --- Construct WPQR Designation ---
designation_parts = [
    "ISO15614-1", process_code, product_code, weld_type_code, filler_code,
    parent_group_code, "S", s_code, t_code, position_code, backing_code, layer_code
]
wpqr_designation = "/".join(designation_parts)

# --- Output Tables ---
essential_var_table = [["WPQR Designation", ""]]

input_table = [
    ["1. Welding process", weld_process_raw],
    ["2. Product form", product_form_raw],
    ["3. Type of weld", weld_type_raw],
    ["4. Filler material designation", filler_designation],
    ["5. Parent material group", parent_group],
    ["6. Deposited metal thickness", deposited_thickness],
    ["7. Parent material thickness", material_thickness],
    ["8. Welding positions", weld_position_raw],
    ["9. Single layer", layering_raw],
    ["10. Weld details", backing_raw]
]

interpretation_table = [
    ["Based on ISO 15614-1 Clause 11:"],
    ["- WPQR designation follows a structured format using essential variables:"],
    ["  Process / Product / Joint / Filler / Parent / Thickness / Position / Backing / Layering"]
]

output_table = [["WPQR Designation", wpqr_designation]]

# --- Append to global context ---
interpretation_rows.extend(interpretation_table)
essential_output_rows.extend(output_table)

# --- Display ---
print()
print(tabulate(essential_var_table, headers=["Essential Variable", ""], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", ""], tablefmt="grid"))
print()
print(tabulate(interpretation_table, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", ""], tablefmt="grid"))




+----------------------+----+
| Essential Variable   |    |
+======================+====+
| WPQR Designation     |    |
+----------------------+----+

+--------------------------------+-----------------------------+
| Input Considered               |                             |
+================================+=============================+
| 1. Welding process             | MAG (135)                   |
+--------------------------------+-----------------------------+
| 2. Product form                | Plate                       |
+--------------------------------+-----------------------------+
| 3. Type of weld                | DCEP (Reverse Polarity)     |
+--------------------------------+-----------------------------+
| 4. Filler material designation | ISO 14341-A-G 42 3 M21 3Si1 |
+--------------------------------+-----------------------------+
| 5. Parent material group       | 1.4 to 1.4                  |
+--------------------------------+-----------------------------+
| 6

In [18]:
# ------------------------------------------------------------------
# ISO 15614-1: Section 8.3 — Related to the Parent Material
# ------------------------------------------------------------------

# 8.3.1 Parent Material Grouping

# 8.3.1.1 General
# - To minimize the number of welding procedure tests,
#   steels, nickel, and nickel alloys are grouped according to ISO/TR 15608.
# - If materials are grouped under ISO/TR 20172, ISO/TR 20173, or ISO/TR 20174,
#   those groupings shall take precedence.
# - A separate WPQR (Welding Procedure Qualification Record) is required
#   for parent materials or combinations not covered under these groupings.
# - Permanent backing material is treated as part of the parent metal and
#   shall belong to the same approval group or subgroup.

# 8.3.1.2 Steels
# - Qualification ranges for steels are provided in Table 5.

# 8.3.1.3 Nickel Alloys
# - Qualification ranges for nickel alloys are provided in Table 6.

# 8.3.1.4 Dissimilar Joints (Steels to Nickel Alloys)
# - Dissimilar joints between steel and nickel alloys are also qualified
#   based on Table 6.



# ISO 15614-1 Table 5 — Welding Qualification Map
# -----------------------------------------------

# Group 1
# 1-1 → 1-1

# Group 2
# 2-1 → 1-1, 2-1
# 2-2 → 1-1, 2-1, 2-2

# Group 3
# 3-1 → 1-1, 2-1, 3-1
# 3-2 → 1-1, 2-1, 2-2, 3-1, 3-2
# 3-3 → 1-1, 2-1, 2-2, 3-1, 3-2, 3-3

# Group 4
# 4-1 → 4-1
# 4-2 → 4-1, 4-2
# 4-3 → 4-1, 4-2, 4-3
# 4-4 → 4-1, 4-2, 4-3, 4-4

# Group 5
# 5-1 → 5-1
# 5-2 → 5-1, 5-2
# 5-3 → 5-1, 5-2, 5-3
# 5-4 → 5-1, 5-2, 5-3, 5-4
# 5-5 → 5-1, 5-2, 5-5

# Group 6
# 6-1 → 6-1
# 6-2 → 6-1, 6-2
# 6-3 → 6-1, 6-2, 6-3
# 6-4 → 6-1, 6-2, 6-3, 6-4
# 6-5 → 6-1, 6-2, 6-3, 6-4, 6-5
# 6-6 → 6-1, 6-2, 6-3, 6-4, 6-5, 6-6

# Group 7 (same sub-group only)
# 7-1 → 7-1
# 7-2 → 7-2
# 7-3 → 7-3
# 7-4 → 7-4
# 7-5 → 7-5
# 7-6 → 7-6
# 7-7 → 7-7

# Group 8
# 8-1 → 8-1
# 8-2 → 8-1, 8-2
# 8-3 → 8-1, 8-2, 8-3
# 8-4 → 8-1, 8-2, 8-4
# 8-5 → 8-1, 8-2, 8-4, 8-5
# 8-6 → 8-1, 8-2, 8-4, 8-5, 8-6
# 8-7 → 8-7
# 8-8 → 8-8

# Group 9
# 9-1 → 9-1
# 9-2 → 9-1, 9-2
# 9-3 → 9-1, 9-2, 9-3
# 9-4 → 9-4
# 9-5 → 9-5
# 9-6 → 9-6
# 9-7 → 9-7
# 9-8 → 9-8
# 9-9 → 9-9

# Group 10 (same sub-group only)
# 10-1 → 10-1
# 10-2 → 10-2
# 10-3 → 10-3
# 10-4 → 10-4
# 10-5 → 10-5
# 10-6 → 10-6
# 10-7 → 10-7
# 10-8 → 10-8
# 10-9 → 10-9
# 10-10 → 10-10

# Group 11
# 11-1 → 11-1, 1-1
# 11-2 → 11-1, 11-2, 1-1
# 11-3 → 11-1, 11-2, 11-3, 1-1
# 11-4 → 11-4
# 11-5 → 11-5
# 11-6 → 11-6
# 11-7 → 11-7
# 11-8 → 11-8
# 11-9 → 11-9
# 11-10 → 11-10
# 11-11 → 11-1, 11-11, 1-1


# -----------------------------------------------
# Notes for ISO 15614-1 Table 5 (Steel Groups)
# -----------------------------------------------

# Note (a):
# Test piece materials in groups 1, 2, 3, and 11 qualify the
# equal or lower specified minimum yield strength steels
# (independent of the material thickness).

# Note (b):
# Test piece materials in groups 4, 5, 6, 8, and 9 qualify steels
# in the same sub-group and any lower sub-group within the same group.

# Note (c):
# Test piece materials in groups 7 and 10 qualify steels
# only in the same sub-group.



# ISO 15614-1 Table 6 — Qualification Map for Nickel Alloy and Nickel Alloy/Steel Groups
# --------------------------------------------------------------------------------------

# Group 41
# 41 → 41c-41

# Group 42
# 42 → 42c-41, 42c-42

# Group 43
# 43 → 43c-41, 43c-42, 43c-43, 45c-45, 47c-47

# Group 44
# 44 → 44c-41, 44c-42, 44c-43, 44c-44

# Group 45
# 45 → 45c-41, 45c-42, 45c-43, 45c-44, 45c-45, 43c-43

# Group 46
# 46 → 46c-41, 46c-42, 46c-43, 46c-44, 46c-45, 46c-46

# Group 47
# 47 → 47c-41, 47c-42, 47c-43, 47c-44, 47c-45, 47c-46, 47c-47, 43c-43, 45c-45

# Group 48
# 48 → 48c-41, 48c-42, 48c-43, 48c-44, 48c-45, 48c-46, 48c-47, 48c-48

# Group 1 (Nickel/Steel)
# 1 → 41c-1, 42c-1, 43c-1, 44c-1, 45c-1, 46c-1, 47c-1, 48c-1

# Group 2 (Nickel/Steel)
# 2 → 41c-2a, 41c-1, 42c-2a, 42c-1, 43c-2a, 43c-1, 44c-2a, 44c-1,
#      45c-2a, 45c-1, 46c-2a, 46c-1, 47c-2a, 47c-1, 48c-2a, 48c-1

# Group 3 (Nickel/Steel)
# 3 → 41c-3a, 41c-2, 41c-1, 42c-3a, 42c-2, 42c-1, 43c-3a, 43c-2, 43c-1,
#      44c-3a, 44c-2, 44c-1, 45c-3a, 45c-2, 45c-1, 46c-3a, 46c-2, 46c-1,
#      47c-3a, 47c-2, 47c-1, 48c-3a, 48c-2, 48c-1

# Group 5 (Nickel/Steel)
# 5 → 41c-5b, 41c-6, 41c-4, 41c-2, 41c-1,
#      42c-5b, 42c-6, 42c-4, 42c-2, 42c-1,
#      43c-5b, 43c-6, 43c-4, 43c-2, 43c-1,
#      44c-5b, 44c-6, 44c-4, 44c-2, 44c-1,
#      45c-5b, 45c-6, 45c-4, 45c-2, 45c-1,
#      46c-5b, 46c-6, 46c-4, 46c-2, 46c-1,
#      47c-5b, 47c-6, 47c-4, 47c-2, 47c-1,
#      48c-5b, 48c-6, 48c-4, 48c-2, 48c-1

# Group 6 (Nickel/Steel)
# 6 → 41c-6b, 41c-4, 41c-2, 41c-1,
#      42c-6b, 42c-4, 42c-2, 42c-1,
#      43c-6b, 43c-4, 43c-2, 43c-1,
#      44c-6b, 44c-4, 44c-2, 44c-1,
#      45c-6b, 45c-4, 45c-2, 45c-1,
#      46c-6b, 46c-4, 46c-2, 46c-1,
#      47c-6b, 47c-4, 47c-2, 47c-1,
#      48c-6b, 48c-4, 48c-2, 48c-1

# Group 8 (Nickel/Steel)
# 8 → 41c-8b, 42c-8b, 43c-8b, 44c-8b, 45c-8b, 46c-8b, 47c-8b, 48c-8b

# Group 11 (Nickel/Steel)
# 11 → 41c-11, 42c-11, 43c-11, 44c-11, 45c-11, 46c-11, 47c-11, 48c-11

# Notes:
# a) Covers the equal or lower specified yield strength steels of the same group.
# b) Covers steels in the same sub-group and any lower sub-group within the same group.
# c) For groups 41 to 48, a procedure test carried out with a solid solution or precipitation hardening alloy
#    in a group covers all solid solution or precipitation hardening alloys, respectively, in the same group.


# Clause Interpretation: ISO 15614-1 Clause 8.3 — Parent Material Group
#
# 1. The welding procedure qualification is specific to the parent material group(s) used during testing.
#
# 2. ISO 15614-1 Clause 8.3 mandates that materials shall be grouped in accordance with ISO/TR 15608.
#    - Steels: Refer Table 5 in ISO 15614-1
#    - Nickel/Nickel-Steel: Refer Table 6 in ISO 15614-1
#
# 3. The qualification is valid for:
#    - The group/sub-group tested
#    - And for certain lower-numbered subgroups depending on the group rules
#
# 4. Mapping logic (from Table 5):
#    - Some groups (e.g., 2-2, 3-2) qualify multiple groups beneath them
#    - Some groups (e.g., 7-1, 10-1) only qualify themselves
#
# 5. If no valid group is found or the input is non-standard (e.g. "special alloy"),
#    a manual review is flagged.
#
# 6. The parent group input is normalized to the pattern: [group]-[subgroup] (e.g., 3-1)
#    - Example: "3.2 to 1.1" → ["3-1", "1-1"]
#
# 7. Output:
#    - Lists all the qualified material groups based on the input test group
#    - Follows the scope and logic described in Table 5
#
# Purpose:
#    - Ensures that WPQR (Welding Procedure Qualification Record) is valid across allowed
#      ranges of material groups to minimize redundant requalification tests.




import re
from tabulate import tabulate

# 1. Clause Target / Essential Variable:
# Parent Material Group

# 2. Clause Logic or Standard Reference:
# ISO 15614-1 Clause 8.3:
# Qualification is valid for the tested group and certain lower/equivalent groups
# based on ISO/TR 15608 and Table 3 of ISO 15614-1.

# --- Extract actual value for a keyword ---
def extract_val(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return row.iloc[0]["Actual Value"].strip() if not row.empty else ""

# --- Normalize inputs like "2.4 to 1.4" → ["2-1", "1-1"] ---
def normalize_group_string(group_str):
    matches = re.findall(r'(\d+)\.\d+', group_str)
    return [f"{g}-1" for g in matches] if matches else None

# --- Extract groups like "3-2", "4-1" ---
def extract_groups(text):
    return re.findall(r'\b\d{1,2}-\d\b', str(text))

# --- Qualification logic per ISO 15614-1 Table 5 (simplified steel groups) ---
steel_qualification_map = {
    '1-1': ['1-1'], '2-1': ['1-1', '2-1'], '2-2': ['1-1', '2-1', '2-2'],
    '3-1': ['1-1', '2-1', '3-1'], '3-2': ['1-1', '2-1', '2-2', '3-1', '3-2'],
    '3-3': ['1-1', '2-1', '2-2', '3-1', '3-2', '3-3'],
    '4-1': ['4-1'], '4-2': ['4-1', '4-2'], '4-3': ['4-1', '4-2', '4-3'],
    '4-4': ['4-1', '4-2', '4-3', '4-4'],
    '5-1': ['5-1'], '5-2': ['5-1', '5-2'], '5-3': ['5-1', '5-2', '5-3'],
    '5-4': ['5-1', '5-2', '5-3', '5-4'], '5-5': ['5-1', '5-2', '5-5'],
    '6-1': ['6-1'], '6-2': ['6-1', '6-2'], '6-3': ['6-1', '6-2', '6-3'],
    '6-4': ['6-1', '6-2', '6-3', '6-4'], '6-5': ['6-1', '6-2', '6-3', '6-4', '6-5'],
    '6-6': ['6-1', '6-2', '6-3', '6-4', '6-5', '6-6'],
    '7-1': ['7-1'], '7-2': ['7-2'], '7-3': ['7-3'], '7-4': ['7-4'],
    '8-1': ['8-1'], '8-2': ['8-1', '8-2'], '8-3': ['8-1', '8-2', '8-3'],
    '8-4': ['8-1', '8-2', '8-4'], '8-5': ['8-1', '8-2', '8-4', '8-5'],
    '9-1': ['9-1'], '9-2': ['9-1', '9-2'], '9-3': ['9-1', '9-2', '9-3'],
    '10-1': ['10-1'],
    '11-1': ['11-1', '1-1'], '11-2': ['11-1', '11-2', '1-1'], '11-3': ['11-1', '11-2', '11-3', '1-1']
}

# --- Extract input ---
parent_group_str = extract_val(essential_variables_df, "parent material group")
normalized_groups = normalize_group_string(parent_group_str)
qualified_materials = set()

# --- Logic: Evaluate qualified groups based on input group ---
if normalized_groups:
    for group in normalized_groups:
        for g in extract_groups(group):
            if g in steel_qualification_map:
                qualified_materials.update(steel_qualification_map[g])
            else:
                qualified_materials.add(f"{g} (manual check required)")
else:
    qualified_materials.add("Manual review needed")

# --- Format tables ---
essential_var_table = [["Parent Material Group", ""]]
input_table = [["Parent Material Group", parent_group_str]]
interpretation_table = [[
    "ISO 15614-1 Clause 8.3: WPQR is valid for the tested group and certain equivalent/lower groups "
    "based on ISO/TR 15608 and Table 3 of ISO 15614-1."
]]
output_table = [["Parent Material Group", ", ".join(sorted(qualified_materials))]]

# --- Append to global output containers ---
interpretation_rows.extend(interpretation_table)
essential_output_rows.extend(output_table)

# --- Display Results ---
print()
print(tabulate(essential_var_table, headers=["Essential Variable", ""], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input considered", ""], tablefmt="grid"))
print()
print(tabulate(interpretation_table, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", ""], tablefmt="grid"))




+-----------------------+----+
| Essential Variable    |    |
+=======================+====+
| Parent Material Group |    |
+-----------------------+----+

+-----------------------+------------+
| Input considered      |            |
+=======================+============+
| Parent Material Group | 1.4 to 1.4 |
+-----------------------+------------+

+--------------------------------------------------------------------------------------------------------------------------------------------------+
| Clause Interpretation                                                                                                                            |
+==================================================================================================================================================+
| ISO 15614-1 Clause 8.3: WPQR is valid for the tested group and certain equivalent/lower groups based on ISO/TR 15608 and Table 3 of ISO 15614-1. |
+--------------------------------------------------

In [19]:
# Essential Variable   1. Shielding Gas
#                      2. Wire System

# Clause Reference: ISO 15614-1 Section 8.5.2 – Gas-shielded Metal Arc Welding (Process 13)

# -------------------------------------------------------------------------
# 8.5.2.1 – Shielding Gases

# The welding procedure qualification is valid only for the nominal composition
# of the shielding gas used in the WPQR.

# ➤ Shielding gas must match the nominal composition used in the test.
# ➤ Shielding gas may be specified using ISO 14175 designation.
#    Example: ISO 14175:2008-M21-ArC-18

# Acceptable Variations (without requalification):
# - CO₂ content can vary ±20% relative to its nominal concentration.
# - Minor gas component adjustments up to ±0.1% are permitted.

# Any change beyond these thresholds → Requalification required.
# -------------------------------------------------------------------------

# 8.5.2.2 – Process Variants

# ➤ Requalification is mandatory if the process variant is changed.

# ➤ For Level 2 WPQRs:
#    - The qualification is valid **only** for the wire system used during procedure testing.
#    - Examples:
#       - If the WPQR was tested using a **single-wire** system,
#         switching to a **multi-wire** system is NOT qualified → Requalification required.
#       - Similarly, multi-wire to single-wire changes also need requalification.

# -------------------------------------------------------------------------
# Clause Summary Table (for reference in code logic):

# +--------------------------+-------------------------------------------------------------+
# | Clause                  | Interpretation Summary                                      |
# +--------------------------+-------------------------------------------------------------+
# | 8.5.2.1 Shielding Gas   | Valid for nominal composition used in WPQR.                 |
# |                          | ±20% CO₂ variation allowed; minor gas changes <0.1% allowed |
# +--------------------------+-------------------------------------------------------------+
# | 8.5.2.2 Wire System     | Qualified only for tested wire system (e.g., single-wire).   |
# |                          | Changing system → requires requalification                  |
# +--------------------------+-------------------------------------------------------------+


# ------------------------------------------------------------------
# Interpretation for Clause ISO 15614-1 – Section 8.5.2
# Target Essential Variables:
#   1. Shielding Gas
#   2. Wire System
# ------------------------------------------------------------------
# Interpretation Applied in Code:
#
# 1. Shielding Gas Validation (Clause 8.5.2.1):
# - The shielding gas used in production must belong to the same nominal group
#   as the one recorded in the WPQR (e.g., M21, M12 from ISO 14175).
# - CO₂ content in the shielding gas is allowed to vary by ±20% from the nominal
#   percentage recorded in the WPQR.
# - Minor gas composition changes of up to ±0.1% are permitted without requalification.
# - The code extracts the M-group (e.g., M21) and CO₂ percentage from the gas string.
# - It then calculates and reports the allowable CO₂ variation range.
#
# 2. Wire System Consistency (Clause 8.5.2.2):
# - For Level 2 qualifications, the wire system used during procedure qualification
#   (e.g., single-wire or multi-wire) must remain unchanged in production.
# - The code infers the wire system from the "mode of metal transfer" input string
#   using keywords such as "multi", "twin", "tandem".
#
# Output Summary:
# - Reports shielding gas group and allowable CO₂ variation range.
# - Classifies the wire system as "Single-wire" or "Multi-wire".
# - Confirms compliance or flags requirement for requalification if group or system changes.
# ------------------------------------------------------------------



import re
from tabulate import tabulate

# ------------------------------------------------------------------
# Essential Variable:
# 1. Shielding Gas
# 2. Wire System
# ------------------------------------------------------------------
# Clause Reference:
# ISO 15614-1 Clause 8.5.2:
# - Shielding gas must match the group (e.g., M21) used in WPQR.
# - CO₂ variation of ±20% from nominal is acceptable.
# - Minor component change up to ±0.1% does NOT require requalification.
# - Wire system (single/multi-wire) must remain the same for Level 2 WPQR.
# ------------------------------------------------------------------

# --- Helper Function: Extract input value ---
def extract_val(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return row.iloc[0]["Actual Value"].strip() if not row.empty else ""

# --- Helper Function: Parse shielding gas string ---
def parse_gas(gas_str):
    # Look for ISO format like M21 or M12
    gas_group_match = re.search(r'\b(M\d{1,2})\b', gas_str.upper())
    # Attempt to extract CO₂ % as a number
    co2_match = re.search(r'CO2\s*[:\-]?\s*(\d+)', gas_str.upper())

    gas_group = gas_group_match.group(1) if gas_group_match else "Unknown"
    co2_percent = int(co2_match.group(1)) if co2_match else None
    return gas_group, co2_percent

# --- Extract input data ---
shielding_gas_str = extract_val(essential_variables_df, "shielding gas")
mode_transfer_str = extract_val(essential_variables_df, "mode of metal transfer")

# --- Parse and calculate ---
gas_group, co2_val = parse_gas(shielding_gas_str)

if co2_val is not None:
    co2_min = round(co2_val * 0.8, 1)
    co2_max = round(co2_val * 1.2, 1)
    co2_range = f"{co2_min:.1f}% – {co2_max:.1f}% CO₂"
else:
    co2_range = "Not detected"

# --- Determine wire system from keywords ---
wire_type = "Single-wire"
if any(term in mode_transfer_str.lower() for term in ["multi", "twin", "tandem"]):
    wire_type = "Multi-wire"

# --- Prepare output tables ---
essential_var_table = [
    ["1. Shielding Gas", ""],
    ["2. Wire System", ""]
]

input_table = [
    ["1. Shielding Gas", shielding_gas_str or "Not Provided"],
    ["2. CO₂ % Extracted", f"{co2_val if co2_val is not None else 'Not Found'}"],
    ["3. Mode of Metal Transfer", mode_transfer_str or "Not Provided"]
]

interpretation_table = [[
    "ISO 15614-1 Clause 8.5.2: Shielding gas group must match WPQR. "
    "±20% CO₂ variation allowed. Wire system (e.g. single-wire) must remain unchanged for Level 2 WPQR."
]]

output_table = [
    ["Shielding Gas", f"{gas_group} ({co2_range})"],
    ["Wire System", wire_type]
]

# --- Append to global output ---
interpretation_rows.extend(interpretation_table)
essential_output_rows.extend(output_table)

# --- Display Output ---
print()
print(tabulate(essential_var_table, headers=["Essential Variable", ""], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", ""], tablefmt="grid"))
print()
print(tabulate(interpretation_table, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", ""], tablefmt="grid"))



+----------------------+----+
| Essential Variable   |    |
+======================+====+
| 1. Shielding Gas     |    |
+----------------------+----+
| 2. Wire System       |    |
+----------------------+----+

+---------------------------+----------------------------+
| Input Considered          |                            |
+===========================+============================+
| 1. Shielding Gas          | M21 (Argon 80 % + Co2 20%) |
+---------------------------+----------------------------+
| 2. CO₂ % Extracted        | 20                         |
+---------------------------+----------------------------+
| 3. Mode of Metal Transfer | Pulse                      |
+---------------------------+----------------------------+

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Clause Interpretation                                                                   

In [20]:
# ------------------------------------------------------------------
# Essential Variable    1. Backing Gas
# Clause Reference:     ISO 15614-1 Section 8.5.6 – Backing Gas (Level 2 Requirements)
# ------------------------------------------------------------------

# General Qualification Rule:
# - A butt weld procedure test *without* backing gas qualifies the procedure for use with
#   backing gases in group I, N1, N2, and N3 (per ISO 14175).
# - The reverse is NOT valid: using backing gas in the test does NOT qualify procedures without it.

# Backing Gas Group Coverage:
# - A main gas group (e.g., I, N, R) qualifies all its sub-groups as defined in ISO 14175.

# Material Group Rules (per ISO/TR 15608):

# For material groups 1 to 6:
# - No re-qualification is needed when switching between group I, N1, N2, and N3 backing gases.

# For material groups 8 and 41 to 48:
# - No re-qualification is required for changes between backing gas groups I, N, and R.

# For material groups 7 and 10:
# - Any change in backing gas classification REQUIRES re-qualification.

# Additional Rule for Material Backing Support:
# - If production welds are made using a material backing support thicker than 5 mm,
#   the deletion of the backing gas is allowed without requiring re-qualification.
# ------------------------------------------------------------------

# -----------------------------
# | Essential Variable Table |
# -----------------------------
# | Essential Variable        | Clause Reference     |
# |---------------------------|-----------------------|
# | 1. Backing Gas            | ISO 15614-1 Clause 8.5.6 |
# -----------------------------

# ------------------------------------------------------------------------
# INTERPRETATION AS PER ISO 15614-1 CLAUSE 8.5.6 (BACKING GAS)
# ------------------------------------------------------------------------

# 1. If no backing gas is used in the WPQR:
#    → The procedure is qualified for backing gases of Group I, N1, N2, and N3
#      as defined in ISO 14175.
#    → This qualification holds true for parent material groups 1 to 6.

# 2. If backing gas IS used in the WPQR:
#    → The qualification is valid only within the same compatible gas group
#      family based on the material group:
#       - For parent material groups 1 to 6:
#         → Qualified for I, N1, N2, N3
#       - For groups 8 and 41 to 48:
#         → Qualified for I, N, R
#       - For groups 7 and 10:
#         → Any change in gas classification requires REQUALIFICATION

# 3. The code extracts:
#    - Backing gas designation (to identify its group: I/N/R)
#    - Parent material group (e.g., 2.1, 8.2)
#    - Checks if backing gas is used or not

# 4. Then:
#    - If no backing gas → Automatically qualifies for I, N1, N2, N3
#    - If backing gas is used →
#        - Checks parent group and determines what gas families are permitted
#        - Flags requalification if material group is 7 or 10




import re
from tabulate import tabulate

# --- Clause Target / Essential Variable ---
# 1. Backing Gas

# --- Clause Logic / Standard Reference ---
# ISO 15614-1 Clause 8.5.6:
# - No backing gas → qualifies for I, N1, N2, N3
# - With backing gas → must match allowed gas group depending on parent material group
# - Groups 7 and 10 always require requalification if gas group changes

# --- Extraction Helpers ---
def extract_val(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return row.iloc[0]["Actual Value"].strip() if not row.empty else ""

def parse_parent_groups(group_str):
    matches = re.findall(r'(\d+)\.\d+', group_str)
    return sorted(set(int(m) for m in matches)) if matches else []

def extract_backing_group(gas_str):
    match = re.search(r'\b([INR]\d*|R)\b', gas_str.upper())
    return match.group(1) if match else None

def determine_backing_qualification(used, group_code, mat_groups):
    if not used:
        return "Qualified for backing gases: I, N1, N2, N3"

    group_base = group_code[0] if group_code else "?"
    for mg in mat_groups:
        if mg in [7, 10]:
            return f"❌ Requalification required (Group {mg} does not permit change)"

    allowed = set()
    for mg in mat_groups:
        if mg in range(1, 7):
            allowed.update(["I", "N1", "N2", "N3"])
        elif mg == 8 or 41 <= mg <= 48:
            allowed.update(["I", "N", "R"])

    return f"Qualified for: {', '.join(sorted(allowed))}"

# --- Data Extraction ---
backing_gas = extract_val(essential_variables_df, "backing gas")
parent_group_str = extract_val(essential_variables_df, "parent material group")
material_groups = parse_parent_groups(parent_group_str)
backing_group = extract_backing_group(backing_gas)
used_backing = backing_gas.strip().lower() not in ["not applicable", "none", "no", ""]

qualified_range = determine_backing_qualification(used_backing, backing_group, material_groups)

# --- Tables ---
essential_var_table = [["1. Backing Gas", ""]]

input_table = [
    ["1. Backing Gas", backing_gas or "Not Provided"],
    ["2. Parent Material Group", parent_group_str or "Not Provided"]
]

interpretation_table = [[
    "ISO 15614-1 Clause 8.5.6:\n"
    "- No backing gas → Qualified for I, N1, N2, N3.\n"
    "- With backing → Must match compatible group depending on material group.\n"
    "- Material groups 7 and 10 require requalification for any group change."
]]

output_table = [["Backing Gas Qualification", qualified_range]]

# --- Append to global lists ---
interpretation_rows.extend(interpretation_table)
essential_output_rows.extend(output_table)

# --- Print Tables ---
print()
print(tabulate(essential_var_table, headers=["Essential Variable", ""], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input Considered", ""], tablefmt="grid"))
print()
print(tabulate(interpretation_table, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", ""], tablefmt="grid"))



+----------------------+----+
| Essential Variable   |    |
+======================+====+
| 1. Backing Gas       |    |
+----------------------+----+

+--------------------------+----------------+
| Input Considered         |                |
+==========================+================+
| 1. Backing Gas           | Not Applicable |
+--------------------------+----------------+
| 2. Parent Material Group | 1.4 to 1.4     |
+--------------------------+----------------+

+---------------------------------------------------------------------------+
| Clause Interpretation                                                     |
+===========================================================================+
| ISO 15614-1 Clause 8.5.6:                                                 |
| - No backing gas → Qualified for I, N1, N2, N3.                           |
| - With backing → Must match compatible group depending on material group. |
| - Material groups 7 and 10 require requalification for 

In [21]:
# Essential Variable   1. Post Weld Heat Treatment (PWHT)
# Section 8.4.11 – Post-Weld Heat Treatment (PWHT) – Level 2 Requirements

# General Rule:
# - Addition or deletion of PWHT is NOT permitted.
# - Separate procedure qualification is required for each PWHT condition.

# a) PWHT Conditions for ISO/TR 15608 Groups: 1–7, 9, 10, 11

# The following PWHT scenarios require separate qualification:
# 1) PWHT below lower transformation temperature (e.g., stress relief).
# 2) PWHT above upper transformation temperature (e.g., normalizing).
# 3) PWHT above upper transformation temperature followed by sub-transformational treatment
#    (e.g., normalizing or quenching followed by tempering).
# 4) PWHT between upper and lower transformation temperatures.

# For Level 2:
# - The qualified temperature range = holding temperature ±20°C (unless otherwise specified).
# - Heating rate, cooling rate, and holding time must be product-specific and documented.

# b) For All Other Material Groups:
# - PWHT must remain within the qualified temperature range per procedure test.

# ISO/TR 14745:2015 – Additional Requirements

# 7.2 – Heating and Cooling Temperature Gradients:

# For temperatures up to 500°C:
# - Temperature variation within the component must NOT exceed 150°C over 4500 mm length.

# For temperatures above 500°C:
# - Temperature variation must NOT exceed 100°C.

# 7.3 – Furnace PWHT Conditions:

# Maximum furnace temperature at insertion/removal:
# - ≤ 400°C for simple shapes, thickness t < 60 mm.
# - ≤ 300°C for complex shapes, or thickness t ≥ 60 mm.

# Heating/Cooling Rate Limits:
# - For thickness t ≤ 25 mm: max rate = 220°C/h
# - For 25 mm < t ≤ 100 mm: max rate = (5500 / t) °C/h
# - For t > 100 mm: max rate = 55°C/h

# Notes:
# - Temperature control refers to actual product/component surface.
# - Prevent direct flame contact and avoid surface oxidation by controlling furnace atmosphere.
# - NOTE 1: Use 1°C = 1 K for gradient calculations.
# - NOTE 2: "Complex components" involve multi-materials, large thickness variation,
#           or intricate welded designs.








# 🔧 INTERPRETATION OF SECTION 8.4.11 – HEAT TREATMENT

# 🔒 NO FLEXIBILITY WITHOUT REQUALIFICATION
# - Post-weld heat treatment (PWHT) cannot be added or removed from a welding procedure without requalifying it.
# - Any change to the heat treatment requires a new procedure qualification.

# 🧪 PWHT REQUIREMENTS BASED ON MATERIAL GROUPS

# For ISO/TR 15608 material groups 1–7, 9–11:
# Each of the following PWHT types is considered a distinct condition. A separate qualification is required for each:

# 1. PWHT below lower transformation temperature (e.g., stress relieving)
# 2. PWHT above upper transformation temperature (e.g., normalizing)
# 3. PWHT above upper transformation temp followed by heat below lower temp (e.g., normalizing or quenching + tempering)
# 4. PWHT in the range between lower and upper transformation temperatures

# ➡ For Level 2:
# - Validated temperature range = Holding temp used in the procedure test ± 20 °C (unless stated otherwise)
# - Heating/cooling rates and holding time must relate to the actual product.

# 🔧 FOR OTHER MATERIALS
# - Follow the specified PWHT temperature range for those materials.

# 🔥 HEATING AND COOLING CONTROLS (FROM ISO/TR 14745:2015(E))

# For temperatures up to 500 °C:
# - Max variation across product/component: 150 °C over 4500 mm
# - Temperature gradient must be gradual (not steep)

# For temperatures above 500 °C:
# - Max variation: 100 °C

# 🔥 FURNACE HEATING SPECIFICS

# When using a furnace for PWHT:
# - Max furnace temp when loading/unloading parts:
#   — 400 °C for simple parts, t < 60 mm
#   — 300 °C for complex parts or t ≥ 60 mm

# Heating and cooling rate limits:
# — For thickness t ≤ 25 mm: 220 °C/hour
# — For 25 mm < t ≤ 100 mm: (5500 / t) °C/hour
# — For t > 100 mm: 55 °C/hour

# Use actual metal temperatures for the above limits (not just furnace air temp).

# ⚙️ ATMOSPHERE & FLAME CONTROL
# - Control furnace atmosphere to avoid excessive oxidation.
# - No direct flame contact on the product/component.

# ℹ️ NOTES
# - 1 °C = 1 K when defining temperature gradients
# - "Complex component" = Product with:
#   • Different materials with significant thickness differences
#   • Multiple welds or intricate design


# Interpretation of PWHT Logic as per ISO 15614-1 Clause 8.4.11 and ISO/TR 14745:2015

# ✅ Clause 8.4.11 (PWHT Applicability & Range):
# - PWHT (Post Weld Heat Treatment) cannot be added or removed without requalification.
# - PWHT is essential when it is recorded in the WPQR and must match the condition tested.
# - Valid temperature range for PWHT is defined as: Soaking Temp ± 20°C,
#   provided it does not exceed Tempering Temp – 20°C.
# - Holding temperature, heating and cooling rate, and soaking time must relate to the actual component.

# ✅ Clause 7.2 (ISO/TR 14745:2015 – Thermal Gradient):
# - For temperatures ≤ 500°C: max thermal variation must be ≤ 150°C across 4500 mm.
# - For temperatures > 500°C: max variation ≤ 100°C.
# - These constraints ensure uniform heating across the component to avoid structural stress.

# ✅ Clause 7.3 (Furnace PWHT):
# - Loading temperature must not exceed 300°C (for t ≥ 60 mm or complex parts).
# - Heating/Cooling rate:
#   - t ≤ 25 mm     → max 220°C/hr
#   - 25 < t ≤ 100  → max (5500 / t) °C/hr
#   - t > 100 mm    → max 55°C/hr
# - These limits avoid cracking or distortion during thermal cycling.
# - Furnace atmosphere must prevent oxidation and avoid direct flame exposure.

# ✅ Soaking Time Rule:
# - Required holding time must be at least 1 hour per 25 mm of thickness (rounded up).
# - This ensures full heat penetration and metallurgical transformation.

# ✅ Output Decision Logic:
# - If soaking and tempering temperatures are defined and within expected limits,
#   PWHT is required and shown as "Yes (Refer 2nd Page)".
# - If missing or out of spec, marked as "Not Applicable".






import pandas as pd
import math
from tabulate import tabulate

# --- Clause Target: Essential Variable ---
# 1. Post Weld Heat Treatment (PWHT)
#
# --- Clause Interpretation ---
# Based on ISO 15614-1 Clause 8.4.11 and ISO/TR 14745:2015
# - Soaking temperature must be within ±20°C but below tempering temperature - 20°C
# - Loading/unloading temp ≤ 300°C
# - Heating/cooling rate limits based on thickness
# - Soaking time required is thickness-based (1 hr per 25 mm)

# --- Helper Functions ---
def extract_val(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return float(row.iloc[0]["Actual Value"]) if not row.empty else None

def extract_str(df, keyword):
    row = df[df["Essential Variable"].str.contains(keyword, case=False, na=False)]
    return row.iloc[0]["Actual Value"].strip() if not row.empty else ""

def extract_thickness(df):
    row = df[df["Essential Variable"].str.contains("Parent material thickness", case=False, na=False)]
    if row.empty:
        return None
    value = row.iloc[0]["Actual Value"]
    lines = value.strip().splitlines()
    thicknesses = []
    for line in lines:
        parts = line.strip().split('-')
        if len(parts) == 2:
            try:
                t = float(parts[1].replace('mm', '').strip())
                thicknesses.append(t)
            except ValueError:
                continue
    return max(thicknesses) * 2 if thicknesses else None

# --- Extract Inputs ---
tempering_temp = extract_val(essential_variables_df, "Tempering Temperature")
soaking_temp   = extract_val(essential_variables_df, "Soaking Temp")
loading_temp   = extract_val(essential_variables_df, "Loading Temp")
unloading_temp = extract_val(essential_variables_df, "Unloading Temp")
rate           = extract_val(essential_variables_df, "Rate of Heating")
soaking_time   = extract_val(essential_variables_df, "Soaking Time")
method         = extract_str(essential_variables_df, "Method")
thickness_mm   = extract_thickness(essential_variables_df)

# --- Calculate Limits ---
soaking_range_low  = soaking_temp - 20 if soaking_temp else None
soaking_range_high = min(soaking_temp + 20, tempering_temp - 20) if soaking_temp and tempering_temp else None

if thickness_mm is not None:
    if thickness_mm <= 25:
        max_rate = 220
    elif thickness_mm <= 100:
        max_rate = round(5500 / thickness_mm, 1)
    else:
        max_rate = 55
    soaking_time_required = f"Minimum {math.ceil(thickness_mm / 25)} hour(s) per 25 mm thickness"
else:
    max_rate = "N/A"
    soaking_time_required = "N/A"

# --- Determine Applicability ---
pwht_required = "Yes (Refer 2nd Page)" if soaking_range_low and soaking_range_high else "Not Applicable"

# --- Table Output ---
essential_var_table = [["1. Post Weld Heat Treatment (PWHT)", ""]]

input_table = [
    ["1. Soaking Temp. (°C)", f"{soaking_temp:.0f}" if soaking_temp else "N/A"],
    ["2. Tempering Temp. (°C)", f"{tempering_temp:.0f}" if tempering_temp else "N/A"],
    ["3. Loading Temp. (°C)", f"{loading_temp:.0f}" if loading_temp else "N/A"],
    ["4. Unloading Temp. (°C)", f"{unloading_temp:.0f}" if unloading_temp else "N/A"],
    ["5. Heating & Cooling Rate (°C/h)", f"{rate:.0f}" if rate else "N/A"],
    ["6. Soaking Time (Hrs)", f"{soaking_time:.0f}" if soaking_time else "N/A"],
    ["7. Method", method or "N/A"],
    ["8. Parent Material Thickness (mm)", f"{thickness_mm:.0f}" if thickness_mm else "N/A"]
]

interpretation = [[
    "Clause 8.4.11 + ISO/TR 14745:\n"
    "- Soaking Temp. must be within ±20°C but below Tempering Temp - 20°C.\n"
    "- Loading/Unloading Temp must be ≤ 300°C.\n"
    f"- Max heating/cooling rate: {max_rate} °C/h.\n"
    f"- Soaking Time: {soaking_time_required}."
]]

output_table = [
    ["Post Weld Heat Treatment (PWHT)", pwht_required],
    ["Soaking Temperature Range", f"{int(soaking_range_low)} – {int(soaking_range_high)} °C" if soaking_range_low and soaking_range_high else "N/A"],
    ["Loading Temp", "Max 300°C"],
    ["Unloading Temp", "Max 300°C"],
    ["Heating Rate", f"Max {max_rate} °C/h"],
    ["Cooling Rate", f"Max {max_rate} °C/h"],
    ["Soaking Time (Hrs)", soaking_time_required]
]

# --- Append to global lists ---
interpretation_rows.extend(interpretation)
essential_output_rows.extend(output_table)

# --- Print Output ---
print()
print(tabulate(essential_var_table, headers=["Essential Variable", ""], tablefmt="grid"))
print()
print(tabulate(input_table, headers=["Input considered", ""], tablefmt="grid"))
print()
print(tabulate(interpretation, headers=["Clause Interpretation"], tablefmt="grid"))
print()
print(tabulate(output_table, headers=["Essential Variable Output", ""], tablefmt="grid"))






+------------------------------------+----+
| Essential Variable                 |    |
+====================================+====+
| 1. Post Weld Heat Treatment (PWHT) |    |
+------------------------------------+----+

+-----------------------------------+----------------+
| Input considered                  |                |
+===================================+================+
| 1. Soaking Temp. (°C)             | 620            |
+-----------------------------------+----------------+
| 2. Tempering Temp. (°C)           | 650            |
+-----------------------------------+----------------+
| 3. Loading Temp. (°C)             | 50             |
+-----------------------------------+----------------+
| 4. Unloading Temp. (°C)           | 100            |
+-----------------------------------+----------------+
| 5. Heating & Cooling Rate (°C/h)  | 150            |
+-----------------------------------+----------------+
| 6. Soaking Time (Hrs)             | 2              |
+-------

In [22]:
from tabulate import tabulate

# --- Display Combined Interpretations ---
if interpretation_rows:
    print("\n=== Combined Interpretations ===\n")
    print(tabulate(interpretation_rows, headers=["Clause Interpretation"], tablefmt="grid"))
else:
    print("\n=== No Interpretations Available ===\n")

# --- Display Combined Essential Variable Outputs ---
if essential_output_rows:
    print("\n=== Combined Essential Variable Outputs ===\n")
    print(tabulate(essential_output_rows, headers=["Essential Variable Output", "Value"], tablefmt="grid"))
else:
    print("\n=== No Essential Variable Outputs Available ===\n")





=== Combined Interpretations ===

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Clause Interpretation                                                                                                                                             |
+===================================================================================================================================================================+
| Based on ISO 15614-1 Clause 8.3.3, plate qualifies pipe only if OD > 500 mm                                                                                       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| or OD > 150 mm with position PC/PF/PA.                                                                                               

In [23]:
# --------------------------------------------------------
# ✅ STEP 0: Install Required Excel Library (Only Once Per Session)
# --------------------------------------------------------

# 📌 PURPOSE:
# The code you're going to run will work with Excel files (.xlsx format).
# For that, Python needs the "openpyxl" library, which allows:
# - Reading cell values
# - Writing data into cells
# - Formatting (fonts, alignments, etc.)
# This step installs it in your Google Colab environment.

# ⚙️ The '!' symbol runs a shell command in Colab or Jupyter
!pip install openpyxl



# --------------------------------------------------------
# ✅ STEP 1: Import Required Python Libraries for Excel and File Handling
# --------------------------------------------------------

# 📌 PURPOSE:
# This step imports all the tools we need to:
# - Work with Excel templates (read/write cells, apply formatting)
# - Upload and download files in Google Colab
# - Handle structured data tables (like CSV or Excel rows)
# - Access file directories and apply short time delays

# --------------------------------------------------------

# 📁 openpyxl: Helps us open, edit, and save Excel (.xlsx) files
import openpyxl

# 🎨 Styles from openpyxl: To apply font type, size, and text alignment inside Excel cells
from openpyxl.styles import Font, Alignment

# 🔡 Utility function: Converts Excel column letters (like 'D') into index numbers (e.g., 4)
# This helps us locate a cell in Excel using code
from openpyxl.utils import column_index_from_string

# 📂 Google Colab file tools: Used to upload and download Excel files from your local machine
from google.colab import files

# 📊 pandas: Used to work with data tables (like Excel or CSV content)
# You’ll use this for cleaning, transforming, and summarizing welding data
import pandas as pd

# 🧠 os: Helps us access files and folders in your current working directory
import os

# ⏳ time: Adds delay to wait for file upload to complete before reading it
import time



# --------------------------------------------------------
# ✅ STEP 2: Use the Uploaded WPQR Template File
# --------------------------------------------------------

# 📌 PURPOSE:
# This step uses the WPQR template file already uploaded and identified in STEP 0.
# No need to re-upload or scan the directory again.

template_path = template_file  # From the upload block at top
print(f"✅ Using Template File: {template_path}")



# --------------------------------------------------------
# ✅ STEP 3: Prepare the Data to Fill in Excel
# --------------------------------------------------------

# 📌 PURPOSE:
# This step gathers all relevant welding data (setup, design, summary, PWHT)
# into one combined dictionary (`extracted_data`) which will be used
# to fill static fields in the Excel template.

# --------------------------------------------------------

# 🎯 1. Prepare Weld Setup Details from df_weld_details
# - We create a dictionary called `weld_dict`
# - Keys = Descriptions from the column "Description"
# - Values = Corresponding welding details from the column "Details"
# - Also replace different types of colons to ensure consistency in keys
weld_dict = {
    k.strip().replace(";", ":").replace("：", ":"): v
    for k, v in zip(df_weld_details["Description"], df_weld_details["Details"])
}

# 🎯 2. Prepare Design Variables from df_applicable
# - Create a dictionary where:
#   Keys = "Description" column
#   Values = "Variable Details" column
design_dict = dict(zip(df_applicable["Description"], df_applicable["Variable Details"]))

# 🎯 3. Add Summary Values (interpass temp, heat input, gas flow rates)
# - Only add each value if it's present (non-empty)
summary_dict = {}

if interpass_temp:
    summary_dict["Interpass temperature Max:"] = str(interpass_temp) + "°C"

if avg_heat_input:
    summary_dict["Heat input (kJ/mm):"] = str(avg_heat_input)

if shielding_max:
    summary_dict["Gas Flow Rate (Shielding):"] = shielding_max

if backing_max:
    summary_dict["Gas Flow Rate (Backing):"] = backing_max

# 🎯 4. Extract PWHT (Post Weld Heat Treatment) Details
# - Check for each known PWHT field in the uploaded df_pwht_details table
# - If found, store its value in the summary_dict
pwht_fields = [
    "PWHT Applicable", "Soaking Temp. (⁰C)", "Loading Temp. (⁰C)",
    "Rate of Heating & Cooling (⁰C)", "Soaking Time (Hrs)",
    "Unloading Temp. (⁰C)", "Method"
]

for field in pwht_fields:
    if field in df_pwht_details["Description"].values:
        # Find the matching row and get its value
        val = df_pwht_details[df_pwht_details["Description"] == field]["Details"].values[0]
        summary_dict[field] = val

# 🧱 5. Combine All Dictionaries into One for Excel Cell Population
# - `extracted_data` contains:
#   - Weld setup fields
#   - Design variables
#   - Summary values like temperatures or flow rates
#   - PWHT information
extracted_data = {**weld_dict, **design_dict, **summary_dict}

# 📘 6. Prepare Essential Variables from Final Table
# - This is pulled from the table generated in Step 12
# - It's used to fill essential variable cells in a different part of the Excel template
# - Each row = [Essential Variable Label, Actual Value]
essential_variable_dict = {
    row[0].strip(): row[1].strip()
    for row in essential_output_rows
}



# --------------------------------------------------------
# ✅ STEP 4: Define Excel Cell Coordinates for Writing Values
# --------------------------------------------------------

# 📌 PURPOSE:
# In this step, we tell the program **where** in the Excel sheet each value should go.
# Each entry is a tuple with:
#   (Label to match, Sheet index, Column letter, Row number)
# Later, these labels will be matched with actual values and written into the specified cells.

# --------------------------------------------------------

# 📄 Static WPQR Fields on Sheet 1 (page index 0)
# These refer to general WPQR information like joint type, heat input, shielding gas, etc.
coordinates_page1 = [
    ("Approved By\nExaminer", 0, 'H', 4),
    ("WPQR No:", 0, 'C', 4),
    ("Examiner Reference Number", 0, 'H', 5),
    ("Manufacturer:", 0, 'C', 6),
    ("Manufacturer Address", 0, 'C', 7),
    ("Date of Welding:", 0, 'H', 7),
    ("Identification Number:", 0, 'H', 6),
    ("Testing Standard", 0, 'C', 8),
    ("Level", 0, 'H', 8),
    # Welding setup details start from row 12 onwards
    ("Product Form:", 0, 'D', 12),
    ("Welding Process(es):", 0, 'D', 13),
    ("Deposited metal Thickness (mm)", 0, 'D', 14),
    ("Type of Joint:", 0, 'D', 15),
    ("Type of Weld:", 0, 'D', 15),  # Merged into same cell
    ("Weld Details", 0, 'D', 16),
    ("Parent material group(s) and sub group(s):", 0, 'D', 17),
    ("Parent material thickness (mm):", 0, 'D', 18),
    ("Throat thickness (mm):", 0, 'D', 19),
    ("Single Layer / Multi run:", 0, 'D', 20),
    ("Outside pipe diameter (mm):", 0, 'D', 21),
    ("Filler material designation (ISO) :", 0, 'D', 22),
    ("Filler material make:", 0, 'D', 23),
    ("Filler material size(mm):", 0, 'D', 24),
    ("Designation of shielding gas/flux:", 0, 'D', 25),
    ("Designation of backing gas:", 0, 'D', 26),
    ("Type of welding current and polarity:", 0, 'D', 27),
    ("Mode of metal transfer:", 0, 'D', 28),
    ("Heat input (kJ/mm):", 0, 'D', 29),
    ("Welding positions:", 0, 'D', 30),
    ("Preheat temperature Min:", 0, 'D', 31),
    ("Interpass temperature Max:", 0, 'D', 32),
    ("Post heating:", 0, 'D', 33),
    ("PWHT Applicable", 0, 'D', 34)
]

# 📄 Static WPQR Fields on Sheet 2 (page index 1)
coordinates_page2 = [
    ("Approved By\nExaminer", 1, 'I', 5),
    ("PWPS No:", 1, 'D', 5),
    ("Preparation / Cleaning Method:", 1, 'I', 6),
    ("WPQR No:", 1, 'D', 6),
    ("Base Material Grade:", 1, 'I', 7),
    ("Manufacturer:", 1, 'D', 8),
    ("Name of Welder:", 1, 'D', 9),
    ("Type of Joint:", 1, 'D', 10),
    ("Welding positions:", 1, 'I', 10),
    ("Outside pipe diameter (mm):", 1, 'I', 9),
    ("Parent material thickness (mm):", 1, 'I', 8),
    ("Filler Material Classification (ASME)", 1, 'G', 24),
    ("Filler material designation (ISO) :", 1, 'G', 25),
    ("Baking & drying Filler Metal :", 1, 'E', 26),
    ("Nozzle Diameter", 1, 'J', 27),
    ("Designation of shielding gas/flux:", 1, 'E', 27),
    ("Designation of backing gas:", 1, 'E', 28),
    ("Mode of Metal Transfer:", 1, 'J', 28),
    ("CTTW (Contact Tube to Work Distance):", 1, 'J', 29),
    ("Torch Angle:", 1, 'J', 30),
    ("Post heating:", 1, 'J', 31),
    ("Details of back gouging / backing:", 1, 'E', 31),
    ("Preheat temperature Min:", 1, 'E', 32),
    ("Oscillation : Amplitude, frequency,", 1, 'J', 32),
    ("Interpass temperature Max:", 1, 'E', 33),
    ("Plasma welding details", 1, 'J', 33),
    ("Tungsten electrode type:", 1, 'E', 34),
    ("Tungsten electrode size:", 1, 'J', 34),
    ("Soaking Temp. (⁰C)", 1, 'E', 38),
    ("Loading Temp. (⁰C)", 1, 'E', 39),
    ("Rate of Heating & Cooling (⁰C)", 1, 'E', 40),
    ("Soaking Time (Hrs)", 1, 'I', 38),
    ("Unloading Temp. (⁰C)", 1, 'I', 39),
    ("Method", 1, 'I', 40),
    ("Gas Flow Rate (Shielding):", 1, 'E', 29),
    ("Gas Flow Rate (Backing):", 1, 'E', 30)
]

# 🔗 Combine both page coordinates into one single list
coordinates = coordinates_page1 + coordinates_page2

# 📘 Essential Variables Table — separate special block used in Step 12
# These are key welding qualification parameters filled into a summary box
essential_variable_coordinates = [
    ("WPQR Designation", 0, 'C', 5),
    ("Product Form Qualified", 0, 'H', 12),
    ("Welding Process Qualification", 0, 'H', 13),
    ("Deposited Weld Thickness (s)", 0, 'H', 14),
    ("Type of joint and weld", 0, 'H', 15),
    ("Weld Details", 0, 'H', 16),
    ("Parent Material Group", 0, 'H', 17),
    ("Parent Material Thickness (t)", 0, 'H', 18),
    ("Throat Thickness (a)", 0, 'H', 19),
    ("Single Layer / Multi run", 0, 'H', 20),
    ("Outside Pipe Diameter (mm)", 0, 'H', 21),
    ("Filler Material Designation", 0, 'H', 22),
    ("Filler Material Make", 0, 'H', 23),
    ("Filler Material Size (Diameter)", 0, 'H', 24),
    ("Shielding Gas", 0, 'H', 25),
    ("Backing Gas", 0, 'H', 26),
    ("Type of Current and Polarity", 0, 'H', 27),
    ("Mode of Metal Transfer", 0, 'H', 28),
    ("Heat Input / Arc Energy", 0, 'H', 29),
    ("Welding Position", 0, 'H', 30),
    ("Preheat temperature Min", 0, 'H', 31),
    ("Interpass Temperature", 0, 'H', 32),
    ("Post-heating", 0, 'H', 33),
    ("Post Weld Heat Treatment (PWHT)", 0, 'H', 34),
    # Also writing WPQR Designation again on Page 2
    ("WPQR Designation", 1, 'D', 7)
]


# --------------------------------------------------------
# ✅ STEP 5: Fill Excel Cells with Extracted Data
# --------------------------------------------------------

# 📌 PURPOSE:
# This step takes all the values you extracted (from design sheet, weld records, PWHT data, etc.)
# and writes them into the appropriate cells in the WPQR Excel template you uploaded earlier.

# 🔄 It uses:
# - The `coordinates` list to know where each value goes in the Excel sheet (Page 1 and 2)
# - The `essential_variable_coordinates` list to write specific WPQR summary fields (like heat input, weld position, etc.)

# --------------------------------------------------------

# 🧾 Load the Excel file into memory so we can edit it
wb = openpyxl.load_workbook(template_path)

# --------------------------------------------------------
# 🧩 1. Fill Standard WPQR Fields (like Manufacturer, Joint Type, Filler Metal, etc.)
# --------------------------------------------------------

# 🔁 Loop over each field defined in the `coordinates` list (from Step 4)
for label, sheet_index, col_letter, row in coordinates:

    # ✅ Get the corresponding value using the label from the extracted_data dictionary
    value = extracted_data.get(label, "")

    # ✏️ If a value is found (non-empty), then write it into the Excel sheet
    if value != "":
        # Get the correct worksheet based on the index (0 = Page 1, 1 = Page 2)
        sheet = wb.worksheets[sheet_index]

        # Convert column letter like 'D' to a number (e.g., 'D' = 4)
        col = column_index_from_string(col_letter)

        # Access the specific cell using row and column
        cell = sheet.cell(row=row, column=col)

        # Write the value into that cell
        cell.value = value

        # Optional Styling:
        cell.font = Font(name='Arial', size=8)  # Set font style and size
        cell.alignment = Alignment(
            vertical='center', horizontal='left', wrap_text=True
        )  # Align and wrap text in the cell

# --------------------------------------------------------
# 🧩 2. Fill Essential Variable Summary Fields (e.g., Heat Input, Weld Thickness, etc.)
# --------------------------------------------------------

# 🔁 Loop over each field defined in the essential_variable_coordinates list
for label, sheet_index, col_letter, row in essential_variable_coordinates:

    # ✅ Lookup value from the `essential_variable_dict` based on label
    value = essential_variable_dict.get(label, "")

    if value != "":
        # Get the correct worksheet and cell location
        sheet = wb.worksheets[sheet_index]
        col = column_index_from_string(col_letter)
        cell = sheet.cell(row=row, column=col)

        # Write the value
        cell.value = value

        # Apply formatting for readability
        cell.font = Font(name='Arial', size=8)
        cell.alignment = Alignment(
            vertical='center', horizontal='left', wrap_text=True
        )


# --------------------------------------------------------
# ✅ STEP 6: Fill Weld Run Table (Page 2)
# --------------------------------------------------------

# 📌 PURPOSE:
# This step fills out the weld run-wise data table on Page 2 of your WPQR Excel template.
# Each row in this table corresponds to one welding run and includes values like:
# - Current (A)
# - Voltage (V)
# - Travel speed
# - Heat input, etc.

# --------------------------------------------------------

# 🧾 This is the starting row number in the Excel file where run data begins
start_row = 18  # Row 18 on Sheet 2 is where the welding data table starts

# 📌 Column B is where we begin writing (B = 2nd column)
start_col = 2

# 📊 These are the specific columns from the DataFrame to be written into the table
columns_to_fill = [
    "Run",
    "Welding Process",
    "Filler material dia.(mm)",
    "Current (A)",
    "Voltage (V)",
    "Type of current/ polarity",
    "Wire feed speed (m/min)",
    "Travel speed (mm/Sec)",
    "Heat input (KJ/mm)",
    "Mode of metal Transfer"
]

# 🧾 Select the second sheet of the Excel file (index 1)
sheet2 = wb.worksheets[1]  # Sheet 2 = Welding Run Table Page

# 🔁 Loop through each row (welding run) in the DataFrame
for row_idx, (_, row_data) in enumerate(welding_details[columns_to_fill].iterrows()):

    # 🔁 Loop through each column defined in columns_to_fill
    for col_offset, col_name in enumerate(columns_to_fill):

        # 🧮 Calculate the correct cell position (row and column)
        excel_row = start_row + row_idx            # Dynamic row (18 + index)
        excel_col = start_col + col_offset         # Dynamic column (starting at column B)

        # 📥 Get the Excel cell object at the calculated position
        cell = sheet2.cell(row=excel_row, column=excel_col)

        # ✍️ Write the value from the DataFrame into the Excel cell
        cell.value = row_data[col_name]

        # 🎨 Apply font and formatting for readability
        cell.font = Font(name='Arial', size=8)
        cell.alignment = Alignment(
            vertical='center',
            horizontal='left',
            wrap_text=True
        )


# --------------------------------------------------------
# ✅ STEP 7: Save and Download Filled WPQR Excel File
# --------------------------------------------------------

# 🗂️ Define the output file name for the Excel workbook
# You can change this name if you want to give it a version or timestamp
output_file = "WPQR_Filled_Final.xlsx"

# 💾 Save the filled-in Excel workbook to this file name
wb.save(output_file)  # 'wb' is the workbook we populated in earlier steps

# ✅ Inform the user that the file has been saved
print(f"✅ WPQR Excel filled and saved as: {output_file}")

# 📥 Google Colab-specific: Trigger a download of the saved file to your computer
# (this works only inside Google Colab)
files.download(output_file)



✅ Using Template File: WPQR-FORMAT.xlsx
✅ WPQR Excel filled and saved as: WPQR_Filled_Final.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>